[1] Model-free RL : Value Iteration

테이블 기반<br>
	(1-1) Q-Learning: 상태 공간에서 사용되는 표(Q-Table) 기반<br>
	(1-2) SARSA(State Action Reward State Action)<br>
심층 기반<br>
	(1-3) Q-Network: 신경망을 사용하여 Q-값을 근사화하는 모델<br>
	(1-4) DQN(Deep Q-Network): 딥러닝을 활용한 Q-Learning의 발전된 형태<br>
	(1-5) Double DQN: Q-Learning의 과대 평가 문제를 완화하기 위한 개선된 버전<br>
	(1-6) Dueling DQN: 상태 가치와 행동의 중요도를 분리하여 Q-값을 계산하는 모델<br>
	(1-7) DRQN(Deep Recurrent Q-Network): 순차적인 경험을 학습하기 위해 RNN 구조를 포함한 DQN<br>
분포 기반<br>
	(1-8) C51(Categorical DQN 51): Q-값의 분포를 학습하는 DQN 확장<br>
	(1-9) IQN(Implicit Quantile Networks): Q-값의 분포를 세밀하게 조정하는 방식<br>
	(1-10) Rainbow: 여러 DQN 확장(PER, Double DQN, C51 등)을 결합한 통합 알고리즘<br>
소프트 기반<br>
	(1-11) SQL(Soft Q-Learning): 엔트로피를 추가하여 Q값 학습을 안정화하는 방식<br>
리플레이/탐색 기반<br>
	(1-12) PER(Prioritized Experience Replay): 중요한 경험을 우선적으로 학습하는 경험 리플레이 전략<br>
	(1-13) HER(Hindsight Experience Replay): 목표 달성을 학습할 수 있도록 과거 경험을 재사용하는 기법<br>
	(1-14) NoisyNet: 신경망 가중치에 노이즈를 추가해 탐색 효율성을 높이는 방식<br>

In [2]:
########################################################################################################
## (1-1) Q-Learning : 상태 공간에서 사용되는 표(Q-Table) 기반
########################################################################################################

import numpy as np

# ======================================
# 0. 난수 시드 고정 (항상 동일한 결과 보장)
# ======================================
np.random.seed(42)

# ======================================
# 1. 환경 설정 (1차원 선형 월드)
# ======================================
n_states = 5     # 상태(State) 개수: 0,1,2,3,4 (4가 목표 상태)
n_actions = 2    # 행동(Action) 개수: 0=왼쪽, 1=오른쪽

# 상태 전이 및 보상 함수
def step(state, action):
    # 행동이 0이면 왼쪽으로 이동, 1이면 오른쪽으로 이동
    if action == 0:
        next_state = max(0, state - 1)              # 왼쪽 끝(0) 이하로 내려가지 않게 처리
    else:
        next_state = min(n_states - 1, state + 1)   # 오른쪽 끝(4) 이상으로 올라가지 않게 처리

    # 목표 상태(4)에 도달한 경우 보상 +1, 그 외에는 -0.01 패널티
    if next_state == n_states - 1:
        reward = 1.0
        done = True                                 # 목표 도달 → 에피소드 종료
    else:
        reward = -0.01                              # 빨리 도달하도록 작은 음수 보상
        done = False

    return next_state, reward, done                 # 다음 상태, 보상, 종료 여부 반환


# 초기 상태 반환 함수
def reset():
    return 0                                        # 항상 state 0에서 에피소드 시작


# ======================================
# 2. Q-Learning 하이퍼파라미터 설정
# ======================================
alpha = 0.1         # 학습률 (Learning Rate)
gamma = 0.9         # 할인율 (Discount Factor)
epsilon = 1.0       # 초기 탐험 확률 (ε-greedy에서 ε)
epsilon_min = 0.05  # 탐험 최소값
epsilon_decay = 0.995  # 에피소드마다 ε 감소율

n_episodes = 500    # 총 학습 반복(에피소드) 횟수
max_steps = 20      # 한 에피소드에서 최대 step (무한 루프 방지용)

# Q-테이블 초기화: 모든 상태-행동 쌍을 0으로 시작
Q = np.zeros((n_states, n_actions))


# ======================================
# 3. ε-greedy 행동 선택 함수
# ======================================
def choose_action(state, epsilon):
    # 일정 확률 ε로 탐험(랜덤 행동 선택)
    if np.random.rand() < epsilon:
        return np.random.randint(n_actions)
    # 나머지 확률로 현재 Q값이 가장 큰 행동 선택 (exploitation)
    return np.argmax(Q[state])


# ======================================
# 4. Q-Learning 학습 루프
# ======================================
reward_history = []     # 에피소드별 총 보상을 저장할 리스트

print("=== 1차원 선형 월드에서의 Q-Learning 학습 시작 ===")

# 전체 에피소드 반복
for episode in range(1, n_episodes + 1):

    state = reset()     # 매 에피소드마다 초기 상태로 리셋
    total_reward = 0.0  # 에피소드 누적 보상 초기화

    # 한 에피소드 안에서 반복
    for step_idx in range(max_steps):

        # 1) ε-greedy 정책으로 행동 선택
        action = choose_action(state, epsilon)

        # 2) 환경에 행동 적용 → 다음 상태, 보상, 종료 여부 반환
        next_state, reward, done = step(state, action)

        # 3) Q(s,a) 업데이트
        #    TD Target = r + γ * max(Q(s', a'))
        best_next_Q = np.max(Q[next_state])                 # 다음 상태에서의 최대 Q
        td_target = reward + gamma * best_next_Q            # TD Target 계산
        td_error = td_target - Q[state, action]             # TD Error 계산
        Q[state, action] += alpha * td_error                # 학습률 α 반영하여 업데이트

        # 4) 보상 누적
        total_reward += reward

        # 5) 상태 업데이트
        state = next_state

        # 종료 상태이면 반복 중단
        if done:
            break

    # ε 감소 (탐험 → 이용 비중 증가)
    epsilon = max(epsilon_min, epsilon * epsilon_decay)

    reward_history.append(total_reward)

    # 50에피소드마다 최근 50개 평균 보상 출력
    if episode % 50 == 0:
        avg_reward = np.mean(reward_history[-50:])
        print(f"[Episode {episode:4d}] 최근 50 에피소드 평균 리워드 = {avg_reward:.3f},  epsilon = {epsilon:.3f}")

print("\n=== 학습 종료 ===\n")


# ======================================
# 5. 학습된 Q테이블 출력
# ======================================
print("▶ 최종 Q-테이블 (행: 상태, 열: 행동[왼쪽, 오른쪽])")
for s in range(n_states):
    print(f"상태 {s}: {Q[s]}")


# ======================================
# 6. 학습된 최적 정책 출력
# ======================================
action_symbols = {0: "←", 1: "→"}    # 행동을 화살표로 표시

print("\n▶ 학습된 정책(Policy)")

policy_str = ""
for s in range(n_states):
    if s == n_states - 1:            # 마지막 상태는 Goal
        policy_str += " G "
    else:
        best_a = np.argmax(Q[s])     # 각 상태에서 Q값이 가장 큰 행동 선택
        policy_str += f" {action_symbols[best_a]} "

print("상태 0  1  2  3  4")
print("     " + policy_str)


# ======================================
# 7. 학습 결과 테스트 실행 (탐험 없이 greedy만)
# ======================================
print("\n▶ 학습된 정책으로 1회 에피소드 실행 예시")

state = reset()               # 초기 상태
trajectory = [state]          # 방문한 상태 기록

for step_idx in range(max_steps):
    action = np.argmax(Q[state])               # 탐험 없이 항상 최적 행동
    next_state, reward, done = step(state, action)
    trajectory.append(next_state)
    state = next_state
    if done:
        break

print("방문한 상태들:", trajectory)
print("스텝 수:", len(trajectory)-1)
print("마지막 상태가 목표(4)면 학습 성공!")


=== 1차원 선형 월드에서의 Q-Learning 학습 시작 ===
[Episode   50] 최근 50 에피소드 평균 리워드 = 0.601,  epsilon = 0.778
[Episode  100] 최근 50 에피소드 평균 리워드 = 0.860,  epsilon = 0.606
[Episode  150] 최근 50 에피소드 평균 리워드 = 0.918,  epsilon = 0.471
[Episode  200] 최근 50 에피소드 평균 리워드 = 0.946,  epsilon = 0.367
[Episode  250] 최근 50 에피소드 평균 리워드 = 0.953,  epsilon = 0.286
[Episode  300] 최근 50 에피소드 평균 리워드 = 0.956,  epsilon = 0.222
[Episode  350] 최근 50 에피소드 평균 리워드 = 0.961,  epsilon = 0.173
[Episode  400] 최근 50 에피소드 평균 리워드 = 0.965,  epsilon = 0.135
[Episode  450] 최근 50 에피소드 평균 리워드 = 0.964,  epsilon = 0.105
[Episode  500] 최근 50 에피소드 평균 리워드 = 0.965,  epsilon = 0.082

=== 학습 종료 ===

▶ 최종 Q-테이블 (행: 상태, 열: 행동[왼쪽, 오른쪽])
상태 0: [0.62170412 0.7019    ]
상태 1: [0.62170963 0.791     ]
상태 2: [0.70189422 0.89      ]
상태 3: [0.79099209 1.        ]
상태 4: [0. 0.]

▶ 학습된 정책(Policy)
상태 0  1  2  3  4
      →  →  →  →  G 

▶ 학습된 정책으로 1회 에피소드 실행 예시
방문한 상태들: [0, 1, 2, 3, 4]
스텝 수: 4
마지막 상태가 목표(4)면 학습 성공!


In [3]:
########################################################################################################
## (1-2) SARSA(State-Action-Reward-State-Action)
########################################################################################################

import numpy as np

# 난수 시드 고정 (결과 재현성 보장)
np.random.seed(42)

# ======================================
# 1. 환경 설정 (1차원 선형 월드)
# ======================================
n_states = 5     # 상태: 0,1,2,3,4 (4가 목표 상태)
n_actions = 2    # 행동: 0=왼쪽, 1=오른쪽

def step(state, action):
    # 행동이 0이면 왼쪽으로 이동
    if action == 0:
        next_state = max(0, state - 1)              # 왼쪽 끝 이하로 못 가게 처리
    # 행동이 1이면 오른쪽으로 이동
    else:
        next_state = min(n_states - 1, state + 1)   # 오른쪽 끝 이상으로 못 가게 처리

    # 목표 상태에 도달하면 보상 +1
    if next_state == n_states - 1:
        reward = 1.0
        done = True                                 # 목표 도달 → 종료
    # 그 외에는 작은 패널티
    else:
        reward = -0.01
        done = False

    return next_state, reward, done

def reset():
    # 매 에피소드 시작 상태는 항상 0
    return 0


# ======================================
# 2. SARSA 하이퍼파라미터 설정
# ======================================
alpha = 0.1         # 학습률
gamma = 0.9         # 할인율
epsilon = 1.0       # 탐험 비율 시작값 (ε-greedy)
epsilon_min = 0.05  # 최소 탐험값
epsilon_decay = 0.995  # 탐험 감소 비율

n_episodes = 500    # 학습 에피소드 수
max_steps = 20      # 한 에피소드에서 최대 스텝 수

# Q 테이블 초기화 (상태 × 행동)
Q = np.zeros((n_states, n_actions))


# ======================================
# 3. ε-greedy 행동 선택 함수
# ======================================
def choose_action(state, epsilon):
    # ε 확률로 탐험
    if np.random.rand() < epsilon:
        return np.random.randint(n_actions)
    # 1-ε 확률로 현재 Q가 가장 큰 행동 선택
    return np.argmax(Q[state])


# ======================================
# 4. SARSA 학습 루프
#    (On-policy: TD Target에 실제 다음 행동 a' 를 사용)
# ======================================
reward_history = []

print("=== 1차원 선형 월드에서의 SARSA 학습 시작 ===")

for episode in range(1, n_episodes + 1):

    # (1) 에피소드 시작: 상태 초기화
    state = reset()
    total_reward = 0.0

    # (2) 초기 상태에서 첫 행동 선택 (SARSA는 s,a 쌍으로 시작)
    action = choose_action(state, epsilon)

    for step_idx in range(max_steps):

        # (3) 현재 상태 s 에서 행동 a 수행
        next_state, reward, done = step(state, action)

        # (4) 다음 상태 s' 에서 다음 행동 a' 선택 (ε-greedy)
        #     SARSA의 핵심: 여기서도 같은 정책(ε-greedy)을 사용
        if not done:
            next_action = choose_action(next_state, epsilon)
        else:
            next_action = None  # 종료 상태에서는 의미 없음

        # (5) SARSA TD Target 계산
        #     - done 이면 다음 상태의 Q는 0
        #     - 아니면 Q(s', a') 사용
        if done:
            td_target = reward                         # 마지막 상태 → 미래 보상 없음
        else:
            td_target = reward + gamma * Q[next_state, next_action]

        # (6) TD Error 및 Q 업데이트
        td_error = td_target - Q[state, action]
        Q[state, action] += alpha * td_error

        # (7) 보상 누적
        total_reward += reward

        # (8) 상태와 행동을 다음 시점으로 이동
        state = next_state
        action = next_action if not done else action  # done이면 action은 더 안 쓰지만 형식상 유지

        # (9) 종료 상태면 에피소드 정리
        if done:
            break

    # (10) 에피소드 종료 후 ε 감소
    epsilon = max(epsilon_min, epsilon * epsilon_decay)

    reward_history.append(total_reward)

    # 50 에피소드마다 로그 출력
    if episode % 50 == 0:
        avg_reward = np.mean(reward_history[-50:])
        print(f"[Episode {episode:4d}] 최근 50 에피소드 평균 리워드 = {avg_reward:.3f}, epsilon = {epsilon:.3f}")

print("\n=== 학습 종료 ===\n")


# ======================================
# 5. 학습된 Q-테이블 출력
# ======================================
print("▶ 최종 Q-테이블 (행: 상태, 열: 행동[←,→])")
for s in range(n_states):
    print(f"상태 {s}: {Q[s]}")


# ======================================
# 6. 학습된 정책 출력
# ======================================
action_symbols = {0: "←", 1: "→"}

print("\n▶ 학습된 정책(Policy)")

policy_str = ""
for s in range(n_states):
    if s == n_states - 1:
        policy_str += " G "
    else:
        best_a = np.argmax(Q[s])
        policy_str += f" {action_symbols[best_a]} "

print("상태 0  1  2  3  4")
print("     " + policy_str)


# ======================================
# 7. 학습된 정책으로 1회 테스트 실행 (탐험 없이 greedy만)
# ======================================
print("\n▶ 학습된 정책으로 1회 에피소드 실행 예시")

state = reset()
trajectory = [state]

for step_idx in range(max_steps):

    # 테스트에서는 탐험 없이 항상 greedy 정책 사용
    action = np.argmax(Q[state])
    next_state, reward, done = step(state, action)

    trajectory.append(next_state)
    state = next_state

    if done:
        break

print("방문한 상태들:", trajectory)
print("스텝 수:", len(trajectory)-1)
print("마지막 상태가 목표(4)면 학습 성공!")


=== 1차원 선형 월드에서의 SARSA 학습 시작 ===
[Episode   50] 최근 50 에피소드 평균 리워드 = 0.518, epsilon = 0.778
[Episode  100] 최근 50 에피소드 평균 리워드 = 0.884, epsilon = 0.606
[Episode  150] 최근 50 에피소드 평균 리워드 = 0.913, epsilon = 0.471
[Episode  200] 최근 50 에피소드 평균 리워드 = 0.949, epsilon = 0.367
[Episode  250] 최근 50 에피소드 평균 리워드 = 0.953, epsilon = 0.286
[Episode  300] 최근 50 에피소드 평균 리워드 = 0.958, epsilon = 0.222
[Episode  350] 최근 50 에피소드 평균 리워드 = 0.961, epsilon = 0.173
[Episode  400] 최근 50 에피소드 평균 리워드 = 0.965, epsilon = 0.135
[Episode  450] 최근 50 에피소드 평균 리워드 = 0.965, epsilon = 0.105
[Episode  500] 최근 50 에피소드 평균 리워드 = 0.964, epsilon = 0.082

=== 학습 종료 ===

▶ 최종 Q-테이블 (행: 상태, 열: 행동[←,→])
상태 0: [0.56804536 0.67414501]
상태 1: [0.55285797 0.77460095]
상태 2: [0.61751656 0.88120465]
상태 3: [0.7167551 1.       ]
상태 4: [0. 0.]

▶ 학습된 정책(Policy)
상태 0  1  2  3  4
      →  →  →  →  G 

▶ 학습된 정책으로 1회 에피소드 실행 예시
방문한 상태들: [0, 1, 2, 3, 4]
스텝 수: 4
마지막 상태가 목표(4)면 학습 성공!


In [4]:

########################################################################################################
## (1-3) Q-Network : 신경망을 사용하여 Q-값을 근사화하는 모델
########################################################################################################

import numpy as np  # 수치 계산을 위한 numpy

# 난수 시드 고정 (실행할 때마다 같은 결과를 얻기 위함)
np.random.seed(42)

# ======================================
# 1. 환경 설정 (1차원 선형 월드)
# ======================================
n_states = 5     # 상태 개수: 0,1,2,3,4 (4가 목표 상태)
n_actions = 2    # 행동 개수: 0=왼쪽, 1=오른쪽

def step(state, action):
    # action이 0이면 왼쪽으로 이동
    if action == 0:
        next_state = max(0, state - 1)              # 왼쪽 끝(0) 아래로 내려가지 않도록 제한
    # action이 1이면 오른쪽으로 이동
    else:
        next_state = min(n_states - 1, state + 1)   # 오른쪽 끝(4) 위로 넘어가지 않도록 제한

    # 목표 상태(4)에 도달한 경우
    if next_state == n_states - 1:
        reward = 1.0                                # 목표 도달 보상 +1
        done = True                                 # 에피소드 종료
    # 그 외의 경우
    else:
        reward = -0.01                              # 이동마다 작은 패널티 부여
        done = False                                # 에피소드 계속 진행

    return next_state, reward, done                 # 다음 상태, 보상, 종료 여부 반환

def reset():
    # 에피소드 시작 시 항상 상태 0에서 시작
    return 0

def state_to_onehot(state):
    # 상태를 one-hot 벡터(1 x n_states)로 변환
    x = np.zeros((1, n_states), dtype=np.float32)   # 1행 n_states열의 0 벡터 생성
    x[0, state] = 1.0                               # 해당 상태 인덱스 위치만 1로 설정
    return x

# ======================================
# 2. Q-Network 파라미터 설정 (순수 NumPy로 신경망 구성)
# ======================================
input_dim = n_states      # 입력 차원: 상태를 one-hot으로 표현하므로 n_states
hidden_dim = 16           # 은닉층 노드 수 (임의로 16으로 설정)
output_dim = n_actions    # 출력 차원: 각 행동에 대한 Q값 2개

# 가중치와 편향을 작은 값으로 초기화
W1 = 0.1 * np.random.randn(input_dim, hidden_dim)   # 1층 가중치 (입력 → 은닉)
b1 = np.zeros((1, hidden_dim))                      # 1층 편향
W2 = 0.1 * np.random.randn(hidden_dim, output_dim)  # 2층 가중치 (은닉 → 출력)
b2 = np.zeros((1, output_dim))                      # 2층 편향

def relu(x):
    # ReLU 활성화 함수: 0보다 작으면 0, 크면 그대로
    return np.maximum(0, x)

def relu_deriv(x):
    # ReLU의 도함수: x>0이면 1, 아니면 0
    return (x > 0).astype(np.float32)

def forward(x):
    # 신경망 순전파: 입력 x에 대해 Q값을 계산
    # x: (1, input_dim) 형태의 one-hot 상태 벡터
    z1 = x @ W1 + b1                # 1층 선형 결합 (1 x hidden_dim)
    a1 = relu(z1)                   # ReLU 활성화 (1 x hidden_dim)
    z2 = a1 @ W2 + b2               # 2층 선형 결합 (1 x output_dim)
    q_values = z2                   # 출력층: 각 행동에 대한 Q값
    return z1, a1, q_values         # 역전파 위해 중간값도 반환

# ======================================
# 3. 하이퍼파라미터 설정 (Q-Learning과 동일 구조)
# ======================================
gamma = 0.9         # 할인율
epsilon = 1.0       # ε-greedy에서 탐험 비율 시작값
epsilon_min = 0.05  # ε의 최소값
epsilon_decay = 0.995  # 에피소드마다 ε 감소 비율
learning_rate = 0.01   # 신경망 파라미터 학습률 (gradient descent step 크기)

n_episodes = 500    # 총 학습 에피소드 수
max_steps = 20      # 한 에피소드에서 최대 스텝 수

# ======================================
# 4. ε-greedy 정책으로 행동 선택 함수
# ======================================
def choose_action(state, epsilon):
    # ε 확률로 랜덤 탐험
    if np.random.rand() < epsilon:
        return np.random.randint(n_actions)         # 0 또는 1 중 랜덤 선택

    # 1-ε 확률로 Q값이 최대인 행동 선택
    x = state_to_onehot(state)                      # 상태를 one-hot으로 변환
    _, _, q_values = forward(x)                     # Q값 계산
    action = int(np.argmax(q_values, axis=1)[0])    # 가장 큰 Q값을 주는 행동 인덱스 선택
    return action

# ======================================
# 5. Q-Network 기반 Q-Learning 학습 루프
# ======================================
reward_history = []  # 에피소드별 총 보상을 저장할 리스트

print("=== 1차원 선형 월드에서의 Q-Network(NumPy) 학습 시작 ===")

for episode in range(1, n_episodes + 1):

    state = reset()          # 에피소드 시작 상태 초기화
    total_reward = 0.0       # 에피소드 누적 보상 초기화

    for step_idx in range(max_steps):

        # (1) ε-greedy 정책으로 행동 선택
        action = choose_action(state, epsilon)

        # (2) 선택한 행동을 환경에 적용 → 다음 상태, 보상, 종료 여부 반환
        next_state, reward, done = step(state, action)

        # (3) 현재 상태와 다음 상태를 one-hot 벡터로 변환
        x = state_to_onehot(state)                    # 현재 상태 (1 x n_states)
        x_next = state_to_onehot(next_state)          # 다음 상태 (1 x n_states)

        # (4) 현재 상태에서의 Q값 계산 (순전파)
        z1, a1, q_values = forward(x)                 # q_values: (1 x n_actions)
        q_value = q_values[0, action]                 # 선택한 행동에 대한 Q값 (스칼라)

        # (5) 다음 상태에서의 최대 Q값 계산 (Q-Learning 방식)
        _, _, q_values_next = forward(x_next)         # 다음 상태에서의 Q값들
        max_next_q = float(np.max(q_values_next, axis=1)[0])  # 그 중 최대값

        # (6) TD Target 계산
        if done:
            target = reward                           # 종료 상태면 미래 보상 없음
        else:
            target = reward + gamma * max_next_q      # r + γ * max_a' Q(s', a')

        # (7) TD Error 계산 (예측 - 타깃)
        #     손실 L = 0.5 * (q_value - target)^2 라고 두면
        #     dL/d(q_value) = (q_value - target)
        td_error = q_value - target                   # 스칼라

        # (8) 출력층(z2)에서의 gradient 계산
        #     z2: (1 x n_actions), 그 중 action 인덱스만 영향을 받음
        dL_dz2 = np.zeros_like(q_values)              # (1 x n_actions) 0으로 초기화
        dL_dz2[0, action] = td_error                  # 선택한 행동 위치에만 td_error 반영

        # (9) 2층 가중치와 편향에 대한 gradient
        #     dL/dW2 = a1^T @ dL_dz2  (hidden_dim x 1) x (1 x n_actions) = (hidden_dim x n_actions)
        dW2 = a1.T @ dL_dz2                           # (hidden_dim x n_actions)
        db2 = dL_dz2                                  # (1 x n_actions)

        # (10) 1층으로 gradient 전파
        #      dL/da1 = dL_dz2 @ W2^T  → (1 x n_actions) @ (n_actions x hidden_dim) = (1 x hidden_dim)
        dL_da1 = dL_dz2 @ W2.T                        # (1 x hidden_dim)
        #      ReLU 미분: dz1 = dL/da1 * ReLU'(z1)
        dL_dz1 = dL_da1 * relu_deriv(z1)              # (1 x hidden_dim)

        # (11) 1층 가중치와 편향에 대한 gradient
        #      dL/dW1 = x^T @ dL_dz1  (input_dim x 1) x (1 x hidden_dim) = (input_dim x hidden_dim)
        dW1 = x.T @ dL_dz1                            # (input_dim x hidden_dim)
        db1 = dL_dz1                                  # (1 x hidden_dim)

        # (12) 파라미터 업데이트 (경사하강법: W ← W - η * dW)
        W2 -= learning_rate * dW2
        b2 -= learning_rate * db2
        W1 -= learning_rate * dW1
        b1 -= learning_rate * db1

        # (13) 보상 누적
        total_reward += reward

        # (14) 상태를 다음 상태로 업데이트
        state = next_state

        # (15) 종료 상태면 에피소드 종료
        if done:
            break

    # (16) 에피소드 종료 후 epsilon 감소 (탐험 비율을 점점 줄임)
    epsilon = max(epsilon_min, epsilon * epsilon_decay)

    # (17) 에피소드별 총 보상을 기록
    reward_history.append(total_reward)

    # (18) 50 에피소드마다 최근 50개 평균 리워드와 현재 epsilon 출력
    if episode % 50 == 0:
        avg_reward = np.mean(reward_history[-50:])
        print(f"[Episode {episode:4d}] 최근 50 에피소드 평균 리워드 = {avg_reward:.3f}, epsilon = {epsilon:.3f}")

print("\n=== 학습 종료 ===\n")

# ======================================
# 6. 학습된 Q-Network로부터 '근사 Q-테이블' 출력
# ======================================
print("▶ 근사 Q-테이블 (행: 상태, 열: 행동[←,→])")

for s in range(n_states):
    x = state_to_onehot(s)                # 상태 s를 one-hot으로 변환
    _, _, q_vals = forward(x)             # Q값 계산
    q_vals_row = q_vals[0]                # (1 x n_actions) → (n_actions,)
    print(f"상태 {s}: {q_vals_row}")

# ======================================
# 7. 학습된 정책(Policy) 확인 (greedy 정책)
# ======================================
action_symbols = {0: "←", 1: "→"}         # 행동 인덱스를 화살표로 표현하기 위한 매핑

print("\n▶ 학습된 정책(Policy)")

policy_str = ""
for s in range(n_states):
    if s == n_states - 1:
        policy_str += " G "               # 목표 상태는 G로 표시
    else:
        x = state_to_onehot(s)
        _, _, q_vals = forward(x)
        best_action = int(np.argmax(q_vals, axis=1)[0])
        policy_str += f" {action_symbols[best_action]} "

print("상태 0  1  2  3  4")
print("     " + policy_str)

# ======================================
# 8. 학습된 정책으로 1회 테스트 실행 (탐험 없이 greedy만 사용)
# ======================================
print("\n▶ 학습된 정책으로 1회 에피소드 실행 예시")

state = reset()                       # 초기 상태 0
trajectory = [state]                  # 방문한 상태들을 저장할 리스트

for step_idx in range(max_steps):

    x = state_to_onehot(state)        # 현재 상태를 one-hot으로 변환
    _, _, q_vals = forward(x)         # Q값 계산
    action = int(np.argmax(q_vals))   # 탐험 없이 항상 greedy 행동 선택

    next_state, reward, done = step(state, action)  # 환경에 행동 적용
    trajectory.append(next_state)     # 방문한 상태 기록
    state = next_state                # 상태 업데이트

    if done:
        break                         # 목표 도달 시 종료

print("방문한 상태들:", trajectory)
print("스텝 수:", len(trajectory) - 1)
print("마지막 상태가 목표(4)면 학습 성공!")


=== 1차원 선형 월드에서의 Q-Network(NumPy) 학습 시작 ===
[Episode   50] 최근 50 에피소드 평균 리워드 = 0.634, epsilon = 0.778
[Episode  100] 최근 50 에피소드 평균 리워드 = 0.817, epsilon = 0.606
[Episode  150] 최근 50 에피소드 평균 리워드 = 0.939, epsilon = 0.471
[Episode  200] 최근 50 에피소드 평균 리워드 = 0.953, epsilon = 0.367
[Episode  250] 최근 50 에피소드 평균 리워드 = 0.951, epsilon = 0.286
[Episode  300] 최근 50 에피소드 평균 리워드 = 0.956, epsilon = 0.222
[Episode  350] 최근 50 에피소드 평균 리워드 = 0.964, epsilon = 0.173
[Episode  400] 최근 50 에피소드 평균 리워드 = 0.962, epsilon = 0.135
[Episode  450] 최근 50 에피소드 평균 리워드 = 0.963, epsilon = 0.105
[Episode  500] 최근 50 에피소드 평균 리워드 = 0.965, epsilon = 0.082

=== 학습 종료 ===

▶ 근사 Q-테이블 (행: 상태, 열: 행동[←,→])
상태 0: [0.61391433 0.71010196]
상태 1: [0.58996149 0.71413935]
상태 2: [0.57754957 0.76342132]
상태 3: [0.59451489 0.96636278]
상태 4: [0.63171271 0.72202013]

▶ 학습된 정책(Policy)
상태 0  1  2  3  4
      →  →  →  →  G 

▶ 학습된 정책으로 1회 에피소드 실행 예시
방문한 상태들: [0, 1, 2, 3, 4]
스텝 수: 4
마지막 상태가 목표(4)면 학습 성공!


In [5]:

########################################################################################################
## (1-4) DQN(Deep Q-Network) : 딥러닝을 활용한 Q-Learning의 발전된 형태 (2013, 2015)
########################################################################################################

import numpy as np  # 수치 계산을 위한 numpy

# 난수 시드 고정 (재현 가능한 결과를 위해)
np.random.seed(42)

# ======================================
# 1. 환경 설정 (1차원 선형 월드)
# ======================================
n_states = 5     # 상태 개수: 0,1,2,3,4 (4가 목표 상태)
n_actions = 2    # 행동 개수: 0=왼쪽, 1=오른쪽

def step(state, action):
    # action이 0이면 왼쪽으로 이동
    if action == 0:
        next_state = max(0, state - 1)              # 왼쪽 끝(0) 아래로 내려가지 않도록 제한
    # action이 1이면 오른쪽으로 이동
    else:
        next_state = min(n_states - 1, state + 1)   # 오른쪽 끝(4) 위로 넘어가지 않도록 제한

    # 목표 상태(4)에 도달한 경우
    if next_state == n_states - 1:
        reward = 1.0                                # 목표 도달 보상 +1
        done = True                                 # 에피소드 종료
    # 그 외의 경우
    else:
        reward = -0.01                              # 이동마다 작은 패널티 부여
        done = False                                # 에피소드 계속 진행

    return next_state, reward, done                 # 다음 상태, 보상, 종료 여부 반환

def reset():
    # 에피소드 시작 시 항상 상태 0에서 시작
    return 0

def state_to_onehot(state):
    # 상태를 one-hot 벡터(1 x n_states)로 변환
    x = np.zeros((1, n_states), dtype=np.float32)   # 1행 n_states열의 0 벡터 생성
    x[0, state] = 1.0                               # 해당 상태 인덱스 위치만 1로 설정
    return x

# ======================================
# 2. DQN용 Q-Network / Target Network 파라미터 정의
#    - 간단한 2층 완전연결 신경망 사용
# ======================================
input_dim = n_states      # 입력 차원: 상태를 one-hot으로 표현하므로 n_states
hidden_dim = 16           # 은닉층 노드 수 (임의로 16으로 설정)
output_dim = n_actions    # 출력 차원: 각 행동에 대한 Q값 2개

# 온라인 네트워크(Online Q-Network) 파라미터
W1 = 0.1 * np.random.randn(input_dim, hidden_dim)   # 1층 가중치 (입력 → 은닉)
b1 = np.zeros((1, hidden_dim))                      # 1층 편향
W2 = 0.1 * np.random.randn(hidden_dim, output_dim)  # 2층 가중치 (은닉 → 출력)
b2 = np.zeros((1, output_dim))                      # 2층 편향

# 타깃 네트워크(Target Q-Network) 파라미터 (초기에는 동일하게 복사)
W1_tgt = W1.copy()
b1_tgt = b1.copy()
W2_tgt = W2.copy()
b2_tgt = b2.copy()

def relu(x):
    # ReLU 활성화 함수: 0보다 작으면 0, 크면 그대로
    return np.maximum(0, x)

def relu_deriv(x):
    # ReLU의 도함수: x>0이면 1, 아니면 0
    return (x > 0).astype(np.float32)

def forward(x, W1, b1, W2, b2):
    # 신경망 순전파: 입력 x에 대해 Q값을 계산
    # x: (배치크기, input_dim) 형태의 상태(one-hot) 벡터
    z1 = x @ W1 + b1                # 1층 선형 결합 (배치 x hidden_dim)
    a1 = relu(z1)                   # ReLU 활성화 (배치 x hidden_dim)
    z2 = a1 @ W2 + b2               # 2층 선형 결합 (배치 x output_dim)
    q_values = z2                   # 출력층: 각 행동에 대한 Q값
    return z1, a1, q_values         # 역전파를 위해 중간값도 함께 반환

# ======================================
# 3. DQN 하이퍼파라미터 설정
# ======================================
gamma = 0.9             # 할인율
epsilon = 1.0           # ε-greedy에서 탐험 비율 시작값
epsilon_min = 0.05      # ε의 최소값
epsilon_decay = 0.995   # 에피소드마다 ε 감소 비율
learning_rate = 0.01    # 신경망 파라미터 학습률(경사하강 step 크기)

n_episodes = 500        # 총 학습 에피소드 수
max_steps = 20          # 한 에피소드에서 최대 스텝 수

buffer_capacity = 1000  # 리플레이 버퍼 최대 크기
batch_size = 32         # 미니배치 크기
warmup_steps = 100      # 최소 이 정도 샘플이 쌓인 후부터 학습 시작
target_update_freq = 20 # 타깃 네트워크를 몇 에피소드마다 한 번씩 갱신할지

# ======================================
# 4. 리플레이 버퍼 구현 (간단한 리스트 버퍼)
# ======================================
replay_buffer = []  # (state, action, reward, next_state, done) 튜플을 저장

def add_to_buffer(state, action, reward, next_state, done):
    # 버퍼에 새 transition 추가
    if len(replay_buffer) >= buffer_capacity:
        replay_buffer.pop(0)  # 가장 오래된 데이터 삭제 (FIFO)
    replay_buffer.append((state, action, reward, next_state, done))

def sample_from_buffer(batch_size):
    # 버퍼에서 랜덤하게 batch_size 개 샘플 추출
    indices = np.random.choice(len(replay_buffer), size=batch_size, replace=False)
    batch = [replay_buffer[i] for i in indices]
    return batch

# ======================================
# 5. ε-greedy 정책으로 행동 선택 함수 (Online Network 사용)
# ======================================
def choose_action(state, epsilon):
    # ε 확률로 랜덤 탐험
    if np.random.rand() < epsilon:
        return np.random.randint(n_actions)         # 0 또는 1 중 랜덤 선택

    # 1-ε 확률로 Q값이 최대인 행동 선택 (Online Network 기준)
    x = state_to_onehot(state)                      # 상태를 one-hot으로 변환
    _, _, q_values = forward(x, W1, b1, W2, b2)     # Q값 계산
    action = int(np.argmax(q_values, axis=1)[0])    # 가장 큰 Q값을 주는 행동 인덱스 선택
    return action

# ======================================
# 6. DQN 학습 함수: 리플레이 버퍼에서 미니배치 샘플 → 경사하강
# ======================================
def train_dqn(batch_size):
    global W1, b1, W2, b2   # 전역 파라미터 사용

    # 버퍼에서 미니배치 샘플 추출
    batch = sample_from_buffer(batch_size)

    # 배치를 각 성분별로 나누어 numpy 배열로 변환
    states      = np.array([s for (s, a, r, ns, d) in batch], dtype=np.int32)
    actions     = np.array([a for (s, a, r, ns, d) in batch], dtype=np.int32)
    rewards     = np.array([r for (s, a, r, ns, d) in batch], dtype=np.float32)
    next_states = np.array([ns for (s, a, r, ns, d) in batch], dtype=np.int32)
    dones       = np.array([d for (s, a, r, ns, d) in batch], dtype=np.bool_)

    # 상태, 다음 상태를 one-hot 벡터로 변환
    X      = np.vstack([state_to_onehot(s)  for s in states])      # (B, n_states)
    X_next = np.vstack([state_to_onehot(ns) for ns in next_states])# (B, n_states)

    # 온라인 네트워크로 현재 상태의 Q값 계산
    z1, a1, q_values = forward(X, W1, b1, W2, b2)                  # q_values: (B, n_actions)

    # 타깃 네트워크로 다음 상태의 Q값 계산
    _, _, q_values_next_tgt = forward(X_next, W1_tgt, b1_tgt, W2_tgt, b2_tgt)  # (B, n_actions)

    # 다음 상태에서의 최대 Q값(max_a' Q_target(s', a')) 추출
    max_next_q = np.max(q_values_next_tgt, axis=1)                  # (B,)

    # TD Target 계산: done이면 미래 보상 없음
    targets = rewards.copy()
    not_dones = (~dones)
    targets[not_dones] += gamma * max_next_q[not_dones]

    # 예측 Q 중에서 실제로 선택한 행동의 Q값만 사용
    # dL/d(q_pred) = (q_pred - target) / B  (MSE 기준)
    B = batch_size
    dL_dz2 = np.zeros_like(q_values)                                # (B, n_actions)
    q_pred_selected = q_values[np.arange(B), actions]               # (B,)
    td_errors = (q_pred_selected - targets) / B                     # (B,)

    # 선택한 행동 위치에만 gradient 반영
    dL_dz2[np.arange(B), actions] = td_errors                       # (B, n_actions)

    # 2층(출력층) 가중치/편향에 대한 gradient
    dW2 = a1.T @ dL_dz2                                             # (hidden_dim x n_actions)
    db2 = np.sum(dL_dz2, axis=0, keepdims=True)                     # (1 x n_actions)

    # 1층으로 gradient 전파
    dL_da1 = dL_dz2 @ W2.T                                          # (B x hidden_dim)
    dL_dz1 = dL_da1 * relu_deriv(z1)                                # (B x hidden_dim)

    # 1층 가중치/편향에 대한 gradient
    dW1 = X.T @ dL_dz1                                              # (input_dim x hidden_dim)
    db1 = np.sum(dL_dz1, axis=0, keepdims=True)                     # (1 x hidden_dim)

    # 파라미터 업데이트 (경사하강법)
    W2 -= learning_rate * dW2
    b2 -= learning_rate * db2
    W1 -= learning_rate * dW1
    b1 -= learning_rate * db1

# ======================================
# 7. DQN 학습 루프
# ======================================
reward_history = []  # 에피소드별 총 보상을 저장할 리스트
total_steps = 0      # 전체 스텝 수(옵션, 여기서는 로그용으로만 사용)

print("=== 1차원 선형 월드에서의 DQN(NumPy) 학습 시작 ===")

for episode in range(1, n_episodes + 1):

    state = reset()          # 에피소드 시작 상태 초기화
    total_reward = 0.0       # 에피소드 누적 보상 초기화

    for step_idx in range(max_steps):

        total_steps += 1

        # (1) ε-greedy 정책으로 행동 선택 (Online Network 기준)
        action = choose_action(state, epsilon)

        # (2) 선택한 행동을 환경에 적용 → 다음 상태, 보상, 종료 여부 반환
        next_state, reward, done = step(state, action)

        # (3) 리플레이 버퍼에 transition 저장
        add_to_buffer(state, action, reward, next_state, done)

        # (4) 일정 step 이상 쌓여야 학습 시작 (warmup_steps 이후)
        if len(replay_buffer) >= max(batch_size, warmup_steps):
            train_dqn(batch_size)

        # (5) 보상 누적
        total_reward += reward

        # (6) 상태를 다음 상태로 업데이트
        state = next_state

        # (7) 종료 상태면 에피소드 종료
        if done:
            break

    # (8) 에피소드 종료 후 epsilon 감소 (탐험 비율을 점점 줄임)
    epsilon = max(epsilon_min, epsilon * epsilon_decay)

    # (9) 에피소드별 총 보상을 기록
    reward_history.append(total_reward)

    # (10) 일정 에피소드마다 타깃 네트워크 파라미터 동기화
    if episode % target_update_freq == 0:
        W1_tgt = W1.copy()
        b1_tgt = b1.copy()
        W2_tgt = W2.copy()
        b2_tgt = b2.copy()

    # (11) 50 에피소드마다 최근 50개 평균 리워드와 현재 epsilon 출력
    if episode % 50 == 0:
        avg_reward = np.mean(reward_history[-50:])
        print(f"[Episode {episode:4d}] 최근 50 에피소드 평균 리워드 = {avg_reward:.3f}, epsilon = {epsilon:.3f}")

print("\n=== 학습 종료 ===\n")

# ======================================
# 8. 학습된 Online Q-Network로부터 '근사 Q-테이블' 출력
# ======================================
print("▶ 근사 Q-테이블 (행: 상태, 열: 행동[←,→])")

for s in range(n_states):
    x = state_to_onehot(s)                        # 상태 s를 one-hot으로 변환
    _, _, q_vals = forward(x, W1, b1, W2, b2)     # Q값 계산
    q_vals_row = q_vals[0]                        # (1 x n_actions) → (n_actions,)
    print(f"상태 {s}: {q_vals_row}")

# ======================================
# 9. 학습된 정책(Policy) 확인 (greedy 정책)
# ======================================
action_symbols = {0: "←", 1: "→"}                 # 행동 인덱스를 화살표로 표현하기 위한 매핑

print("\n▶ 학습된 정책(Policy)")

policy_str = ""
for s in range(n_states):
    if s == n_states - 1:
        policy_str += " G "                       # 목표 상태는 G로 표시
    else:
        x = state_to_onehot(s)
        _, _, q_vals = forward(x, W1, b1, W2, b2)
        best_action = int(np.argmax(q_vals, axis=1)[0])
        policy_str += f" {action_symbols[best_action]} "

print("상태 0  1  2  3  4")
print("     " + policy_str)

# ======================================
# 10. 학습된 정책으로 1회 테스트 실행 (탐험 없이 greedy만 사용)
# ======================================
print("\n▶ 학습된 정책으로 1회 에피소드 실행 예시")

state = reset()                       # 초기 상태 0
trajectory = [state]                  # 방문한 상태들을 저장할 리스트

for step_idx in range(max_steps):

    x = state_to_onehot(state)        # 현재 상태를 one-hot으로 변환
    _, _, q_vals = forward(x, W1, b1, W2, b2)  # Q값 계산
    action = int(np.argmax(q_vals))   # 탐험 없이 항상 greedy 행동 선택

    next_state, reward, done = step(state, action)  # 환경에 행동 적용
    trajectory.append(next_state)     # 방문한 상태 기록
    state = next_state                # 상태 업데이트

    if done:
        break                         # 목표 도달 시 종료

print("방문한 상태들:", trajectory)
print("스텝 수:", len(trajectory) - 1)
print("마지막 상태가 목표(4)면 학습 성공!")


=== 1차원 선형 월드에서의 DQN(NumPy) 학습 시작 ===
[Episode   50] 최근 50 에피소드 평균 리워드 = 0.691, epsilon = 0.778
[Episode  100] 최근 50 에피소드 평균 리워드 = 0.859, epsilon = 0.606
[Episode  150] 최근 50 에피소드 평균 리워드 = 0.916, epsilon = 0.471
[Episode  200] 최근 50 에피소드 평균 리워드 = 0.951, epsilon = 0.367
[Episode  250] 최근 50 에피소드 평균 리워드 = 0.955, epsilon = 0.286
[Episode  300] 최근 50 에피소드 평균 리워드 = 0.954, epsilon = 0.222
[Episode  350] 최근 50 에피소드 평균 리워드 = 0.960, epsilon = 0.173
[Episode  400] 최근 50 에피소드 평균 리워드 = 0.964, epsilon = 0.135
[Episode  450] 최근 50 에피소드 평균 리워드 = 0.967, epsilon = 0.105
[Episode  500] 최근 50 에피소드 평균 리워드 = 0.966, epsilon = 0.082

=== 학습 종료 ===

▶ 근사 Q-테이블 (행: 상태, 열: 행동[←,→])
상태 0: [0.55878423 0.68169877]
상태 1: [0.53764078 0.68297671]
상태 2: [0.52598826 0.73224926]
상태 3: [0.54513745 0.96248031]
상태 4: [0.57992423 0.69895181]

▶ 학습된 정책(Policy)
상태 0  1  2  3  4
      →  →  →  →  G 

▶ 학습된 정책으로 1회 에피소드 실행 예시
방문한 상태들: [0, 1, 2, 3, 4]
스텝 수: 4
마지막 상태가 목표(4)면 학습 성공!


In [6]:

########################################################################################################
## (1-5) Double DQN : Q-Learning의 과대 평가 문제를 완화하기 위한 개선된 버전
########################################################################################################

import numpy as np  # 수치 계산을 위한 numpy

# 난수 시드 고정 (재현 가능한 결과를 위해)
np.random.seed(42)

# ======================================
# 1. 환경 설정 (1차원 선형 월드)
# ======================================
n_states = 5     # 상태 개수: 0,1,2,3,4 (4가 목표 상태)
n_actions = 2    # 행동 개수: 0=왼쪽, 1=오른쪽

def step(state, action):
    # action이 0이면 왼쪽으로 이동
    if action == 0:
        next_state = max(0, state - 1)              # 왼쪽 끝(0) 아래로 내려가지 않도록 제한
    # action이 1이면 오른쪽으로 이동
    else:
        next_state = min(n_states - 1, state + 1)   # 오른쪽 끝(4) 위로 넘어가지 않도록 제한

    # 목표 상태(4)에 도달한 경우
    if next_state == n_states - 1:
        reward = 1.0                                # 목표 도달 보상 +1
        done = True                                 # 에피소드 종료
    # 그 외의 경우
    else:
        reward = -0.01                              # 이동마다 작은 패널티 부여
        done = False                                # 에피소드 계속 진행

    return next_state, reward, done                 # 다음 상태, 보상, 종료 여부 반환

def reset():
    # 에피소드 시작 시 항상 상태 0에서 시작
    return 0

def state_to_onehot(state):
    # 상태를 one-hot 벡터(1 x n_states)로 변환
    x = np.zeros((1, n_states), dtype=np.float32)   # 1행 n_states열의 0 벡터 생성
    x[0, state] = 1.0                               # 해당 상태 인덱스 위치만 1로 설정
    return x

# ======================================
# 2. Double DQN용 Q-Network / Target Network 파라미터 정의
#    - 간단한 2층 완전연결 신경망 사용
# ======================================
input_dim = n_states      # 입력 차원: one-hot 상태
hidden_dim = 16           # 은닉층 노드 수
output_dim = n_actions    # 출력 차원: 각 행동에 대한 Q값 2개

# 온라인 네트워크(Online Q-Network) 파라미터
W1 = 0.1 * np.random.randn(input_dim, hidden_dim)   # 1층 가중치 (입력 → 은닉)
b1 = np.zeros((1, hidden_dim))                      # 1층 편향
W2 = 0.1 * np.random.randn(hidden_dim, output_dim)  # 2층 가중치 (은닉 → 출력)
b2 = np.zeros((1, output_dim))                      # 2층 편향

# 타깃 네트워크(Target Q-Network) 파라미터 (초기에는 동일하게 복사)
W1_tgt = W1.copy()
b1_tgt = b1.copy()
W2_tgt = W2.copy()
b2_tgt = b2.copy()

def relu(x):
    # ReLU 활성화 함수: 0보다 작으면 0, 크면 그대로
    return np.maximum(0, x)

def relu_deriv(x):
    # ReLU의 도함수: x>0이면 1, 아니면 0
    return (x > 0).astype(np.float32)

def forward(x, W1, b1, W2, b2):
    # 신경망 순전파: 입력 x에 대해 Q값을 계산
    # x: (배치크기, input_dim) 형태의 상태(one-hot) 벡터
    z1 = x @ W1 + b1                # 1층 선형 결합 (배치 x hidden_dim)
    a1 = relu(z1)                   # ReLU 활성화 (배치 x hidden_dim)
    z2 = a1 @ W2 + b2               # 2층 선형 결합 (배치 x output_dim)
    q_values = z2                   # 출력층: 각 행동에 대한 Q값
    return z1, a1, q_values         # 역전파를 위해 중간값도 함께 반환

# ======================================
# 3. Double DQN 하이퍼파라미터 설정
# ======================================
gamma = 0.9             # 할인율
epsilon = 1.0           # ε-greedy에서 탐험 비율 시작값
epsilon_min = 0.05      # ε의 최소값
epsilon_decay = 0.995   # 에피소드마다 ε 감소 비율
learning_rate = 0.01    # 신경망 파라미터 학습률(경사하강 step 크기)

n_episodes = 500        # 총 학습 에피소드 수
max_steps = 20          # 한 에피소드에서 최대 스텝 수

buffer_capacity = 1000  # 리플레이 버퍼 최대 크기
batch_size = 32         # 미니배치 크기
warmup_steps = 100      # 최소 이 정도 샘플이 쌓인 후부터 학습 시작
target_update_freq = 20 # 타깃 네트워크를 몇 에피소드마다 한 번씩 갱신할지

# ======================================
# 4. 리플레이 버퍼 구현 (간단한 리스트 버퍼)
# ======================================
replay_buffer = []  # (state, action, reward, next_state, done) 튜플을 저장

def add_to_buffer(state, action, reward, next_state, done):
    # 버퍼에 새 transition 추가
    if len(replay_buffer) >= buffer_capacity:
        replay_buffer.pop(0)  # 가장 오래된 데이터 삭제 (FIFO)
    replay_buffer.append((state, action, reward, next_state, done))

def sample_from_buffer(batch_size):
    # 버퍼에서 랜덤하게 batch_size 개 샘플 추출
    indices = np.random.choice(len(replay_buffer), size=batch_size, replace=False)
    batch = [replay_buffer[i] for i in indices]
    return batch

# ======================================
# 5. ε-greedy 정책으로 행동 선택 함수 (Online Network 사용)
# ======================================
def choose_action(state, epsilon):
    # ε 확률로 랜덤 탐험
    if np.random.rand() < epsilon:
        return np.random.randint(n_actions)         # 0 또는 1 중 랜덤 선택

    # 1-ε 확률로 Q값이 최대인 행동 선택 (Online Network 기준)
    x = state_to_onehot(state)                      # 상태를 one-hot으로 변환
    _, _, q_values = forward(x, W1, b1, W2, b2)     # Q값 계산
    action = int(np.argmax(q_values, axis=1)[0])    # 가장 큰 Q값을 주는 행동 인덱스 선택
    return action

# ======================================
# 6. Double DQN 학습 함수
#    - 다음 상태에서 행동 선택은 Online Network
#    - 그 행동의 Q값 평가는 Target Network
# ======================================
def train_double_dqn(batch_size):
    global W1, b1, W2, b2   # 전역 파라미터 사용

    # 버퍼에서 미니배치 샘플 추출
    batch = sample_from_buffer(batch_size)

    # 배치를 각 성분별로 나누어 numpy 배열로 변환
    states      = np.array([s for (s, a, r, ns, d) in batch], dtype=np.int32)
    actions     = np.array([a for (s, a, r, ns, d) in batch], dtype=np.int32)
    rewards     = np.array([r for (s, a, r, ns, d) in batch], dtype=np.float32)
    next_states = np.array([ns for (s, a, r, ns, d) in batch], dtype=np.int32)
    dones       = np.array([d for (s, a, r, ns, d) in batch], dtype=bool)

    # 상태, 다음 상태를 one-hot 벡터로 변환
    X      = np.vstack([state_to_onehot(s)  for s in states])      # (B, n_states)
    X_next = np.vstack([state_to_onehot(ns) for ns in next_states])# (B, n_states)

    # 온라인 네트워크로 현재 상태의 Q값 계산
    z1, a1, q_values = forward(X, W1, b1, W2, b2)                  # q_values: (B, n_actions)

    # 다음 상태에서의 Q값 (Online Network) 계산 → argmax용
    _, _, q_values_next_online = forward(X_next, W1, b1, W2, b2)   # (B, n_actions)
    best_next_actions = np.argmax(q_values_next_online, axis=1)    # (B,)

    # 타깃 네트워크로 다음 상태의 Q값 계산 → 선택된 행동의 값만 사용
    _, _, q_values_next_tgt = forward(X_next, W1_tgt, b1_tgt, W2_tgt, b2_tgt)  # (B, n_actions)
    next_q_selected = q_values_next_tgt[np.arange(len(batch)), best_next_actions]  # (B,)

    # TD Target 계산: done이면 미래 보상 없음
    targets = rewards.copy()
    not_dones = (~dones)
    targets[not_dones] += gamma * next_q_selected[not_dones]

    # 예측 Q 중에서 실제로 선택한 행동의 Q값만 사용
    B = batch_size
    dL_dz2 = np.zeros_like(q_values)                                # (B, n_actions)
    q_pred_selected = q_values[np.arange(B), actions]               # (B,)
    td_errors = (q_pred_selected - targets) / B                     # (B,)

    # 선택한 행동 위치에만 gradient 반영
    dL_dz2[np.arange(B), actions] = td_errors                       # (B, n_actions)

    # 2층(출력층) 가중치/편향에 대한 gradient
    dW2 = a1.T @ dL_dz2                                             # (hidden_dim x n_actions)
    db2 = np.sum(dL_dz2, axis=0, keepdims=True)                     # (1 x n_actions)

    # 1층으로 gradient 전파
    dL_da1 = dL_dz2 @ W2.T                                          # (B x hidden_dim)
    dL_dz1 = dL_da1 * relu_deriv(z1)                                # (B x hidden_dim)

    # 1층 가중치/편향에 대한 gradient
    dW1 = X.T @ dL_dz1                                              # (input_dim x hidden_dim)
    db1 = np.sum(dL_dz1, axis=0, keepdims=True)                     # (1 x hidden_dim)

    # 파라미터 업데이트 (경사하강법)
    W2 -= learning_rate * dW2
    b2 -= learning_rate * db2
    W1 -= learning_rate * dW1
    b1 -= learning_rate * db1

# ======================================
# 7. Double DQN 학습 루프
# ======================================
reward_history = []  # 에피소드별 총 보상을 저장할 리스트
total_steps = 0      # 전체 스텝 수(옵션)

print("=== 1차원 선형 월드에서의 Double DQN(NumPy) 학습 시작 ===")

for episode in range(1, n_episodes + 1):

    state = reset()          # 에피소드 시작 상태 초기화
    total_reward = 0.0       # 에피소드 누적 보상 초기화

    for step_idx in range(max_steps):

        total_steps += 1

        # (1) ε-greedy 정책으로 행동 선택 (Online Network 기준)
        action = choose_action(state, epsilon)

        # (2) 선택한 행동을 환경에 적용 → 다음 상태, 보상, 종료 여부 반환
        next_state, reward, done = step(state, action)

        # (3) 리플레이 버퍼에 transition 저장
        add_to_buffer(state, action, reward, next_state, done)

        # (4) 일정 step 이상 쌓여야 학습 시작 (warmup_steps 이후)
        if len(replay_buffer) >= max(batch_size, warmup_steps):
            train_double_dqn(batch_size)

        # (5) 보상 누적
        total_reward += reward

        # (6) 상태를 다음 상태로 업데이트
        state = next_state

        # (7) 종료 상태면 에피소드 종료
        if done:
            break

    # (8) 에피소드 종료 후 epsilon 감소 (탐험 비율을 점점 줄임)
    epsilon = max(epsilon_min, epsilon * epsilon_decay)

    # (9) 에피소드별 총 보상을 기록
    reward_history.append(total_reward)

    # (10) 일정 에피소드마다 타깃 네트워크 파라미터 동기화
    if episode % target_update_freq == 0:
        W1_tgt = W1.copy()
        b1_tgt = b1.copy()
        W2_tgt = W2.copy()
        b2_tgt = b2.copy()

    # (11) 50 에피소드마다 최근 50개 평균 리워드와 현재 epsilon 출력
    if episode % 50 == 0:
        avg_reward = np.mean(reward_history[-50:])
        print(f"[Episode {episode:4d}] 최근 50 에피소드 평균 리워드 = {avg_reward:.3f}, epsilon = {epsilon:.3f}")

print("\n=== 학습 종료 ===\n")

# ======================================
# 8. 학습된 Online Q-Network로부터 '근사 Q-테이블' 출력
# ======================================
print("▶ 근사 Q-테이블 (행: 상태, 열: 행동[←,→])")

for s in range(n_states):
    x = state_to_onehot(s)                        # 상태 s를 one-hot으로 변환
    _, _, q_vals = forward(x, W1, b1, W2, b2)     # Q값 계산
    q_vals_row = q_vals[0]                        # (1 x n_actions) → (n_actions,)
    print(f"상태 {s}: {q_vals_row}")

# ======================================
# 9. 학습된 정책(Policy) 확인 (greedy 정책)
# ======================================
action_symbols = {0: "←", 1: "→"}                 # 행동 인덱스를 화살표로 표현하기 위한 매핑

print("\n▶ 학습된 정책(Policy)")

policy_str = ""
for s in range(n_states):
    if s == n_states - 1:
        policy_str += " G "                       # 목표 상태는 G로 표시
    else:
        x = state_to_onehot(s)
        _, _, q_vals = forward(x, W1, b1, W2, b2)
        best_action = int(np.argmax(q_vals, axis=1)[0])
        policy_str += f" {action_symbols[best_action]} "

print("상태 0  1  2  3  4")
print("     " + policy_str)

# ======================================
# 10. 학습된 정책으로 1회 테스트 실행 (탐험 없이 greedy만 사용)
# ======================================
print("\n▶ 학습된 정책으로 1회 에피소드 실행 예시")

state = reset()                       # 초기 상태 0
trajectory = [state]                  # 방문한 상태들을 저장할 리스트

for step_idx in range(max_steps):

    x = state_to_onehot(state)        # 현재 상태를 one-hot으로 변환
    _, _, q_vals = forward(x, W1, b1, W2, b2)  # Q값 계산
    action = int(np.argmax(q_vals))   # 탐험 없이 항상 greedy 행동 선택

    next_state, reward, done = step(state, action)  # 환경에 행동 적용
    trajectory.append(next_state)     # 방문한 상태 기록
    state = next_state                # 상태 업데이트

    if done:
        break                         # 목표 도달 시 종료

print("방문한 상태들:", trajectory)
print("스텝 수:", len(trajectory) - 1)
print("마지막 상태가 목표(4)면 학습 성공!")


=== 1차원 선형 월드에서의 Double DQN(NumPy) 학습 시작 ===
[Episode   50] 최근 50 에피소드 평균 리워드 = 0.691, epsilon = 0.778
[Episode  100] 최근 50 에피소드 평균 리워드 = 0.859, epsilon = 0.606
[Episode  150] 최근 50 에피소드 평균 리워드 = 0.916, epsilon = 0.471
[Episode  200] 최근 50 에피소드 평균 리워드 = 0.951, epsilon = 0.367
[Episode  250] 최근 50 에피소드 평균 리워드 = 0.955, epsilon = 0.286
[Episode  300] 최근 50 에피소드 평균 리워드 = 0.954, epsilon = 0.222
[Episode  350] 최근 50 에피소드 평균 리워드 = 0.960, epsilon = 0.173
[Episode  400] 최근 50 에피소드 평균 리워드 = 0.964, epsilon = 0.135
[Episode  450] 최근 50 에피소드 평균 리워드 = 0.967, epsilon = 0.105
[Episode  500] 최근 50 에피소드 평균 리워드 = 0.966, epsilon = 0.082

=== 학습 종료 ===

▶ 근사 Q-테이블 (행: 상태, 열: 행동[←,→])
상태 0: [0.55874722 0.6816946 ]
상태 1: [0.53761728 0.68297569]
상태 2: [0.52602922 0.73223193]
상태 3: [0.54516807 0.96246224]
상태 4: [0.57990547 0.69895713]

▶ 학습된 정책(Policy)
상태 0  1  2  3  4
      →  →  →  →  G 

▶ 학습된 정책으로 1회 에피소드 실행 예시
방문한 상태들: [0, 1, 2, 3, 4]
스텝 수: 4
마지막 상태가 목표(4)면 학습 성공!


In [7]:

########################################################################################################
## (1-6) Dueling DQN : 상태 가치와 행동의 중요도를 분리하여 Q-값을 계산하는 모델
########################################################################################################

import numpy as np  # 수치 계산을 위한 numpy

# 난수 시드 고정 (재현 가능한 결과를 위해)
np.random.seed(42)

# ======================================
# 1. 환경 설정 (1차원 선형 월드)
# ======================================
n_states = 5     # 상태 개수: 0,1,2,3,4 (4가 목표 상태)
n_actions = 2    # 행동 개수: 0=왼쪽, 1=오른쪽

def step(state, action):
    # action이 0이면 왼쪽으로 이동
    if action == 0:
        next_state = max(0, state - 1)              # 왼쪽 끝(0) 아래로 내려가지 않도록 제한
    # action이 1이면 오른쪽으로 이동
    else:
        next_state = min(n_states - 1, state + 1)   # 오른쪽 끝(4) 위로 넘어가지 않도록 제한

    # 목표 상태(4)에 도달한 경우
    if next_state == n_states - 1:
        reward = 1.0                                # 목표 도달 보상 +1
        done = True                                 # 에피소드 종료
    # 그 외의 경우
    else:
        reward = -0.01                              # 이동마다 작은 패널티 부여
        done = False                                # 에피소드 계속 진행

    return next_state, reward, done                 # 다음 상태, 보상, 종료 여부 반환

def reset():
    # 에피소드 시작 시 항상 상태 0에서 시작
    return 0

def state_to_onehot(state):
    # 상태를 one-hot 벡터(1 x n_states)로 변환
    x = np.zeros((1, n_states), dtype=np.float32)   # 1행 n_states열의 0 벡터 생성
    x[0, state] = 1.0                               # 해당 상태 인덱스 위치만 1로 설정
    return x

# ======================================
# 2. Dueling DQN용 Q-Network / Target Network 파라미터 정의
#    - 공통 feature 층 + Value Stream + Advantage Stream
# ======================================
input_dim = n_states      # 입력 차원: one-hot 상태
hidden_dim = 16           # 은닉층 노드 수
output_dim = n_actions    # 출력 차원: 각 행동에 대한 Q값 2개

# 온라인 네트워크(Online Dueling Q-Network) 파라미터
W1 = 0.1 * np.random.randn(input_dim, hidden_dim)    # 1층 가중치 (입력 → 은닉)
b1 = np.zeros((1, hidden_dim))                       # 1층 편향

# Value Stream: V(s) 출력 (스칼라)
Wv = 0.1 * np.random.randn(hidden_dim, 1)            # 은닉 → Value
bv = np.zeros((1, 1))                                # Value 편향

# Advantage Stream: A(s,a) 출력 (각 행동별)
Wa = 0.1 * np.random.randn(hidden_dim, output_dim)   # 은닉 → Advantage
ba = np.zeros((1, output_dim))                       # Advantage 편향

# 타깃 네트워크(Target Dueling Q-Network) 파라미터 (초기에는 동일하게 복사)
W1_tgt = W1.copy()
b1_tgt = b1.copy()
Wv_tgt = Wv.copy()
bv_tgt = bv.copy()
Wa_tgt = Wa.copy()
ba_tgt = ba.copy()

def relu(x):
    # ReLU 활성화 함수: 0보다 작으면 0, 크면 그대로
    return np.maximum(0, x)

def relu_deriv(x):
    # ReLU의 도함수: x>0이면 1, 아니면 0
    return (x > 0).astype(np.float32)

def forward(x, W1, b1, Wv, bv, Wa, ba):
    # 신경망 순전파: 입력 x에 대해 Q값을 계산
    # x: (배치크기, input_dim) 형태의 상태(one-hot) 벡터

    # 공통 feature 층 (shared layer)
    z1 = x @ W1 + b1                # 1층 선형 결합 (배치 x hidden_dim)
    a1 = relu(z1)                   # ReLU 활성화 (배치 x hidden_dim)

    # Value Stream: V(s)
    z_v = a1 @ Wv + bv              # (배치 x 1)

    # Advantage Stream: A(s,a)
    z_a = a1 @ Wa + ba              # (배치 x n_actions)

    # Advantage의 평균을 빼서 식: Q(s,a) = V(s) + (A(s,a) - mean_a A(s,a))
    mean_a = np.mean(z_a, axis=1, keepdims=True)  # (배치 x 1)
    q_values = z_v + (z_a - mean_a)               # (배치 x n_actions)

    # 역전파를 위해 중간값들까지 함께 반환
    return z1, a1, z_v, z_a, q_values

# ======================================
# 3. Dueling DQN 하이퍼파라미터 설정
# ======================================
gamma = 0.9             # 할인율
epsilon = 1.0           # ε-greedy에서 탐험 비율 시작값
epsilon_min = 0.05      # ε의 최소값
epsilon_decay = 0.995   # 에피소드마다 ε 감소 비율
learning_rate = 0.01    # 신경망 파라미터 학습률(경사하강 step 크기)

n_episodes = 500        # 총 학습 에피소드 수
max_steps = 20          # 한 에피소드에서 최대 스텝 수

buffer_capacity = 1000  # 리플레이 버퍼 최대 크기
batch_size = 32         # 미니배치 크기
warmup_steps = 100      # 최소 이 정도 샘플이 쌓인 후부터 학습 시작
target_update_freq = 20 # 타깃 네트워크를 몇 에피소드마다 한 번씩 갱신할지

# ======================================
# 4. 리플레이 버퍼 구현 (간단한 리스트 버퍼)
# ======================================
replay_buffer = []  # (state, action, reward, next_state, done) 튜플을 저장

def add_to_buffer(state, action, reward, next_state, done):
    # 버퍼에 새 transition 추가
    if len(replay_buffer) >= buffer_capacity:
        replay_buffer.pop(0)  # 가장 오래된 데이터 삭제 (FIFO)
    replay_buffer.append((state, action, reward, next_state, done))

def sample_from_buffer(batch_size):
    # 버퍼에서 랜덤하게 batch_size 개 샘플 추출
    indices = np.random.choice(len(replay_buffer), size=batch_size, replace=False)
    batch = [replay_buffer[i] for i in indices]
    return batch

# ======================================
# 5. ε-greedy 정책으로 행동 선택 함수 (Online Network 사용)
# ======================================
def choose_action(state, epsilon):
    # ε 확률로 랜덤 탐험
    if np.random.rand() < epsilon:
        return np.random.randint(n_actions)         # 0 또는 1 중 랜덤 선택

    # 1-ε 확률로 Q값이 최대인 행동 선택 (Online Dueling Network 기준)
    x = state_to_onehot(state)                      # 상태를 one-hot으로 변환
    _, _, _, _, q_values = forward(x, W1, b1, Wv, bv, Wa, ba)  # Q값 계산
    action = int(np.argmax(q_values, axis=1)[0])    # 가장 큰 Q값을 주는 행동 인덱스 선택
    return action

# ======================================
# 6. Dueling DQN 학습 함수 (표준 DQN 타깃: max_a Q_target(s', a))
# ======================================
def train_dueling_dqn(batch_size):
    global W1, b1, Wv, bv, Wa, ba   # 전역 파라미터 사용

    # 버퍼에서 미니배치 샘플 추출
    batch = sample_from_buffer(batch_size)

    # 배치를 각 성분별로 나누어 numpy 배열로 변환
    states      = np.array([s  for (s, a, r, ns, d) in batch], dtype=np.int32)
    actions     = np.array([a  for (s, a, r, ns, d) in batch], dtype=np.int32)
    rewards     = np.array([r  for (s, a, r, ns, d) in batch], dtype=np.float32)
    next_states = np.array([ns for (s, a, r, ns, d) in batch], dtype=np.int32)
    dones       = np.array([d  for (s, a, r, ns, d) in batch], dtype=bool)

    # 상태, 다음 상태를 one-hot 벡터로 변환
    X      = np.vstack([state_to_onehot(s)  for s in states])      # (B, n_states)
    X_next = np.vstack([state_to_onehot(ns) for ns in next_states])# (B, n_states)

    # 온라인 네트워크로 현재 상태의 Q값 계산
    z1, a1, z_v, z_a, q_values = forward(X, W1, b1, Wv, bv, Wa, ba)  # q_values: (B, n_actions)

    # 타깃 네트워크로 다음 상태의 Q값 계산
    _, _, _, _, q_values_next_tgt = forward(X_next, W1_tgt, b1_tgt, Wv_tgt, bv_tgt, Wa_tgt, ba_tgt)  # (B, n_actions)

    # 다음 상태에서의 최대 Q값(max_a' Q_target(s', a')) 추출
    max_next_q = np.max(q_values_next_tgt, axis=1)                  # (B,)

    # TD Target 계산: done이면 미래 보상 없음
    targets = rewards.copy()
    not_dones = (~dones)
    targets[not_dones] += gamma * max_next_q[not_dones]

    # 예측 Q 중에서 실제로 선택한 행동의 Q값만 사용
    B = batch_size
    D = np.zeros_like(q_values)                                     # (B, n_actions)
    q_pred_selected = q_values[np.arange(B), actions]               # (B,)
    td_errors = (q_pred_selected - targets) / B                     # (B,)

    # 선택한 행동 위치에만 gradient 반영
    D[np.arange(B), actions] = td_errors                            # dL/dQ (B, n_actions)

    # Dueling 구조에 맞는 gradient 분해
    # dL/dz_v = Σ_k dL/dQ_k (row-wise sum)
    row_sum = np.sum(D, axis=1, keepdims=True)                      # (B, 1)
    dL_dz_v = row_sum                                               # (B, 1)

    # dL/dz_a = D - (1/|A|) * row_sum  (A 평균을 빼는 부분 반영)
    dL_dz_a = D - row_sum / n_actions                               # (B, n_actions)

    # Value Stream 파라미터 gradient
    dWv = a1.T @ dL_dz_v                                            # (hidden_dim x 1)
    dbv = np.sum(dL_dz_v, axis=0, keepdims=True)                    # (1 x 1)

    # Advantage Stream 파라미터 gradient
    dWa = a1.T @ dL_dz_a                                            # (hidden_dim x n_actions)
    dba = np.sum(dL_dz_a, axis=0, keepdims=True)                    # (1 x n_actions)

    # 공통 은닉층으로 gradient 전파
    # dL/da1 = dL/dz_v * Wv^T + dL/dz_a * Wa^T
    dL_da1 = dL_dz_v @ Wv.T + dL_dz_a @ Wa.T                        # (B x hidden_dim)

    # ReLU 이전 z1에 대한 gradient
    dL_dz1 = dL_da1 * relu_deriv(z1)                                # (B x hidden_dim)

    # 1층(공통층) 파라미터 gradient
    dW1 = X.T @ dL_dz1                                              # (input_dim x hidden_dim)
    db1 = np.sum(dL_dz1, axis=0, keepdims=True)                     # (1 x hidden_dim)

    # 파라미터 업데이트 (경사하강법)
    Wv -= learning_rate * dWv
    bv -= learning_rate * dbv
    Wa -= learning_rate * dWa
    ba -= learning_rate * dba
    W1 -= learning_rate * dW1
    b1 -= learning_rate * db1

# ======================================
# 7. Dueling DQN 학습 루프
# ======================================
reward_history = []  # 에피소드별 총 보상을 저장할 리스트
total_steps = 0      # 전체 스텝 수(옵션)

print("=== 1차원 선형 월드에서의 Dueling DQN(NumPy) 학습 시작 ===")

for episode in range(1, n_episodes + 1):

    state = reset()          # 에피소드 시작 상태 초기화
    total_reward = 0.0       # 에피소드 누적 보상 초기화

    for step_idx in range(max_steps):

        total_steps += 1

        # (1) ε-greedy 정책으로 행동 선택 (Online Dueling Network 기준)
        action = choose_action(state, epsilon)

        # (2) 선택한 행동을 환경에 적용 → 다음 상태, 보상, 종료 여부 반환
        next_state, reward, done = step(state, action)

        # (3) 리플레이 버퍼에 transition 저장
        add_to_buffer(state, action, reward, next_state, done)

        # (4) 일정 step 이상 쌓여야 학습 시작 (warmup_steps 이후)
        if len(replay_buffer) >= max(batch_size, warmup_steps):
            train_dueling_dqn(batch_size)

        # (5) 보상 누적
        total_reward += reward

        # (6) 상태를 다음 상태로 업데이트
        state = next_state

        # (7) 종료 상태면 에피소드 종료
        if done:
            break

    # (8) 에피소드 종료 후 epsilon 감소 (탐험 비율을 점점 줄임)
    epsilon = max(epsilon_min, epsilon * epsilon_decay)

    # (9) 에피소드별 총 보상을 기록
    reward_history.append(total_reward)

    # (10) 일정 에피소드마다 타깃 네트워크 파라미터 동기화
    if episode % target_update_freq == 0:
        W1_tgt = W1.copy()
        b1_tgt = b1.copy()
        Wv_tgt = Wv.copy()
        bv_tgt = bv.copy()
        Wa_tgt = Wa.copy()
        ba_tgt = ba.copy()

    # (11) 50 에피소드마다 최근 50개 평균 리워드와 현재 epsilon 출력
    if episode % 50 == 0:
        avg_reward = np.mean(reward_history[-50:])
        print(f"[Episode {episode:4d}] 최근 50 에피소드 평균 리워드 = {avg_reward:.3f}, epsilon = {epsilon:.3f}")

print("\n=== 학습 종료 ===\n")

# ======================================
# 8. 학습된 Online Dueling Q-Network로부터 '근사 Q-테이블' 출력
# ======================================
print("▶ 근사 Q-테이블 (행: 상태, 열: 행동[←,→])")

for s in range(n_states):
    x = state_to_onehot(s)                                    # 상태 s를 one-hot으로 변환
    _, _, _, _, q_vals = forward(x, W1, b1, Wv, bv, Wa, ba)   # Q값 계산
    q_vals_row = q_vals[0]                                    # (1 x n_actions) → (n_actions,)
    print(f"상태 {s}: {q_vals_row}")

# ======================================
# 9. 학습된 정책(Policy) 확인 (greedy 정책)
# ======================================
action_symbols = {0: "←", 1: "→"}                             # 행동 인덱스를 화살표로 표현하기 위한 매핑

print("\n▶ 학습된 정책(Policy)")

policy_str = ""
for s in range(n_states):
    if s == n_states - 1:
        policy_str += " G "                                   # 목표 상태는 G로 표시
    else:
        x = state_to_onehot(s)
        _, _, _, _, q_vals = forward(x, W1, b1, Wv, bv, Wa, ba)
        best_action = int(np.argmax(q_vals, axis=1)[0])
        policy_str += f" {action_symbols[best_action]} "

print("상태 0  1  2  3  4")
print("     " + policy_str)

# ======================================
# 10. 학습된 정책으로 1회 테스트 실행 (탐험 없이 greedy만 사용)
# ======================================
print("\n▶ 학습된 정책으로 1회 에피소드 실행 예시")

state = reset()                       # 초기 상태 0
trajectory = [state]                  # 방문한 상태들을 저장할 리스트

for step_idx in range(max_steps):

    x = state_to_onehot(state)        # 현재 상태를 one-hot으로 변환
    _, _, _, _, q_vals = forward(x, W1, b1, Wv, bv, Wa, ba)  # Q값 계산
    action = int(np.argmax(q_vals))   # 탐험 없이 항상 greedy 행동 선택

    next_state, reward, done = step(state, action)  # 환경에 행동 적용
    trajectory.append(next_state)     # 방문한 상태 기록
    state = next_state                # 상태 업데이트

    if done:
        break                         # 목표 도달 시 종료

print("방문한 상태들:", trajectory)
print("스텝 수:", len(trajectory) - 1)
print("마지막 상태가 목표(4)면 학습 성공!")


=== 1차원 선형 월드에서의 Dueling DQN(NumPy) 학습 시작 ===
[Episode   50] 최근 50 에피소드 평균 리워드 = 0.691, epsilon = 0.778
[Episode  100] 최근 50 에피소드 평균 리워드 = 0.859, epsilon = 0.606
[Episode  150] 최근 50 에피소드 평균 리워드 = 0.916, epsilon = 0.471
[Episode  200] 최근 50 에피소드 평균 리워드 = 0.951, epsilon = 0.367
[Episode  250] 최근 50 에피소드 평균 리워드 = 0.955, epsilon = 0.286
[Episode  300] 최근 50 에피소드 평균 리워드 = 0.954, epsilon = 0.222
[Episode  350] 최근 50 에피소드 평균 리워드 = 0.960, epsilon = 0.173
[Episode  400] 최근 50 에피소드 평균 리워드 = 0.964, epsilon = 0.135
[Episode  450] 최근 50 에피소드 평균 리워드 = 0.967, epsilon = 0.105
[Episode  500] 최근 50 에피소드 평균 리워드 = 0.966, epsilon = 0.082

=== 학습 종료 ===

▶ 근사 Q-테이블 (행: 상태, 열: 행동[←,→])
상태 0: [0.61058338 0.69467441]
상태 1: [0.62574346 0.79322378]
상태 2: [0.65669643 0.92230427]
상태 3: [0.75054659 1.03802841]
상태 4: [0.66478158 0.86254989]

▶ 학습된 정책(Policy)
상태 0  1  2  3  4
      →  →  →  →  G 

▶ 학습된 정책으로 1회 에피소드 실행 예시
방문한 상태들: [0, 1, 2, 3, 4]
스텝 수: 4
마지막 상태가 목표(4)면 학습 성공!


In [8]:
########################################################################################################
## (1-7) DRQN(Deep Recurrent Q-Network) : 순차적인 경험을 학습하기 위해 RNN 구조를 포함한 DQN
########################################################################################################

import numpy as np  # 수치 계산을 위한 numpy

# 난수 시드 고정 (실행마다 동일한 결과를 얻기 위해)
np.random.seed(42)

# ======================================
# 1. 환경 설정 (1차원 선형 월드)
# ======================================
n_states = 5     # 상태 개수: 0,1,2,3,4 (4가 목표 상태)
n_actions = 2    # 행동 개수: 0=왼쪽, 1=오른쪽

def step(state, action):
    # action이 0이면 왼쪽으로 이동
    if action == 0:
        next_state = max(0, state - 1)              # 왼쪽 끝(0) 아래로 내려가지 않도록 제한
    # action이 1이면 오른쪽으로 이동
    else:
        next_state = min(n_states - 1, state + 1)   # 오른쪽 끝(4) 위로 넘어가지 않도록 제한

    # 목표 상태(4)에 도달한 경우
    if next_state == n_states - 1:
        reward = 1.0                                # 목표 도달 보상 +1
        done = True                                 # 에피소드 종료
    # 그 외의 경우
    else:
        reward = -0.01                              # 한 스텝마다 작은 패널티 부여(빨리 도달하도록 유도)
        done = False                                # 에피소드 계속 진행

    return next_state, reward, done                 # 다음 상태, 보상, 종료 여부 반환

def reset():
    # 에피소드 시작 시 항상 상태 0에서 시작
    return 0

def state_to_onehot(state):
    # 상태를 one-hot 벡터(1 x n_states)로 변환
    x = np.zeros((1, n_states), dtype=np.float32)   # 1행 n_states열의 영벡터 생성
    x[0, state] = 1.0                               # 현재 상태 인덱스 위치만 1로 설정
    return x

# ======================================
# 2. DRQN용 RNN-Q 네트워크 파라미터 정의
#    - 단일 은닉층 RNN + 선형 출력층
#    - h_t = tanh(x_t Wxh + h_{t-1} Whh + bh)
#    - Q(s_t, a) = h_t Wout + bout
# ======================================
input_dim  = n_states   # 입력 차원: 상태를 one-hot으로 표시하므로 n_states
hidden_dim = 16         # RNN 은닉 상태 차원
output_dim = n_actions  # 출력 차원: 각 행동에 대한 Q값 2개

# 입력 → 은닉 가중치 (Wxh)
Wxh = 0.1 * np.random.randn(input_dim, hidden_dim)   # (input_dim x hidden_dim)
# 은닉 → 은닉 가중치 (Whh)
Whh = 0.1 * np.random.randn(hidden_dim, hidden_dim)  # (hidden_dim x hidden_dim)
# 은닉 편향 (bh)
bh  = np.zeros((1, hidden_dim))                      # (1 x hidden_dim)

# 은닉 → 출력(Q값) 가중치 (Wout)
Wout = 0.1 * np.random.randn(hidden_dim, output_dim) # (hidden_dim x output_dim)
# 출력 편향 (bout)
bout = np.zeros((1, output_dim))                     # (1 x output_dim)

def rnn_step(x, h_prev):
    # RNN 한 스텝 순전파
    # x      : (1 x input_dim)  현재 상태의 one-hot 벡터
    # h_prev : (1 x hidden_dim) 직전 시점의 은닉 상태
    # 반환값 : h_next(다음 은닉 상태), q_values(현재 상태에서의 각 행동 Q값)

    # 1층 선형 결합: z = x Wxh + h_prev Whh + bh
    z = x @ Wxh + h_prev @ Whh + bh      # (1 x hidden_dim)

    # tanh 활성화: h_next = tanh(z)
    h_next = np.tanh(z)                  # (1 x hidden_dim)

    # 출력층: q_values = h_next Wout + bout
    q_values = h_next @ Wout + bout      # (1 x output_dim)

    return z, h_next, q_values

# ======================================
# 3. DRQN 하이퍼파라미터 설정
# ======================================
gamma = 0.9             # 할인율(미래 보상 반영 비율)
epsilon = 1.0           # ε-greedy에서 탐험 비율 시작값
epsilon_min = 0.05      # ε의 최소값
epsilon_decay = 0.995   # 에피소드마다 ε 감소 비율
learning_rate = 0.01    # 신경망 파라미터 학습률(경사하강 step 크기)

n_episodes = 500        # 총 학습 에피소드 수
max_steps  = 20         # 한 에피소드에서 최대 스텝 수 (무한 루프 방지)

# ======================================
# 4. ε-greedy 정책으로 행동 선택
#    - 입력: 현재 상태, 현재 은닉 상태
#    - 출력: 선택된 행동, 그때의 Q값, 업데이트된 은닉 상태
# ======================================
def choose_action_with_rnn(state, h_prev, epsilon):
    # 1) 상태를 one-hot 벡터로 변환
    x = state_to_onehot(state)                   # (1 x n_states)

    # 2) RNN 한 스텝 순전파
    z, h_next, q_values = rnn_step(x, h_prev)   # h_next: 업데이트된 은닉 상태

    # 3) ε-greedy로 행동 선택
    if np.random.rand() < epsilon:
        action = np.random.randint(n_actions)    # 탐험: 무작위 행동 선택
    else:
        action = int(np.argmax(q_values))        # 이용: Q값이 최대인 행동 선택

    return action, z, h_next, q_values

# ======================================
# 5. DRQN 학습 루프
#    - 각 스텝마다 TD(0) 손실에 대해 단일 스텝 BPTT(역전파)
#    - h_prev까지는 gradient를 전파하지 않는 truncation(1-step BPTT)
# ======================================
reward_history = []  # 에피소드별 총 보상 기록용 리스트

print("=== 1차원 선형 월드에서의 DRQN(NumPy) 학습 시작 ===")

for episode in range(1, n_episodes + 1):

    state = reset()                              # 에피소드 시작 시 상태 초기화 (0)
    h_prev = np.zeros((1, hidden_dim))          # 에피소드 시작 시 은닉 상태 0으로 초기화
    total_reward = 0.0                          # 에피소드별 누적 보상 초기화

    for step_idx in range(max_steps):

        # (1) 현재 상태와 은닉 상태를 이용해 ε-greedy로 행동 선택
        action, z, h, q_values = choose_action_with_rnn(state, h_prev, epsilon)

        # (2) 선택한 행동을 환경에 적용 → 다음 상태, 보상, 종료 여부 반환
        next_state, reward, done = step(state, action)

        # (3) 다음 상태에서의 Q값을 계산하여 TD Target 구성
        #     여기서는 target용 RNN 한 스텝을 별도로 수행 (탐험 없이 greedy 기준)
        x_next = state_to_onehot(next_state)                   # 다음 상태 one-hot
        z_next, h_next, q_next = rnn_step(x_next, h)           # 다음 시점 은닉 상태, Q값

        # done이면 미래 보상 없음, 아니면 max_a' Q(s', a') 반영
        if done:
            td_target = reward                                 # 목표 상태 도달 시
        else:
            td_target = reward + gamma * np.max(q_next)        # TD Target = r + γ max Q'

        # (4) 현재 시점에서 선택한 행동의 Q값 추출
        q_pred = q_values[0, action]                           # 스칼라 값

        # (5) 손실: L = 0.5 * (q_pred - td_target)^2
        #     dL/dq_pred = (q_pred - td_target)
        dL_dq = q_pred - td_target                             # 스칼라 gradient

        # (6) 출력층(Wout, bout)에 대한 gradient
        #     q_values = h @ Wout + bout
        #     dL/dWout = h^T @ dL/dq (단, 선택한 action에만 반영)
        dQ = np.zeros_like(q_values)                           # (1 x output_dim)
        dQ[0, action] = dL_dq                                  # 선택한 action 위치에만 gradient

        dWout = h.T @ dQ                                       # (hidden_dim x output_dim)
        dbout = dQ                                             # (1 x output_dim)

        # (7) 은닉 상태 h에 대한 gradient
        #     dL/dh = dQ @ Wout^T
        dL_dh = dQ @ Wout.T                                    # (1 x hidden_dim)

        # (8) tanh 이전 z에 대한 gradient
        #     h = tanh(z) 이므로 dh/dz = (1 - tanh(z)^2)
        dh_dz = 1.0 - np.tanh(z) ** 2                          # (1 x hidden_dim)
        dL_dz = dL_dh * dh_dz                                  # (1 x hidden_dim)

        # (9) 입력층(Wxh), 순환층(Whh), 편향(bh)에 대한 gradient
        x = state_to_onehot(state)                             # 현재 상태 one-hot (1 x input_dim)

        dWxh = x.T @ dL_dz                                     # (input_dim x hidden_dim)
        dWhh = h_prev.T @ dL_dz                                # (hidden_dim x hidden_dim)
        dbh  = dL_dz                                           # (1 x hidden_dim)

        # (10) 경사하강법으로 파라미터 업데이트
        Wout -= learning_rate * dWout
        bout -= learning_rate * dbout
        Wxh  -= learning_rate * dWxh
        Whh  -= learning_rate * dWhh
        bh   -= learning_rate * dbh

        # (11) 보상 누적
        total_reward += reward

        # (12) 다음 스텝을 위해 상태와 은닉 상태를 업데이트
        state  = next_state
        h_prev = h                                             # 바로 이전 은닉 상태를 현재 h로 설정

        # (13) 목표에 도달하면 에피소드 종료
        if done:
            break

    # (14) 에피소드 종료 후 epsilon 감소 (탐험 비율을 점점 줄임)
    epsilon = max(epsilon_min, epsilon * epsilon_decay)

    # (15) 에피소드별 총 보상 기록
    reward_history.append(total_reward)

    # (16) 50 에피소드마다 최근 50개 평균 리워드와 현재 epsilon 출력
    if episode % 50 == 0:
        avg_reward = np.mean(reward_history[-50:])
        print(f"[Episode {episode:4d}] 최근 50 에피소드 평균 리워드 = {avg_reward:.3f}, epsilon = {epsilon:.3f}")

print("\n=== 학습 종료 ===\n")

# ======================================
# 6. 학습된 DRQN에서 '근사 Q-테이블' 출력
#    - 은닉 상태를 0으로 두고, 각 상태를 독립적으로 넣어서 Q값을 본다
# ======================================
print("▶ 근사 Q-테이블 (행: 상태, 열: 행동[←,→])")

for s in range(n_states):
    # 각 상태별로 은닉 상태를 0에서 시작한다고 가정
    h0 = np.zeros((1, hidden_dim))                      # 초기 은닉 상태
    x  = state_to_onehot(s)                             # 상태 s의 one-hot
    z, h, q_vals = rnn_step(x, h0)                      # 한 스텝 순전파
    q_vals_row = q_vals[0]                              # (1 x output_dim) → (output_dim,)
    print(f"상태 {s}: {q_vals_row}")

# ======================================
# 7. 학습된 정책(Policy) 확인 (greedy 정책)
#    - 에피소드 실행 시에는 은닉 상태가 계속 이어진다는 점이 DRQN의 특징
# ======================================
action_symbols = {0: "←", 1: "→"}                       # 행동 인덱스를 화살표로 표현하기 위한 매핑

print("\n▶ 학습된 정책(Policy)")

policy_str = ""
for s in range(n_states):
    if s == n_states - 1:
        policy_str += " G "                             # 목표 상태는 G로 표시
    else:
        h0 = np.zeros((1, hidden_dim))                  # 정책만 볼 때는 h=0 기준으로 계산
        x  = state_to_onehot(s)
        z, h, q_vals = rnn_step(x, h0)
        best_action = int(np.argmax(q_vals, axis=1)[0])
        policy_str += f" {action_symbols[best_action]} "

print("상태 0  1  2  3  4")
print("     " + policy_str)

# ======================================
# 8. 학습된 정책으로 1회 에피소드 실행 (탐험 없이 greedy만 사용)
#    - 여기서는 은닉 상태를 실제로 시퀀스 전체에 걸쳐 사용
# ======================================
print("\n▶ 학습된 정책으로 1회 에피소드 실행 예시")

state = reset()                               # 초기 상태 0
h_prev = np.zeros((1, hidden_dim))           # 은닉 상태 0으로 초기화
trajectory = [state]                          # 방문한 상태들을 저장할 리스트

for step_idx in range(max_steps):

    x = state_to_onehot(state)               # 현재 상태를 one-hot으로 변환
    z, h, q_vals = rnn_step(x, h_prev)       # RNN 한 스텝 순전파
    action = int(np.argmax(q_vals))          # 탐험 없이 항상 greedy 행동 선택

    next_state, reward, done = step(state, action)  # 환경에 행동 적용
    trajectory.append(next_state)            # 방문한 상태 기록

    # 다음 스텝을 위해 상태와 은닉 상태 업데이트
    state  = next_state
    h_prev = h

    if done:
        break                                # 목표 도달 시 종료

print("방문한 상태들:", trajectory)
print("스텝 수:", len(trajectory) - 1)
print("마지막 상태가 목표(4)면 학습 성공!")


=== 1차원 선형 월드에서의 DRQN(NumPy) 학습 시작 ===
[Episode   50] 최근 50 에피소드 평균 리워드 = 0.646, epsilon = 0.778
[Episode  100] 최근 50 에피소드 평균 리워드 = 0.815, epsilon = 0.606
[Episode  150] 최근 50 에피소드 평균 리워드 = 0.934, epsilon = 0.471
[Episode  200] 최근 50 에피소드 평균 리워드 = 0.946, epsilon = 0.367
[Episode  250] 최근 50 에피소드 평균 리워드 = 0.953, epsilon = 0.286
[Episode  300] 최근 50 에피소드 평균 리워드 = 0.961, epsilon = 0.222
[Episode  350] 최근 50 에피소드 평균 리워드 = 0.962, epsilon = 0.173
[Episode  400] 최근 50 에피소드 평균 리워드 = 0.963, epsilon = 0.135
[Episode  450] 최근 50 에피소드 평균 리워드 = 0.967, epsilon = 0.105
[Episode  500] 최근 50 에피소드 평균 리워드 = 0.964, epsilon = 0.082

=== 학습 종료 ===

▶ 근사 Q-테이블 (행: 상태, 열: 행동[←,→])
상태 0: [0.59828139 0.69852075]
상태 1: [0.62906271 0.79959098]
상태 2: [0.66495218 0.90457089]
상태 3: [0.63253563 0.99507243]
상태 4: [0.6295555 0.7800156]

▶ 학습된 정책(Policy)
상태 0  1  2  3  4
      →  →  →  →  G 

▶ 학습된 정책으로 1회 에피소드 실행 예시
방문한 상태들: [0, 1, 2, 3, 4]
스텝 수: 4
마지막 상태가 목표(4)면 학습 성공!


In [9]:

########################################################################################################
## (1-8) C51(Categorical DQN 51) : Q-값의 분포를 학습하는 DQN 확장
########################################################################################################

import numpy as np  # 수치 계산을 위한 numpy

# 난수 시드 고정 (재현 가능한 결과를 위해)
np.random.seed(42)

# ======================================
# 1. 환경 설정 (1차원 선형 월드)
# ======================================
n_states = 5     # 상태 개수: 0,1,2,3,4 (4가 목표 상태)
n_actions = 2    # 행동 개수: 0=왼쪽, 1=오른쪽

def step(state, action):
    # action이 0이면 왼쪽으로 이동
    if action == 0:
        next_state = max(0, state - 1)              # 왼쪽 끝(0) 아래로 내려가지 않도록 제한
    # action이 1이면 오른쪽으로 이동
    else:
        next_state = min(n_states - 1, state + 1)   # 오른쪽 끝(4) 위로 넘어가지 않도록 제한

    # 목표 상태(4)에 도달한 경우
    if next_state == n_states - 1:
        reward = 1.0                                # 목표 도달 보상 +1
        done = True                                 # 에피소드 종료
    # 그 외의 경우
    else:
        reward = -0.01                              # 한 스텝마다 작은 패널티 부여
        done = False                                # 에피소드 계속 진행

    return next_state, reward, done                 # 다음 상태, 보상, 종료 여부 반환

def reset():
    # 에피소드 시작 시 항상 상태 0에서 시작
    return 0

def state_to_onehot(state):
    # 상태를 one-hot 벡터(1 x n_states)로 변환
    x = np.zeros((1, n_states), dtype=np.float32)   # 1행 n_states열의 0 벡터 생성
    x[0, state] = 1.0                               # 해당 상태 인덱스 위치만 1로 설정
    return x

# ======================================
# 2. C51(51-atom Categorical DQN) 설정
#    - Vmin, Vmax: 가치 분포의 최소/최대 값
#    - n_atoms   : 원자(atom) 개수
# ======================================
n_atoms = 51                    # C51 논문에서 제안한 atom 개수
Vmin = -1.0                     # 최소 가치 (이 환경에 맞춰 대략 설정)
Vmax =  1.0                     # 최대 가치
delta_z = (Vmax - Vmin) / (n_atoms - 1)          # 인접 atom 간 간격
z_support = np.linspace(Vmin, Vmax, n_atoms)     # shape: (n_atoms,)

# ======================================
# 3. C51용 Q-Network / Target Network 정의
#    - 입력: 상태(one-hot)
#    - 출력: 각 행동별 원자 분포의 로짓 (logits) → softmax로 확률 분포
# ======================================
input_dim = n_states                    # 상태 차원 (one-hot)
hidden_dim = 16                         # 은닉층 크기
output_dim = n_actions * n_atoms        # 각 행동마다 n_atoms개의 확률 → 총 출력차원

# 온라인 네트워크(Online C51 Q-Network) 파라미터
W1 = 0.1 * np.random.randn(input_dim, hidden_dim)   # 1층 가중치 (입력 → 은닉)
b1 = np.zeros((1, hidden_dim))                      # 1층 편향
W2 = 0.1 * np.random.randn(hidden_dim, output_dim)  # 2층 가중치 (은닉 → 행동×원자)
b2 = np.zeros((1, output_dim))                      # 2층 편향

# 타깃 네트워크(Target C51 Q-Network) 파라미터 (초기에는 동일하게 복사)
W1_tgt = W1.copy()
b1_tgt = b1.copy()
W2_tgt = W2.copy()
b2_tgt = b2.copy()

def relu(x):
    # ReLU 활성화 함수: 0보다 작으면 0, 크면 그대로
    return np.maximum(0, x)

def relu_deriv(x):
    # ReLU의 도함수: x>0이면 1, 아니면 0
    return (x > 0).astype(np.float32)

def softmax(x, axis=-1):
    # Softmax 함수: 숫자 안정성 확보 위해 최대값을 빼고 exp 계산
    x_shifted = x - np.max(x, axis=axis, keepdims=True)  # overflow 방지
    exp_x = np.exp(x_shifted)
    sum_exp = np.sum(exp_x, axis=axis, keepdims=True)
    return exp_x / (sum_exp + 1e-8)                      # 0으로 나눌 위험 방지용 작은 값 추가

def forward(x, W1, b1, W2, b2):
    # C51 네트워크 순전파
    # x           : (배치크기, input_dim)
    # 반환값      : z1, h, logits(배치 x 행동 x 원자), probs(동일 크기)
    z1 = x @ W1 + b1                             # 1층 선형 결합 (배치 x hidden_dim)
    h  = relu(z1)                                # ReLU 활성화 (배치 x hidden_dim)

    logits_flat = h @ W2 + b2                    # (배치 x (n_actions * n_atoms))
    batch_size = x.shape[0]
    logits = logits_flat.reshape(batch_size, n_actions, n_atoms)  # (배치 x 행동 x 원자)

    probs = softmax(logits, axis=2)              # atom 차원(마지막 축)에 대해 softmax → 확률 분포
    return z1, h, logits, probs

# ======================================
# 4. C51 하이퍼파라미터 설정
# ======================================
gamma = 0.9             # 할인율
epsilon = 1.0           # ε-greedy에서 탐험 비율 시작값
epsilon_min = 0.05      # ε의 최소값
epsilon_decay = 0.995   # 에피소드마다 ε 감소 비율
learning_rate = 0.001   # 신경망 파라미터 학습률 (C51이므로 조금 작게 설정)

n_episodes = 500        # 총 학습 에피소드 수
max_steps = 20          # 한 에피소드에서 최대 스텝 수

buffer_capacity = 1000  # 리플레이 버퍼 최대 크기
batch_size = 32         # 미니배치 크기
warmup_steps = 100      # 최소 이 정도 샘플이 쌓인 후부터 학습 시작
target_update_freq = 20 # 타깃 네트워크를 몇 에피소드마다 한 번씩 갱신할지

# ======================================
# 5. 리플레이 버퍼 구현 (간단한 리스트 버퍼)
# ======================================
replay_buffer = []  # (state, action, reward, next_state, done) 튜플을 저장

def add_to_buffer(state, action, reward, next_state, done):
    # 버퍼에 새 transition 추가
    if len(replay_buffer) >= buffer_capacity:
        replay_buffer.pop(0)  # 가장 오래된 데이터 삭제 (FIFO)
    replay_buffer.append((state, action, reward, next_state, done))

def sample_from_buffer(batch_size):
    # 버퍼에서 랜덤하게 batch_size 개 샘플 추출
    indices = np.random.choice(len(replay_buffer), size=batch_size, replace=False)
    batch = [replay_buffer[i] for i in indices]
    return batch

# ======================================
# 6. ε-greedy 정책으로 행동 선택 (C51 분포 → 기대값 Q로 변환 후 사용)
# ======================================
def choose_action(state, epsilon):
    # ε 확률로 랜덤 탐험
    if np.random.rand() < epsilon:
        return np.random.randint(n_actions)         # 0 또는 1 중 랜덤 선택

    # 1-ε 확률로 분포의 기대값(∑ z * p)을 기준으로 Q값을 계산하고, 그 중 최대 행동 선택
    x = state_to_onehot(state)                      # 상태를 one-hot으로 변환
    _, _, _, probs = forward(x, W1, b1, W2, b2)     # (1 x 행동 x 원자) 크기의 확률 분포

    # 각 행동별 기대 Q값 계산: Q(a) = Σ_j z_j * p(a, j)
    q_values = np.sum(z_support[None, None, :] * probs, axis=2)  # (1 x 행동)
    action = int(np.argmax(q_values, axis=1)[0])                 # Q값이 최대인 행동 선택
    return action

# ======================================
# 7. C51의 핵심: 분포 프로젝션 함수
#    - r + γ z_j 를 [Vmin, Vmax] 구간의 atom들로 분배하는 함수
# ======================================
def project_distribution(rewards, dones, next_probs):
    # rewards    : (B,)        - 보상 r
    # dones      : (B,) bool   - 종료 여부
    # next_probs : (B, n_atoms) - 다음 상태에서의 선택된 행동 a*에 대한 분포 p(z'|s',a*)
    # 반환값     : (B, n_atoms) - 현재 상태에서의 타깃 분포 m(z)
    batch_size = rewards.shape[0]

    # Tz = r + γ z (단, 종료 상태면 Tz = r)
    # rewards[:, None] : (B x 1)
    # z_support[None,:]: (1 x n_atoms)
    Tz = rewards[:, None] + gamma * z_support[None, :] * (1.0 - dones[:, None].astype(np.float32))
    Tz = np.clip(Tz, Vmin, Vmax)   # Vmin, Vmax 범위로 클리핑

    # b = (Tz - Vmin) / Δz  (atom index 실수값)
    b = (Tz - Vmin) / delta_z
    l = np.floor(b).astype(int)
    u = np.ceil(b).astype(int)

    # 인덱스는 [0, n_atoms-1] 범위로 잘라줌
    l = np.clip(l, 0, n_atoms - 1)
    u = np.clip(u, 0, n_atoms - 1)

    # 최종 타깃 분포 m(z) 초기화
    m = np.zeros((batch_size, n_atoms), dtype=np.float32)

    # 각 atom j에 대해 m의 해당 위치에 분배
    for j in range(n_atoms):
        # 각 배치에서 l_ij, u_ij 위치에 확률을 나누어 더함
        lj = l[:, j]          # (B,)
        uj = u[:, j]          # (B,)
        bj = b[:, j]          # (B,)
        pj = next_probs[:, j] # (B,)  현재 atom j의 확률

        # m_{l} += p_j * (u - b)
        m[np.arange(batch_size), lj] += pj * (uj - bj)
        # m_{u} += p_j * (b - l)
        m[np.arange(batch_size), uj] += pj * (bj - lj)

    # 수치적 안정성을 위해 정규화(합이 1이 되도록)
    m_sum = np.sum(m, axis=1, keepdims=True)
    m = m / (m_sum + 1e-8)
    return m

# ======================================
# 8. C51 학습 함수 (리플레이 버퍼에서 미니배치 샘플 → 분포 업데이트)
# ======================================
def train_c51(batch_size):
    global W1, b1, W2, b2   # 전역 파라미터 사용

    # 버퍼에서 미니배치 샘플 추출
    batch = sample_from_buffer(batch_size)

    # 배치를 각 성분별로 나누어 numpy 배열로 변환
    states      = np.array([s  for (s, a, r, ns, d) in batch], dtype=np.int32)
    actions     = np.array([a  for (s, a, r, ns, d) in batch], dtype=np.int32)
    rewards     = np.array([r  for (s, a, r, ns, d) in batch], dtype=np.float32)
    next_states = np.array([ns for (s, a, r, ns, d) in batch], dtype=np.int32)
    dones       = np.array([d  for (s, a, r, ns, d) in batch], dtype=bool)

    # 상태, 다음 상태를 one-hot 벡터로 변환
    X      = np.vstack([state_to_onehot(s)  for s in states])      # (B, n_states)
    X_next = np.vstack([state_to_onehot(ns) for ns in next_states])# (B, n_states)

    batch_size = X.shape[0]

    # 온라인 네트워크로 현재 상태의 로짓/분포 계산
    z1, h, logits, probs = forward(X, W1, b1, W2, b2)              # probs: (B, 행동, 원자)

    # 타깃 네트워크로 다음 상태의 분포 계산
    _, _, _, next_probs_all = forward(X_next, W1_tgt, b1_tgt, W2_tgt, b2_tgt)  # (B, 행동, 원자)

    # 다음 상태에서 각 행동별 기대 Q값 계산
    # Q_next(a) = Σ z * p(a,z)
    q_next = np.sum(z_support[None, None, :] * next_probs_all, axis=2)  # (B, 행동)

    # Double DQN 스타일: 다음 상태에서 행동 선택은 온라인 네트워크로 해도 되지만,
    # 여기서는 간단히 타깃 네트워크 기준으로 max Q 행동 선택
    best_next_actions = np.argmax(q_next, axis=1)                  # (B,)

    # 선택된 행동 a*에 대한 다음 상태의 분포 p(z|s',a*)
    next_probs = next_probs_all[np.arange(batch_size), best_next_actions, :]  # (B, n_atoms)

    # C51 분포 프로젝션으로 타깃 분포 m 계산
    target_dist = project_distribution(rewards, dones, next_probs)           # (B, n_atoms)

    # 현재 상태에서 선택된 행동 a에 대한 분포 p(z|s,a)
    current_probs = probs[np.arange(batch_size), actions, :]                 # (B, n_atoms)

    # cross-entropy 손실 L = - Σ m(z) * log p(z)
    # softmax + cross-entropy의 gradient: dL/dlogits = p - m
    # 따라서, 선택된 행동에 대해서만 gradient p - m을 반영
    d_logits = np.zeros_like(logits)                                         # (B, 행동, 원자)
    d_logits[np.arange(batch_size), actions, :] = (current_probs - target_dist)  # (B, 원자)

    # 2층(출력층) gradient 계산
    d_logits_flat = d_logits.reshape(batch_size, -1)                         # (B, 행동*원자)
    dW2 = h.T @ d_logits_flat                                                # (hidden_dim x (행동*원자))
    db2 = np.sum(d_logits_flat, axis=0, keepdims=True)                       # (1 x (행동*원자))

    # 은닉층으로 gradient 전파
    d_h = d_logits_flat @ W2.T                                               # (B x hidden_dim)
    d_z1 = d_h * relu_deriv(z1)                                              # (B x hidden_dim)

    # 1층(입력층) gradient 계산
    dW1 = X.T @ d_z1                                                         # (input_dim x hidden_dim)
    db1 = np.sum(d_z1, axis=0, keepdims=True)                                # (1 x hidden_dim)

    # 파라미터 업데이트 (경사하강법)
    W2 -= learning_rate * dW2
    b2 -= learning_rate * db2
    W1 -= learning_rate * dW1
    b1 -= learning_rate * db1

# ======================================
# 9. C51 학습 루프
# ======================================
reward_history = []  # 에피소드별 총 보상을 저장할 리스트
total_steps = 0      # 전체 스텝 수(옵션)

print("=== 1차원 선형 월드에서의 C51 (Categorical DQN, NumPy) 학습 시작 ===")

for episode in range(1, n_episodes + 1):

    state = reset()          # 에피소드 시작 상태 초기화
    total_reward = 0.0       # 에피소드 누적 보상 초기화

    for step_idx in range(max_steps):

        total_steps += 1

        # (1) ε-greedy 정책으로 행동 선택 (분포 기대값 기준 Q 사용)
        action = choose_action(state, epsilon)

        # (2) 선택한 행동을 환경에 적용 → 다음 상태, 보상, 종료 여부 반환
        next_state, reward, done = step(state, action)

        # (3) 리플레이 버퍼에 transition 저장
        add_to_buffer(state, action, reward, next_state, done)

        # (4) 일정 step 이상 쌓여야 학습 시작 (warmup_steps 이후)
        if len(replay_buffer) >= max(batch_size, warmup_steps):
            train_c51(batch_size)

        # (5) 보상 누적
        total_reward += reward

        # (6) 상태를 다음 상태로 업데이트
        state = next_state

        # (7) 종료 상태면 에피소드 종료
        if done:
            break

    # (8) 에피소드 종료 후 epsilon 감소 (탐험 비율을 점점 줄임)
    epsilon = max(epsilon_min, epsilon * epsilon_decay)

    # (9) 에피소드별 총 보상을 기록
    reward_history.append(total_reward)

    # (10) 일정 에피소드마다 타깃 네트워크 파라미터 동기화
    if episode % target_update_freq == 0:
        W1_tgt = W1.copy()
        b1_tgt = b1.copy()
        W2_tgt = W2.copy()
        b2_tgt = b2.copy()

    # (11) 50 에피소드마다 최근 50개 평균 리워드와 현재 epsilon 출력
    if episode % 50 == 0:
        avg_reward = np.mean(reward_history[-50:])
        print(f"[Episode {episode:4d}] 최근 50 에피소드 평균 리워드 = {avg_reward:.3f}, epsilon = {epsilon:.3f}")

print("\n=== 학습 종료 ===\n")

# ======================================
# 10. 학습된 C51 분포로부터 '근사 Q-테이블' 출력
#     - 각 상태별로 분포를 forward 후, 기대값 Q(s,a)을 계산
# ======================================
print("▶ 근사 Q-테이블 (행: 상태, 열: 행동[←,→])")

for s in range(n_states):
    x = state_to_onehot(s)                                         # 상태 s를 one-hot으로 변환
    _, _, _, probs = forward(x, W1, b1, W2, b2)                    # (1 x 행동 x 원자)
    q_vals = np.sum(z_support[None, None, :] * probs, axis=2)      # (1 x 행동)
    q_row = q_vals[0]                                              # (행동,)
    print(f"상태 {s}: {q_row}")

# ======================================
# 11. 학습된 정책(Policy) 확인 (greedy 정책)
# ======================================
action_symbols = {0: "←", 1: "→"}                                   # 행동 인덱스를 화살표로 표현

print("\n▶ 학습된 정책(Policy)")

policy_str = ""
for s in range(n_states):
    if s == n_states - 1:
        policy_str += " G "                                         # 목표 상태는 G로 표시
    else:
        x = state_to_onehot(s)
        _, _, _, probs = forward(x, W1, b1, W2, b2)
        q_vals = np.sum(z_support[None, None, :] * probs, axis=2)   # (1 x 행동)
        best_action = int(np.argmax(q_vals, axis=1)[0])
        policy_str += f" {action_symbols[best_action]} "

print("상태 0  1  2  3  4")
print("     " + policy_str)

# ======================================
# 12. 학습된 정책으로 1회 에피소드 실행 (탐험 없이 greedy만 사용)
# ======================================
print("\n▶ 학습된 정책으로 1회 에피소드 실행 예시")

state = reset()                               # 초기 상태 0
trajectory = [state]                          # 방문한 상태들을 저장할 리스트

for step_idx in range(max_steps):

    x = state_to_onehot(state)
    _, _, _, probs = forward(x, W1, b1, W2, b2)
    q_vals = np.sum(z_support[None, None, :] * probs, axis=2)  # (1 x 행동)
    action = int(np.argmax(q_vals))                            # 탐험 없이 greedy 행동 선택

    next_state, reward, done = step(state, action)
    trajectory.append(next_state)
    state = next_state

    if done:
        break

print("방문한 상태들:", trajectory)
print("스텝 수:", len(trajectory) - 1)
print("마지막 상태가 목표(4)면 학습 성공!")


=== 1차원 선형 월드에서의 C51 (Categorical DQN, NumPy) 학습 시작 ===
[Episode   50] 최근 50 에피소드 평균 리워드 = 0.781, epsilon = 0.778
[Episode  100] 최근 50 에피소드 평균 리워드 = 0.858, epsilon = 0.606
[Episode  150] 최근 50 에피소드 평균 리워드 = 0.690, epsilon = 0.471
[Episode  200] 최근 50 에피소드 평균 리워드 = 0.300, epsilon = 0.367
[Episode  250] 최근 50 에피소드 평균 리워드 = -0.029, epsilon = 0.286
[Episode  300] 최근 50 에피소드 평균 리워드 = -0.076, epsilon = 0.222
[Episode  350] 최근 50 에피소드 평균 리워드 = -0.179, epsilon = 0.173
[Episode  400] 최근 50 에피소드 평균 리워드 = -0.200, epsilon = 0.135
[Episode  450] 최근 50 에피소드 평균 리워드 = -0.116, epsilon = 0.105
[Episode  500] 최근 50 에피소드 평균 리워드 = 0.362, epsilon = 0.082

=== 학습 종료 ===

▶ 근사 Q-테이블 (행: 상태, 열: 행동[←,→])
상태 0: [-0.04470164 -0.04139259]
상태 1: [-0.0460864  -0.04379602]
상태 2: [-0.04630476 -0.04427904]
상태 3: [-0.04213484 -0.04170734]
상태 4: [-0.03890941 -0.03502329]

▶ 학습된 정책(Policy)
상태 0  1  2  3  4
      →  →  →  →  G 

▶ 학습된 정책으로 1회 에피소드 실행 예시
방문한 상태들: [0, 1, 2, 3, 4]
스텝 수: 4
마지막 상태가 목표(4)면 학습 성공!


In [10]:

########################################################################################################
## (1-9) IQN(Implicit Quantile Networks) : Q-값의 분포를 세밀하게 조정하는 방식
########################################################################################################

import numpy as np  # 수치 계산을 위한 numpy

# 난수 시드 고정 (실행마다 동일한 결과를 얻기 위해)
np.random.seed(42)

# ======================================
# 1. 환경 설정 (1차원 선형 월드)
# ======================================
n_states = 5   # 상태 개수: 0,1,2,3,4  (4가 목표 상태)
n_actions = 2  # 행동 개수: 0=왼쪽, 1=오른쪽

def step(state, action):
    # action이 0이면 왼쪽으로 이동
    if action == 0:
        next_state = max(0, state - 1)              # 왼쪽 끝(0) 아래로 내려가지 않도록 제한
    # action이 1이면 오른쪽으로 이동
    else:
        next_state = min(n_states - 1, state + 1)   # 오른쪽 끝(4) 위로 넘어가지 않도록 제한

    # 목표 상태(4)에 도달한 경우
    if next_state == n_states - 1:
        reward = 1.0                                # 목표 도달 보상 +1
        done = True                                 # 에피소드 종료
    # 그 외의 경우
    else:
        reward = -0.01                              # 한 스텝마다 작은 패널티 (빨리 도달하도록 유도)
        done = False                                # 에피소드 계속 진행

    return next_state, reward, done                 # 다음 상태, 보상, 종료 여부 반환

def reset():
    # 에피소드 시작 시 항상 상태 0에서 시작
    return 0

def state_to_onehot(state):
    # 상태를 one-hot 벡터(1 x n_states)로 변환
    x = np.zeros((1, n_states), dtype=np.float32)   # 1행 n_states열의 0 벡터 생성
    x[0, state] = 1.0                               # 해당 상태 인덱스 위치만 1로 설정
    return x

# ======================================
# 2. IQN(Implicit Quantile Network) 구조 정의
#    - 입력: 상태(one-hot) + 샘플링된 quantile τ (스칼라)
#    - 출력: 각 행동별 quantile 값 Q(s,a | τ)
# ======================================
input_dim  = n_states + 1    # 상태 one-hot(n_states) + τ(1)
hidden_dim = 32              # 은닉층 크기
output_dim = n_actions       # 출력 차원: 각 행동별 quantile 값

# 온라인 네트워크(Online IQN) 파라미터
W1 = 0.1 * np.random.randn(input_dim, hidden_dim)   # 1층 가중치 (입력 → 은닉)
b1 = np.zeros((1, hidden_dim))                      # 1층 편향
W2 = 0.1 * np.random.randn(hidden_dim, output_dim)  # 2층 가중치 (은닉 → 행동별 quantile 값)
b2 = np.zeros((1, output_dim))                      # 2층 편향

# 타깃 네트워크(Target IQN) 파라미터 (초기에는 온라인과 동일하게 설정)
W1_tgt = W1.copy()
b1_tgt = b1.copy()
W2_tgt = W2.copy()
b2_tgt = b2.copy()

def relu(x):
    # ReLU 활성화 함수: 0보다 작으면 0, 크면 그대로
    return np.maximum(0, x)

def relu_deriv(x):
    # ReLU 도함수: x>0이면 1, 아니면 0
    return (x > 0).astype(np.float32)

def forward_iqn(states, taus, W1, b1, W2, b2):
    # IQN 순전파 함수
    # states : (B, n_states)       - 배치 상태(one-hot)
    # taus   : (B, N, 1)           - 각 배치에 대해 N개의 quantile τ 샘플
    # 반환값 : inp_flat, z1, h, q  (q: (B, N, n_actions))
    B = states.shape[0]                 # 배치 크기
    N = taus.shape[1]                   # quantile 샘플 개수

    # 상태를 (B, 1, n_states) → (B, N, n_states) 로 반복 확장
    state_expanded = np.repeat(states[:, None, :], N, axis=1)  # (B, N, n_states)

    # 상태와 τ를 concat → (B, N, n_states + 1)
    inp = np.concatenate([state_expanded, taus], axis=2)

    # (B, N, input_dim) → (B*N, input_dim) 로 평탄화
    inp_flat = inp.reshape(B * N, -1)

    # 1층: 선형 + ReLU
    z1 = inp_flat @ W1 + b1            # (B*N, hidden_dim)
    h  = relu(z1)                      # (B*N, hidden_dim)

    # 2층: 선형 출력 (각 행동별 quantile 값)
    out = h @ W2 + b2                  # (B*N, n_actions)

    # 다시 (B, N, n_actions) 형태로 reshape
    q = out.reshape(B, N, output_dim)

    return inp_flat, z1, h, q

# ======================================
# 3. IQN 하이퍼파라미터 설정
# ======================================
gamma = 0.9             # 할인율
epsilon = 1.0           # ε-greedy에서 탐험 비율 시작값
epsilon_min = 0.05      # ε의 최소값
epsilon_decay = 0.995   # 에피소드마다 ε 감소 비율

learning_rate = 0.001   # 신경망 파라미터 학습률
n_episodes = 500        # 총 학습 에피소드 수
max_steps  = 20         # 한 에피소드에서 최대 스텝 수

buffer_capacity   = 1000  # 리플레이 버퍼 최대 크기
batch_size        = 32    # 미니배치 크기
warmup_steps      = 100   # 최소 이 정도 샘플이 쌓인 후부터 학습 시작
target_update_epi = 50    # 타깃 네트워크를 몇 에피소드마다 업데이트할지

n_quantiles_train = 32    # 학습 시 사용할 quantile 샘플 개수
n_quantiles_eval  = 128   # 평가(Q-테이블 출력) 시 사용할 quantile 샘플 개수

# ======================================
# 4. 리플레이 버퍼 구현 (간단한 리스트 버퍼)
# ======================================
replay_buffer = []  # (state, action, reward, next_state, done) 튜플 저장

def add_to_buffer(state, action, reward, next_state, done):
    # 버퍼가 가득 찼으면 가장 오래된 데이터 삭제
    if len(replay_buffer) >= buffer_capacity:
        replay_buffer.pop(0)
    # 새 transition 추가
    replay_buffer.append((state, action, reward, next_state, done))

def sample_from_buffer(batch_size):
    # 버퍼에서 랜덤하게 batch_size 개의 transition 샘플 추출
    indices = np.random.choice(len(replay_buffer), size=batch_size, replace=False)
    batch = [replay_buffer[i] for i in indices]
    return batch

# ======================================
# 5. ε-greedy 정책 (IQN 분포의 기대값을 이용)
# ======================================
def choose_action(state, epsilon):
    # ε 확률로 랜덤 행동(탐험)
    if np.random.rand() < epsilon:
        return np.random.randint(n_actions)

    # 1-ε 확률로 분포의 기대값(평균)을 기준으로 greedy 행동 선택
    x = state_to_onehot(state)                                        # (1, n_states)
    B = 1
    N = n_quantiles_train                                             # 기대값 계산용 τ 샘플 개수
    taus = np.random.rand(B, N, 1).astype(np.float32)                 # (1, N, 1), U(0,1)에서 샘플링

    _, _, _, q = forward_iqn(x, taus, W1, b1, W2, b2)                 # q: (1, N, n_actions)

    q_mean = np.mean(q, axis=1)                                       # (1, n_actions)  → 각 행동별 기대값
    action = int(np.argmax(q_mean, axis=1)[0])                        # 기대값이 가장 큰 행동 선택
    return action

# ======================================
# 6. IQN 학습 함수
#    - quantile regression + Huber loss 사용
# ======================================
def train_iqn(batch_size):
    global W1, b1, W2, b2

    # 리플레이 버퍼에서 미니배치 샘플 추출
    batch = sample_from_buffer(batch_size)

    # 각 성분별로 분리하여 numpy 배열로 변환
    states      = np.array([s  for (s, a, r, ns, d) in batch], dtype=np.int32)
    actions     = np.array([a  for (s, a, r, ns, d) in batch], dtype=np.int32)
    rewards     = np.array([r  for (s, a, r, ns, d) in batch], dtype=np.float32)
    next_states = np.array([ns for (s, a, r, ns, d) in batch], dtype=np.int32)
    dones       = np.array([d  for (s, a, r, ns, d) in batch], dtype=bool)

    # 상태들을 one-hot 벡터로 변환
    X      = np.vstack([state_to_onehot(s)  for s in states])      # (B, n_states)
    X_next = np.vstack([state_to_onehot(ns) for ns in next_states])# (B, n_states)

    B = batch_size                       # 배치 크기
    N = n_quantiles_train                # 학습 시 quantile 샘플 개수

    # 현재 상태용 quantile τ 샘플링 (U(0,1))
    taus      = np.random.rand(B, N, 1).astype(np.float32)         # (B, N, 1)
    # 다음 상태용 quantile τ 샘플링 (타깃용)
    taus_next = np.random.rand(B, N, 1).astype(np.float32)         # (B, N, 1)

    # 온라인 네트워크로 현재 상태의 quantile 분포 계산
    inp_flat, z1, h, q = forward_iqn(X, taus, W1, b1, W2, b2)      # q: (B, N, n_actions)

    # 현재 상태에서 선택된 행동에 대한 quantile 값만 추출 (B, N)
    batch_idx = np.arange(B)[:, None]                              # (B, 1)
    tau_idx   = np.arange(N)[None, :]                              # (1, N)
    q_pred = q[batch_idx, tau_idx, actions[:, None]]               # (B, N)

    # 타깃 네트워크로 다음 상태의 quantile 분포 계산
    _, _, _, q_next_all = forward_iqn(X_next, taus_next, W1_tgt, b1_tgt, W2_tgt, b2_tgt)  # (B, N, n_actions)

    # 다음 상태에서 각 행동별 기대 Q값(평균) 계산
    q_next_mean = np.mean(q_next_all, axis=1)                      # (B, n_actions)

    # 다음 상태에서 greedy 행동 선택 (Double DQN 스타일의 행동 선택)
    best_next_actions = np.argmax(q_next_mean, axis=1)             # (B,)

    # 선택된 greedy 행동에 대한 다음 상태의 quantile 값만 추출 (B, N)
    q_next = q_next_all[batch_idx, tau_idx, best_next_actions[:, None]]  # (B, N)

    # done이면 미래 보상이 없으므로 (1 - done) 곱해줌
    not_dones = (~dones).astype(np.float32)                        # (B,)

    # TD Target: Z_target = r + γ * (1 - done) * Z_next
    Z_target = rewards[:, None] + gamma * not_dones[:, None] * q_next   # (B, N)

    # δ = Z_target - Z_pred  (quantile 간 오차)
    delta = Z_target - q_pred                                      # (B, N)

    # quantile fraction τ (각 샘플마다 다름)
    taus_flat = taus.squeeze(-1)                                   # (B, N)

    # δ < 0 인지에 따른 indicator (quantile regression의 비대칭 가중치 요소)
    indicator = (delta < 0).astype(np.float32)                     # (B, N)

    # Huber 손실의 파라미터 kappa
    kappa = 1.0

    # Huber 손실 값 계산
    abs_delta = np.abs(delta)                                      # (B, N)
    huber = np.where(abs_delta <= kappa,
                     0.5 * delta**2,
                     kappa * (abs_delta - 0.5 * kappa))           # (B, N)

    # IQN의 quantile 가중치: |τ - 1_{δ<0}|
    weight = np.abs(taus_flat - indicator)                         # (B, N)

    # 최종 loss 요소 = weight * huber  (여기서는 mean(loss_elements)를 쓰지만,
    # backward에서 평균을 고려하여 gradient에 1/(B*N) 반영)
    # loss_elements = weight * huber
    # L = mean(loss_elements)

    # Huber 손실의 도함수: d huber / dδ
    grad_huber = np.where(abs_delta <= kappa,
                          delta,
                          kappa * np.sign(delta))                  # (B, N)

    # L = (1/(B*N)) * Σ weight * huber 이므로
    # dL/dδ = (1/(B*N)) * weight * d huber/dδ
    BN = B * N
    dL_ddelta = weight * grad_huber / BN                           # (B, N)

    # δ = Z_target - Z_pred 이므로 dL/dZ_pred = - dL/dδ
    dL_dZ_pred = -dL_ddelta                                        # (B, N)

    # q (B, N, n_actions) 에서 선택된 action 위치에만 gradient를 반영
    dQ = np.zeros_like(q)                                          # (B, N, n_actions)
    dQ[batch_idx, tau_idx, actions[:, None]] = dL_dZ_pred          # (B, N)

    # 이제 dQ를 이용해 신경망 파라미터에 대한 gradient 계산
    # (B, N, n_actions) → (B*N, n_actions) 로 reshape
    dQ_flat = dQ.reshape(BN, output_dim)                           # (B*N, n_actions)

    # 2층(출력층) gradient: out = h @ W2 + b2
    dW2 = h.T @ dQ_flat                                            # (hidden_dim x n_actions)
    db2 = np.sum(dQ_flat, axis=0, keepdims=True)                   # (1 x n_actions)

    # 은닉층에 전달되는 gradient
    dh = dQ_flat @ W2.T                                            # (B*N, hidden_dim)

    # 1층 z1에 대한 gradient (ReLU 도함수 곱)
    dz1 = dh * relu_deriv(z1)                                      # (B*N, hidden_dim)

    # 1층(입력층) gradient: z1 = inp_flat @ W1 + b1
    dW1 = inp_flat.T @ dz1                                         # (input_dim x hidden_dim)
    db1 = np.sum(dz1, axis=0, keepdims=True)                       # (1 x hidden_dim)

    # 경사하강법으로 파라미터 업데이트
    W2 -= learning_rate * dW2
    b2 -= learning_rate * db2
    W1 -= learning_rate * dW1
    b1 -= learning_rate * db1

# ======================================
# 7. IQN 학습 루프
# ======================================
reward_history = []   # 에피소드별 총 보상을 기록할 리스트
total_steps    = 0    # 전체 스텝 수 (옵션)

print("=== 1차원 선형 월드에서의 IQN(NumPy) 학습 시작 ===")

for episode in range(1, n_episodes + 1):

    state = reset()                     # 에피소드 시작 상태(0) 초기화
    total_reward = 0.0                  # 에피소드 누적 보상 초기화

    for step_idx in range(max_steps):
        total_steps += 1

        # (1) ε-greedy 정책으로 행동 선택
        action = choose_action(state, epsilon)

        # (2) 선택한 행동을 환경에 적용 → 다음 상태, 보상, 종료 여부 반환
        next_state, reward, done = step(state, action)

        # (3) 리플레이 버퍼에 transition 저장
        add_to_buffer(state, action, reward, next_state, done)

        # (4) 일정 step 이상 샘플이 쌓였으면 학습 시작
        if len(replay_buffer) >= max(batch_size, warmup_steps):
            train_iqn(batch_size)

        # (5) 누적 보상 업데이트
        total_reward += reward

        # (6) 상태 업데이트
        state = next_state

        # (7) 목표 상태에 도달한 경우 에피소드 종료
        if done:
            break

    # (8) 에피소드 종료 후 epsilon 감소 (탐험 비율을 점점 줄임)
    epsilon = max(epsilon_min, epsilon * epsilon_decay)

    # (9) 에피소드별 총 보상을 기록
    reward_history.append(total_reward)

    # (10) 일정 에피소드마다 타깃 네트워크를 업데이트하고 로그 출력
    if episode % target_update_epi == 0:
        W1_tgt = W1.copy()
        b1_tgt = b1.copy()
        W2_tgt = W2.copy()
        b2_tgt = b2.copy()

    if episode % 50 == 0:
        avg_reward = np.mean(reward_history[-50:])
        print(f"[Episode {episode:4d}] 최근 50 에피소드 평균 리워드 = {avg_reward:.3f}, epsilon = {epsilon:.3f}")

print("\n=== 학습 종료 ===\n")

# ======================================
# 8. 학습된 IQN으로 '근사 Q-테이블' 출력
#    - 여러 τ를 샘플링하여 기대값 Q(s,a)를 추정
# ======================================
print("▶ 근사 Q-테이블 (행: 상태, 열: 행동[←,→])")

for s in range(n_states):
    x = state_to_onehot(s)                                            # (1, n_states)
    B = 1
    N = n_quantiles_eval                                              # 평가용 quantile 샘플 개수
    taus = np.random.rand(B, N, 1).astype(np.float32)                 # (1, N, 1)

    _, _, _, q = forward_iqn(x, taus, W1, b1, W2, b2)                 # (1, N, n_actions)
    q_mean = np.mean(q, axis=1)[0]                                    # (n_actions,)  기대값

    print(f"상태 {s}: {q_mean}")

# ======================================
# 9. 학습된 정책(Policy) 확인 (greedy 정책)
# ======================================
action_symbols = {0: "←", 1: "→"}                                     # 행동 인덱스를 화살표로 표현

print("\n▶ 학습된 정책(Policy)")

policy_str = ""
for s in range(n_states):
    if s == n_states - 1:
        policy_str += " G "                                           # 목표 상태는 G로 표시
    else:
        x = state_to_onehot(s)
        B = 1
        N = n_quantiles_eval
        taus = np.random.rand(B, N, 1).astype(np.float32)
        _, _, _, q = forward_iqn(x, taus, W1, b1, W2, b2)
        q_mean = np.mean(q, axis=1)                                   # (1, n_actions)
        best_action = int(np.argmax(q_mean, axis=1)[0])
        policy_str += f" {action_symbols[best_action]} "

print("상태 0  1  2  3  4")
print("     " + policy_str)

# ======================================
# 10. 학습된 정책으로 1회 에피소드 실행 (탐험 없이 greedy만 사용)
# ======================================
print("\n▶ 학습된 정책으로 1회 에피소드 실행 예시")

state = reset()                               # 초기 상태 0
trajectory = [state]                          # 방문한 상태들을 저장할 리스트

for step_idx in range(max_steps):

    x = state_to_onehot(state)
    B = 1
    N = n_quantiles_eval
    taus = np.random.rand(B, N, 1).astype(np.float32)
    _, _, _, q = forward_iqn(x, taus, W1, b1, W2, b2)
    q_mean = np.mean(q, axis=1)               # (1, n_actions)
    action = int(np.argmax(q_mean, axis=1)[0])# 탐험 없이 항상 greedy 행동 선택

    next_state, reward, done = step(state, action)
    trajectory.append(next_state)
    state = next_state

    if done:
        break

print("방문한 상태들:", trajectory)
print("스텝 수:", len(trajectory) - 1)
print("마지막 상태가 목표(4)면 학습 성공!")


=== 1차원 선형 월드에서의 IQN(NumPy) 학습 시작 ===
[Episode   50] 최근 50 에피소드 평균 리워드 = 0.602, epsilon = 0.778
[Episode  100] 최근 50 에피소드 평균 리워드 = 0.807, epsilon = 0.606
[Episode  150] 최근 50 에피소드 평균 리워드 = 0.919, epsilon = 0.471
[Episode  200] 최근 50 에피소드 평균 리워드 = 0.949, epsilon = 0.367
[Episode  250] 최근 50 에피소드 평균 리워드 = 0.953, epsilon = 0.286
[Episode  300] 최근 50 에피소드 평균 리워드 = 0.955, epsilon = 0.222
[Episode  350] 최근 50 에피소드 평균 리워드 = 0.962, epsilon = 0.173
[Episode  400] 최근 50 에피소드 평균 리워드 = 0.965, epsilon = 0.135
[Episode  450] 최근 50 에피소드 평균 리워드 = 0.964, epsilon = 0.105
[Episode  500] 최근 50 에피소드 평균 리워드 = 0.966, epsilon = 0.082

=== 학습 종료 ===

▶ 근사 Q-테이블 (행: 상태, 열: 행동[←,→])
상태 0: [-0.03301757  0.13893495]
상태 1: [-0.00705666  0.21028906]
상태 2: [0.01188504 0.21389171]
상태 3: [-0.03812291  0.23077639]
상태 4: [0.04196164 0.20656024]

▶ 학습된 정책(Policy)
상태 0  1  2  3  4
      →  →  →  →  G 

▶ 학습된 정책으로 1회 에피소드 실행 예시
방문한 상태들: [0, 1, 2, 3, 4]
스텝 수: 4
마지막 상태가 목표(4)면 학습 성공!


In [11]:

########################################################################################################
## (1-10) Rainbow : 여러 DQN 확장(PER, Double DQN, C51 등)을 결합한 통합 알고리즘 (DeepMind, 2017)
########################################################################################################

import numpy as np  # 수치 계산을 위한 numpy

# 난수 시드 고정 (재현 가능한 결과를 위해)
np.random.seed(42)

# ======================================
# 1. 환경 설정 (1차원 선형 월드)
#    상태: 0,1,2,3,4  (4가 목표 상태)
#    행동: 0=왼쪽, 1=오른쪽
# ======================================
n_states = 5
n_actions = 2

def step(state, action):
    # action이 0이면 왼쪽으로 한 칸 이동
    if action == 0:
        next_state = max(0, state - 1)
    # action이 1이면 오른쪽으로 한 칸 이동
    else:
        next_state = min(n_states - 1, state + 1)

    # 목표 상태(4)에 도달한 경우
    if next_state == n_states - 1:
        reward = 1.0      # 목표 도달 보상
        done = True       # 에피소드 종료
    else:
        reward = -0.01    # 매 스텝마다 작은 패널티
        done = False

    return next_state, reward, done

def reset():
    # 항상 상태 0에서 에피소드 시작
    return 0

def state_to_onehot(state):
    # 상태를 one-hot 벡터(1 x n_states)로 변환
    x = np.zeros((1, n_states), dtype=np.float32)
    x[0, state] = 1.0
    return x

# ======================================
# 2. Rainbow에서 사용할 C51 설정
#    - 분포형 값함수(Categorical DQN)
#    - 가치 범위 [Vmin, Vmax]를 n_atoms개 구간으로 나눔
# ======================================
n_atoms = 51                 # atom 개수
Vmin = -1.0                  # 최소 가치
Vmax =  1.0                  # 최대 가치
delta_z = (Vmax - Vmin) / (n_atoms - 1)  # atom 간격
z_support = np.linspace(Vmin, Vmax, n_atoms)  # atom 값들 (shape: (n_atoms,))

# ======================================
# 3. Rainbow용 네트워크 구조
#    - 입력: 상태 one-hot (5차원)
#    - 은닉층: Noisy Linear (고정 sigma, 학습되는 W,b)
#    - 출력: Dueling 구조 + C51 분포
#      * Value stream  : V(s, z) (각 atom에 대한 값)
#      * Advantage stream: A(s, a, z)
#      * Q 분포: V + (A - 평균(A))
# ======================================
input_dim  = n_states       # 상태 one-hot 차원
hidden_dim = 32             # 은닉층 크기

# Noisy layer의 sigma는 여기서는 "고정"으로 두고, W,b만 학습
noisy_sigma_1   = 0.5       # 1층 노이즈 스케일
noisy_sigma_val = 0.5       # Value stream 노이즈 스케일
noisy_sigma_adv = 0.5       # Advantage stream 노이즈 스케일

# ----- 온라인 네트워크 파라미터 -----
# 1층: 입력 → 은닉 (Noisy Linear)
W1 = 0.1 * np.random.randn(input_dim, hidden_dim)
b1 = np.zeros((1, hidden_dim))
W1_sigma = noisy_sigma_1 * np.ones_like(W1)  # 고정 sigma

# Value stream: 은닉 → atom 값 (Noisy Linear)
W_val = 0.1 * np.random.randn(hidden_dim, n_atoms)
b_val = np.zeros((1, n_atoms))
W_val_sigma = noisy_sigma_val * np.ones_like(W_val)

# Advantage stream: 은닉 → (행동 × atom) (Noisy Linear)
W_adv = 0.1 * np.random.randn(hidden_dim, n_actions * n_atoms)
b_adv = np.zeros((1, n_actions * n_atoms))
W_adv_sigma = noisy_sigma_adv * np.ones_like(W_adv)

# ----- 타깃 네트워크 파라미터 (초기에는 동일하게 설정) -----
W1_tgt = W1.copy()
b1_tgt = b1.copy()
W_val_tgt = W_val.copy()
b_val_tgt = b_val.copy()
W_adv_tgt = W_adv.copy()
b_adv_tgt = b_adv.copy()

def relu(x):
    # ReLU 활성화 함수
    return np.maximum(0, x)

def relu_deriv(x):
    # ReLU 도함수: x>0이면 1, 아니면 0
    return (x > 0).astype(np.float32)

def noisy_linear(x, W, b, W_sigma, use_noise=True):
    # Noisy Linear 레이어:
    # y = x @ (W + W_sigma * eps) + b
    # - eps는 N(0,1)에서 샘플
    # - 여기서는 sigma는 고정, W만 학습
    if use_noise:
        eps = np.random.randn(*W.shape)          # W와 같은 shape의 노이즈
        W_eff = W + W_sigma * eps                # 유효 가중치
    else:
        W_eff = W                                # 타깃 네트워크 등에서 noise 없이 사용

    y = x @ W_eff + b                            # 선형 연산
    return y, W_eff                              # y와, 역전파에 사용할 W_eff 반환

def forward_rainbow(x, use_noise=True, target=False):
    # Rainbow 네트워크 순전파
    # x          : (배치, input_dim)
    # use_noise  : Noisy layer 사용 여부
    # target     : 타깃 네트워크 사용 여부
    # 반환값     : h, logits, probs
    if not target:
        # 온라인 네트워크 사용
        z1, W1_eff = noisy_linear(x, W1, b1, W1_sigma, use_noise=use_noise)  # (B, hidden)
        h = relu(z1)                                                         # (B, hidden)

        v_logits, W_val_eff = noisy_linear(h, W_val, b_val, W_val_sigma, use_noise=use_noise)   # (B, n_atoms)
        adv_flat_logits, W_adv_eff = noisy_linear(h, W_adv, b_adv, W_adv_sigma, use_noise=use_noise)  # (B, n_actions*n_atoms)
    else:
        # 타깃 네트워크 사용 (noise 없이)
        z1 = x @ W1_tgt + b1_tgt                              # (B, hidden)
        h = relu(z1)                                          # (B, hidden)

        v_logits = h @ W_val_tgt + b_val_tgt                  # (B, n_atoms)
        adv_flat_logits = h @ W_adv_tgt + b_adv_tgt           # (B, n_actions*n_atoms)
        W1_eff = W1_tgt
        W_val_eff = W_val_tgt
        W_adv_eff = W_adv_tgt

    batch_size = x.shape[0]                                   # 배치 크기
    adv_logits = adv_flat_logits.reshape(batch_size, n_actions, n_atoms)  # (B, A, Z)

    v_logits_expanded = v_logits[:, None, :]                  # (B, 1, Z)
    adv_mean = np.mean(adv_logits, axis=1, keepdims=True)     # (B, 1, Z)

    logits = v_logits_expanded + (adv_logits - adv_mean)      # Dueling 결합 (B, A, Z)

    # Softmax를 atom 축(Z) 방향으로 적용하여 확률 분포로 변환
    logits_shifted = logits - np.max(logits, axis=2, keepdims=True)  # overflow 방지
    exp_logits = np.exp(logits_shifted)
    probs = exp_logits / (np.sum(exp_logits, axis=2, keepdims=True) + 1e-8)  # (B, A, Z)

    return h, logits, probs, W1_eff, W_val_eff, W_adv_eff, z1, v_logits, adv_logits

# ======================================
# 4. Prioritized Replay Buffer (PER)
#    - Rainbow 구성 요소: PER (Proportional)
# ======================================
buffer_capacity = 1000     # 최대 버퍼 크기
replay_buffer = []         # (s, a, r, ns, done) 튜플 저장
priorities = []            # 각 transition의 priority 저장

alpha = 0.6                # PER에서 priority 지수 (0=uniform, 1=greedy)
beta = 0.4                 # IS(importance sampling) 지수
priority_eps = 1e-6        # priority가 0이 되지 않도록 하는 작은 값

def add_to_buffer(state, action, reward, next_state, done):
    # 새로운 transition을 리플레이 버퍼에 추가
    # priority는 현재까지의 최대 priority로 초기화
    if len(priorities) > 0:
        max_prio = max(priorities)
    else:
        max_prio = 1.0

    if len(replay_buffer) >= buffer_capacity:
        # 버퍼가 가득 찼으면 FIFO로 가장 오래된 것 제거
        replay_buffer.pop(0)
        priorities.pop(0)

    replay_buffer.append((state, action, reward, next_state, done))
    priorities.append(max_prio)

def sample_from_buffer_per(batch_size):
    # PER(Proportional)에 따라 미니배치 샘플링
    prios = np.array(priorities, dtype=np.float32)
    # priority^alpha
    scaled_prios = prios ** alpha
    # 확률 분포로 정규화
    P = scaled_prios / (np.sum(scaled_prios) + 1e-8)

    # 인덱스를 확률 P에 따라 샘플
    indices = np.random.choice(len(replay_buffer), size=batch_size, p=P, replace=False)

    # IS weight 계산: w_i = (N * P(i))^-beta / max_i(...)
    N = len(replay_buffer)
    weights = (N * P[indices]) ** (-beta)
    weights = weights / (np.max(weights) + 1e-8)
    weights = weights.astype(np.float32)

    batch = [replay_buffer[i] for i in indices]
    return batch, indices, weights

# ======================================
# 5. C51 분포 프로젝션 함수
#    - r + γ z 를 [Vmin, Vmax] 구간의 atom들로 분배
# ======================================
def project_distribution(rewards, dones, next_probs):
    # rewards : (B,)       - 보상
    # dones   : (B,) bool  - 종료 여부
    # next_probs : (B, n_atoms) - 다음 상태 분포 p(z|s',a*)
    # 반환값 : (B, n_atoms) - 현재 상태 분포의 타깃 m(z)
    batch_size = rewards.shape[0]

    # 종료 상태면 미래 보상 없음, 아니면 r + γ z
    Tz = rewards[:, None] + (1.0 - dones.astype(np.float32))[:, None] * (gamma * z_support[None, :])
    Tz = np.clip(Tz, Vmin, Vmax)           # [Vmin, Vmax] 범위로 자름

    b = (Tz - Vmin) / delta_z              # atom index의 실수 위치
    l = np.floor(b).astype(int)
    u = np.ceil(b).astype(int)

    l = np.clip(l, 0, n_atoms - 1)
    u = np.clip(u, 0, n_atoms - 1)

    m = np.zeros((batch_size, n_atoms), dtype=np.float32)

    for j in range(n_atoms):
        lj = l[:, j]
        uj = u[:, j]
        bj = b[:, j]
        pj = next_probs[:, j]

        m[np.arange(batch_size), lj] += pj * (uj - bj)
        m[np.arange(batch_size), uj] += pj * (bj - lj)

    m_sum = np.sum(m, axis=1, keepdims=True)
    m = m / (m_sum + 1e-8)
    return m

# ======================================
# 6. 하이퍼파라미터 설정 (Rainbow 스타일)
#    - Double DQN + Dueling + PER + NoisyNet + C51
# ======================================
gamma = 0.9             # 할인율
learning_rate = 0.0005  # 학습률 (조금 작게)
n_episodes = 500        # 에피소드 수
max_steps  = 20         # 에피소드당 최대 스텝 수

batch_size       = 32   # 미니배치 크기
warmup_steps     = 100  # 최소 이 정도 샘플 쌓인 후 학습 시작
target_update_ep = 20   # 타깃 네트워크 업데이트 주기(에피소드 단위)

# Rainbow에서는 보통 NoisyNet으로 탐험을 하므로 epsilon 거의 사용하지 않음
epsilon = 0.0           # ε-greedy는 0으로 두고, NoisyNet만으로 탐험
epsilon_min = 0.0
epsilon_decay = 1.0     # 사용하지 않지만 형식상 둠

# ======================================
# 7. 행동 선택 함수 (NoisyNet + 분포 기대값 기반 greedy)
# ======================================
def choose_action(state):
    # 상태를 one-hot으로 변환
    x = state_to_onehot(state)                         # (1, n_states)
    # 온라인 네트워크 순전파 (Noisy 사용)
    _, _, probs, _, _, _, _, _, _ = forward_rainbow(x, use_noise=True, target=False)
    # 기대 Q값 계산: Q(a) = Σ_z z * p(z|s,a)
    q_values = np.sum(z_support[None, None, :] * probs, axis=2)  # (1, n_actions)
    action = int(np.argmax(q_values, axis=1)[0])
    return action

# ======================================
# 8. Rainbow 학습 함수
# ======================================
def train_rainbow(batch_size):
    global W1, b1, W_val, b_val, W_adv, b_adv
    global W1_tgt, b1_tgt, W_val_tgt, b_val_tgt, W_adv_tgt, b_adv_tgt
    global priorities

    # PER에서 미니배치 샘플
    batch, indices, is_weights = sample_from_buffer_per(batch_size)

    # 배치를 각 성분별로 분리
    states      = np.array([s  for (s, a, r, ns, d) in batch], dtype=np.int32)
    actions     = np.array([a  for (s, a, r, ns, d) in batch], dtype=np.int32)
    rewards     = np.array([r  for (s, a, r, ns, d) in batch], dtype=np.float32)
    next_states = np.array([ns for (s, a, r, ns, d) in batch], dtype=np.int32)
    dones       = np.array([d  for (s, a, r, ns, d) in batch], dtype=bool)

    # 상태를 one-hot으로 변환
    X      = np.vstack([state_to_onehot(s)  for s in states])      # (B, n_states)
    X_next = np.vstack([state_to_onehot(ns) for ns in next_states])# (B, n_states)
    B = X.shape[0]

    batch_idx = np.arange(B)

    # ----- 1) 온라인 네트워크로 현재 상태 분포 계산 -----
    h, logits, probs, W1_eff, W_val_eff, W_adv_eff, z1, v_logits, adv_logits = forward_rainbow(
        X, use_noise=True, target=False
    )  # probs: (B, A, Z)

    # 현재 상태에서 수행한 행동 a에 대한 분포 p(z|s,a)
    current_probs = probs[batch_idx, actions, :]                   # (B, n_atoms)

    # ----- 2) Double DQN: 다음 상태에서 온라인 네트워크로 행동 선택 -----
    _, _, probs_next_online, _, _, _, _, _, _ = forward_rainbow(
        X_next, use_noise=True, target=False
    )  # (B, A, Z)

    q_next_online = np.sum(z_support[None, None, :] * probs_next_online, axis=2)  # (B, A)
    best_next_actions = np.argmax(q_next_online, axis=1)                          # (B,)

    # ----- 3) 타깃 네트워크에서 선택된 행동의 분포 가져오기 -----
    _, _, probs_next_target, _, _, _, _, _, _ = forward_rainbow(
        X_next, use_noise=False, target=True
    )  # (B, A, Z)

    next_dist = probs_next_target[batch_idx, best_next_actions, :]               # (B, n_atoms)

    # ----- 4) C51 분포 프로젝션으로 타깃 분포 계산 -----
    target_dist = project_distribution(rewards, dones, next_dist)               # (B, n_atoms)

    # ----- 5) 손실 및 gradient 계산 (Cross-Entropy + IS weight) -----
    # cross-entropy: L = - Σ m(z) * log p(z)
    # softmax + cross-entropy의 gradient: dL/dlogits = p - m
    # PER의 IS weight w_i를 곱해줌
    is_w = is_weights[:, None]                                                  # (B, 1)

    d_logits = np.zeros_like(logits)                                            # (B, A, Z)
    diff = (current_probs - target_dist) * is_w                                 # (B, Z)
    d_logits[batch_idx, actions, :] = diff                                      # (B, Z)를 해당 action 위치에 배치

    # ----- 6) Dueling 구조에 따른 gradient 분리 -----
    # logits = V + (Adv - mean_adv)
    # ∂L/∂V = Σ_a ∂L/∂logits_a
    # ∂L/∂Adv_a = ∂L/∂logits_a - 1/A Σ_a' ∂L/∂logits_a'
    dV = np.sum(d_logits, axis=1)                                               # (B, Z)
    mean_d_logits = np.mean(d_logits, axis=1, keepdims=True)                    # (B, 1, Z)
    dAdv = d_logits - mean_d_logits                                             # (B, A, Z)

    # ----- 7) Value/Advantage stream에 대한 gradient -----
    dV_flat = dV                                                                # (B, Z)
    dAdv_flat = dAdv.reshape(B, n_actions * n_atoms)                            # (B, A*Z)

    dW_val = h.T @ dV_flat                                                      # (hidden x Z)
    db_val = np.sum(dV_flat, axis=0, keepdims=True)                             # (1 x Z)

    dW_adv = h.T @ dAdv_flat                                                    # (hidden x A*Z)
    db_adv = np.sum(dAdv_flat, axis=0, keepdims=True)                           # (1 x A*Z)

    # 은닉층에 대한 gradient: dh = dV @ W_val_eff^T + dAdv_flat @ W_adv_eff^T
    dh = dV_flat @ W_val_eff.T + dAdv_flat @ W_adv_eff.T                        # (B, hidden)

    # 1층 z1에 대한 gradient (ReLU 도함수 곱)
    dz1 = dh * relu_deriv(z1)                                                   # (B, hidden)

    # 1층에 대한 gradient: dW1, db1
    dW1 = X.T @ dz1                                                             # (input_dim x hidden)
    db1 = np.sum(dz1, axis=0, keepdims=True)                                    # (1 x hidden)

    # ----- 8) 파라미터 업데이트 (경사하강법) -----
    W_val -= learning_rate * dW_val
    b_val -= learning_rate * db_val

    W_adv -= learning_rate * dW_adv
    b_adv -= learning_rate * db_adv

    W1 -= learning_rate * dW1
    b1 -= learning_rate * db1

    # ----- 9) PER priority 업데이트 -----
    # priority는 |p - m|의 합으로 정의(간단한 형태)
    td_errors = np.sum(np.abs(current_probs - target_dist), axis=1)             # (B,)
    new_priorities = td_errors + priority_eps

    for idx_buf, pr in zip(indices, new_priorities):
        priorities[idx_buf] = pr

# ======================================
# 9. Rainbow 학습 루프
# ======================================
reward_history = []
total_steps = 0

print("=== 1차원 선형 월드에서의 Rainbow (Double + Dueling + PER + NoisyNet + C51, NumPy) 학습 시작 ===")

for episode in range(1, n_episodes + 1):

    state = reset()
    total_reward = 0.0

    for step_idx in range(max_steps):
        total_steps += 1

        # 1) NoisyNet 기반 행동 선택 (ε-greedy는 사용하지 않음)
        action = choose_action(state)

        # 2) 환경에 행동 적용
        next_state, reward, done = step(state, action)

        # 3) 리플레이 버퍼에 저장
        add_to_buffer(state, action, reward, next_state, done)

        # 4) 충분한 샘플이 쌓이면 Rainbow 학습 수행
        if len(replay_buffer) >= max(batch_size, warmup_steps):
            train_rainbow(batch_size)

        total_reward += reward
        state = next_state

        if done:
            break

    reward_history.append(total_reward)

    # 5) 타깃 네트워크 파라미터 주기적으로 동기화
    if episode % target_update_ep == 0:
        W1_tgt = W1.copy()
        b1_tgt = b1.copy()
        W_val_tgt = W_val.copy()
        b_val_tgt = b_val.copy()
        W_adv_tgt = W_adv.copy()
        b_adv_tgt = b_adv.copy()

    # 6) 50 에피소드마다 로그 출력
    if episode % 50 == 0:
        avg_reward = np.mean(reward_history[-50:])
        print(f"[Episode {episode:4d}] 최근 50 에피소드 평균 리워드 = {avg_reward:.3f}")

print("\n=== 학습 종료 ===\n")

# ======================================
# 10. Rainbow 근사 Q-테이블 출력
#      - 각 상태에서 분포의 기대값으로 Q(s,a) 추정
# ======================================
print("▶ Rainbow 근사 Q-테이블 (행: 상태, 열: 행동[←,→])")

for s in range(n_states):
    x = state_to_onehot(s)
    _, _, probs, _, _, _, _, _, _ = forward_rainbow(x, use_noise=False, target=False)
    q_vals = np.sum(z_support[None, None, :] * probs, axis=2)  # (1, A)
    q_row = q_vals[0]
    print(f"상태 {s}: {q_row}")

# ======================================
# 11. 학습된 정책(Policy) 확인
# ======================================
action_symbols = {0: "←", 1: "→"}

print("\n▶ 학습된 정책(Policy)")
policy_str = ""
for s in range(n_states):
    if s == n_states - 1:
        policy_str += " G "
    else:
        x = state_to_onehot(s)
        _, _, probs, _, _, _, _, _, _ = forward_rainbow(x, use_noise=False, target=False)
        q_vals = np.sum(z_support[None, None, :] * probs, axis=2)
        best_action = int(np.argmax(q_vals, axis=1)[0])
        policy_str += f" {action_symbols[best_action]} "

print("상태 0  1  2  3  4")
print("     " + policy_str)

# ======================================
# 12. Rainbow 정책으로 1회 에피소드 실행 예시
# ======================================
print("\n▶ Rainbow 정책으로 1회 에피소드 실행 예시")

state = reset()
trajectory = [state]

for step_idx in range(max_steps):
    x = state_to_onehot(state)
    _, _, probs, _, _, _, _, _, _ = forward_rainbow(x, use_noise=False, target=False)
    q_vals = np.sum(z_support[None, None, :] * probs, axis=2)
    action = int(np.argmax(q_vals, axis=1)[0])

    next_state, reward, done = step(state, action)
    trajectory.append(next_state)
    state = next_state

    if done:
        break

print("방문한 상태들:", trajectory)
print("스텝 수:", len(trajectory) - 1)
print("마지막 상태가 목표(4)면 학습 성공!")


=== 1차원 선형 월드에서의 Rainbow (Double + Dueling + PER + NoisyNet + C51, NumPy) 학습 시작 ===
[Episode   50] 최근 50 에피소드 평균 리워드 = 0.741
[Episode  100] 최근 50 에피소드 평균 리워드 = 0.779
[Episode  150] 최근 50 에피소드 평균 리워드 = 0.652
[Episode  200] 최근 50 에피소드 평균 리워드 = 0.442
[Episode  250] 최근 50 에피소드 평균 리워드 = 0.284
[Episode  300] 최근 50 에피소드 평균 리워드 = 0.059
[Episode  350] 최근 50 에피소드 평균 리워드 = 0.110
[Episode  400] 최근 50 에피소드 평균 리워드 = 0.728
[Episode  450] 최근 50 에피소드 평균 리워드 = 0.908
[Episode  500] 최근 50 에피소드 평균 리워드 = 0.893

=== 학습 종료 ===

▶ Rainbow 근사 Q-테이블 (행: 상태, 열: 행동[←,→])
상태 0: [-0.01080802 -0.01070711]
상태 1: [-0.01080802 -0.01070711]
상태 2: [-0.01080802 -0.01070711]
상태 3: [-0.01080802 -0.01070711]
상태 4: [-0.01080802 -0.01070711]

▶ 학습된 정책(Policy)
상태 0  1  2  3  4
      →  →  →  →  G 

▶ Rainbow 정책으로 1회 에피소드 실행 예시
방문한 상태들: [0, 1, 2, 3, 4]
스텝 수: 4
마지막 상태가 목표(4)면 학습 성공!


In [12]:
########################################################################################################
## (1-11) SQL(Soft Q-Learning): 엔트로피를 추가하여 Q값 학습을 안정화하는 방식
########################################################################################################

import numpy as np  # 수치 계산을 위한 numpy

# =====================================================
# 0. 난수 시드 고정 (실행마다 같은 결과가 나오도록)
# =====================================================
np.random.seed(42)

# =====================================================
# 1. 환경 정의 (1차원 선형 월드)
#    - 상태: 0, 1, 2, 3, 4 (4가 목표 상태)
#    - 행동: 0=왼쪽, 1=오른쪽
# =====================================================
n_states = 5   # 상태 개수
n_actions = 2  # 행동 개수: 0(←), 1(→)

def step(state, action):
    # 현재 상태 state에서 action을 했을 때, 다음 상태와 보상을 반환하는 함수

    # 행동이 0이면 왼쪽으로 한 칸 이동
    if action == 0:
        next_state = max(0, state - 1)              # 0보다 작아지지 않도록 처리
    # 행동이 1이면 오른쪽으로 한 칸 이동
    else:
        next_state = min(n_states - 1, state + 1)   # 4보다 커지지 않도록 처리

    # 만약 목표 상태(4)에 도달했다면
    if next_state == n_states - 1:
        reward = 1.0        # 목표 도달 보상
        done = True         # 에피소드 종료
    else:
        reward = -0.01      # 매 스텝마다 작은 패널티(빨리 도달하도록 유도)
        done = False        # 아직 종료 아님

    return next_state, reward, done  # (다음 상태, 보상, 종료 여부)

def reset():
    # 에피소드 시작 시 항상 상태 0에서 시작
    return 0

# =====================================================
# 2. Soft Q-Learning (Maximum Entropy RL) 설정
#    - Q(s,a): 상태-행동 가치 함수 (표 형태)
#    - 정책: softmax( Q(s,·) / tau )  (탐험은 tau로 제어)
# =====================================================
Q = np.zeros((n_states, n_actions), dtype=np.float32)  # Q 테이블 0으로 초기화

alpha = 0.1      # 학습률 (learning rate)
gamma = 0.9      # 할인율 (discount factor)
tau   = 0.5      # 소프트맥스 온도(temperature), 크면 탐험↑, 작으면 greedy에 가까움

n_episodes = 500 # 총 학습 에피소드 수
max_steps  = 20  # 한 에피소드에서 최대 스텝 수(무한루프 방지용)

# =====================================================
# 3. Softmax 기반 정책 함수 π(a|s)
#    - Q값을 소프트맥스로 확률로 변환해서 행동을 샘플링
# =====================================================
def softmax_policy(state, tau):
    # 주어진 상태 state에서 softmax( Q(state,·)/tau )로 행동 확률을 계산하고 하나 샘플링

    # 현재 상태에서의 Q값 벡터 (길이: n_actions)
    q_vals = Q[state]                                 # shape: (n_actions,)

    # 온도 tau로 나누어서 softmax에 넣을 값 생성
    prefs = q_vals / tau                              # 선호도(기준값)

    # 숫자 안정성을 위해 최대값을 빼고 exp 계산
    prefs_shifted = prefs - np.max(prefs)             # overflow 방지
    exp_prefs = np.exp(prefs_shifted)                 # exp 연산
    probs = exp_prefs / np.sum(exp_prefs)             # 정규화해서 확률 벡터로 변환

    # probs에 따라 행동을 하나 샘플링
    action = np.random.choice(np.arange(n_actions), p=probs)

    return action, probs                              # (선택된 행동, 행동확률분포)

# =====================================================
# 4. Soft Q-Learning 업데이트 식
#    - Soft Bellman backup:
#      V(s') = tau * log Σ_a' exp( Q(s',a') / tau )
#      Q(s,a) ← Q(s,a) + α [ r + γ * V(s') - Q(s,a) ]
# =====================================================
def soft_value(next_state, tau):
    # 다음 상태 next_state에서의 soft value V(s')
    # V(s') = tau * log Σ_a exp(Q(s',a)/tau)

    q_next = Q[next_state]                               # (n_actions,)
    prefs = q_next / tau
    prefs_shifted = prefs - np.max(prefs)                # overflow 방지
    exp_prefs = np.exp(prefs_shifted)
    log_sum = np.log(np.sum(exp_prefs) + 1e-8)           # log Σ exp
    v = tau * log_sum                                    # soft value
    return v

# =====================================================
# 5. 학습 루프 (Soft Q-Learning)
# =====================================================
reward_history = []  # 에피소드별 총 보상 기록용 리스트

print("=== 1차원 선형 월드에서의 Soft Q-Learning(SQL) 학습 시작 ===")

for episode in range(1, n_episodes + 1):
    # 에피소드 시작 시 상태 초기화
    state = reset()
    total_reward = 0.0

    for step_idx in range(max_steps):
        # 1) softmax 정책으로 행동 선택 (탐험/이용 자동 조절)
        action, action_probs = softmax_policy(state, tau)

        # 2) 환경에 행동 적용 → 다음 상태, 보상, 종료 여부
        next_state, reward, done = step(state, action)

        # 3) soft Q-러닝 업데이트
        #    - 다음 상태가 종료 상태이면 V(next_state)=0으로 처리
        if done:
            v_next = 0.0
        else:
            v_next = soft_value(next_state, tau)    # soft value V(s')

        td_target = reward + gamma * v_next         # r + γ V(s')
        td_error  = td_target - Q[state, action]    # TD 오차
        Q[state, action] += alpha * td_error        # Q 업데이트

        # 4) 누적 보상 계산 및 상태 이동
        total_reward += reward
        state = next_state

        # 5) 목표 상태 도달 시 에피소드 종료
        if done:
            break

    # 에피소드별 총 보상을 기록
    reward_history.append(total_reward)

    # 50 에피소드마다 최근 50개 평균 리워드 출력
    if episode % 50 == 0:
        avg_reward = np.mean(reward_history[-50:])
        print(f"[Episode {episode:4d}] 최근 50 에피소드 평균 리워드 = {avg_reward:.3f}")

print("\n=== 학습 종료 ===\n")

# =====================================================
# 6. 학습된 Q-테이블 출력
# =====================================================
print("▶ 최종 Soft Q-테이블 (행: 상태, 열: 행동[←,→])")
for s in range(n_states):
    print(f"상태 {s}: {Q[s]}")

# =====================================================
# 7. 학습된 정책(가장 확률이 높은 행동 기준의 greedy 정책) 출력
#    - 실제 정책은 softmax지만, 설명을 위해 argmax 기준으로 화살표 표시
# =====================================================
action_symbols = {0: "←", 1: "→"}  # 행동 인덱스를 화살표로 매핑

print("\n▶ 학습된 정책(Policy: greedy w.r.t Soft Q)")
policy_str = ""
for s in range(n_states):
    if s == n_states - 1:
        policy_str += " G "                         # 목표 상태는 G로 표시
    else:
        # 해당 상태에서 Q값이 가장 큰 행동을 greedy 선택
        best_action = int(np.argmax(Q[s]))
        policy_str += f" {action_symbols[best_action]} "

print("상태 0  1  2  3  4")
print("     " + policy_str)

# =====================================================
# 8. 학습된 정책(softmax 기반 greedy)으로 1회 에피소드 실행 예시
#    - 여기서는 tau를 매우 작게 해서 거의 greedy에 가깝게 사용해도 되고
#      그냥 argmax로만 행동을 골라도 됨.
# =====================================================
print("\n▶ 학습된 정책으로 1회 에피소드 실행 예시")

state = reset()
trajectory = [state]

for step_idx in range(max_steps):
    # softmax 정책 대신, Q값 기준 argmax로 완전히 greedy하게 행동
    action = int(np.argmax(Q[state]))

    next_state, reward, done = step(state, action)
    trajectory.append(next_state)
    state = next_state

    if done:
        break

print("방문한 상태들:", trajectory)
print("스텝 수:", len(trajectory) - 1)
print("마지막 상태가 목표(4)면 학습 성공!")


=== 1차원 선형 월드에서의 Soft Q-Learning(SQL) 학습 시작 ===
[Episode   50] 최근 50 에피소드 평균 리워드 = 0.600
[Episode  100] 최근 50 에피소드 평균 리워드 = 0.569
[Episode  150] 최근 50 에피소드 평균 리워드 = 0.456
[Episode  200] 최근 50 에피소드 평균 리워드 = 0.486
[Episode  250] 최근 50 에피소드 평균 리워드 = 0.601
[Episode  300] 최근 50 에피소드 평균 리워드 = 0.556
[Episode  350] 최근 50 에피소드 평균 리워드 = 0.531
[Episode  400] 최근 50 에피소드 평균 리워드 = 0.400
[Episode  450] 최근 50 에피소드 평균 리워드 = 0.444
[Episode  500] 최근 50 에피소드 평균 리워드 = 0.611

=== 학습 종료 ===

▶ 최종 Soft Q-테이블 (행: 상태, 열: 행동[←,→])
상태 0: [0.29254955 0.27151373]
상태 1: [0.29254955 0.22254117]
상태 2: [0.27151343 0.07617304]
상태 3: [0.2223115  0.99999976]
상태 4: [0. 0.]

▶ 학습된 정책(Policy: greedy w.r.t Soft Q)
상태 0  1  2  3  4
      ←  ←  ←  →  G 

▶ 학습된 정책으로 1회 에피소드 실행 예시
방문한 상태들: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
스텝 수: 20
마지막 상태가 목표(4)면 학습 성공!


In [13]:
########################################################################################################
## (1-12) PER(Prioritized Experience Replay) : 중요한 경험을 우선적으로 학습하는 경험 리플레이 전략
########################################################################################################

import numpy as np  # 수치 계산을 위한 numpy

# =====================================================
# 0. 난수 시드 고정 (실행마다 같은 결과가 나오도록)
# =====================================================
np.random.seed(42)

# =====================================================
# 1. 환경 정의 (1차원 선형 월드)
#    - 상태: 0, 1, 2, 3, 4  (4가 목표 상태)
#    - 행동: 0=왼쪽, 1=오른쪽
# =====================================================
n_states = 5   # 상태 개수
n_actions = 2  # 행동 개수: 0(←), 1(→)

def step(state, action):
    # 현재 상태 state에서 action을 했을 때
    # 다음 상태(next_state), 보상(reward), 종료 여부(done)를 반환하는 함수

    # 행동이 0이면 왼쪽으로 한 칸 이동
    if action == 0:
        # 최소 상태 0을 넘지 않도록 방지
        next_state = max(0, state - 1)
    # 행동이 1이면 오른쪽으로 한 칸 이동
    else:
        # 최대 상태 4를 넘지 않도록 방지
        next_state = min(n_states - 1, state + 1)

    # 목표 상태(4)에 도달했는지 확인
    if next_state == n_states - 1:
        # 목표 도달 시 보상 +1
        reward = 1.0
        # 에피소드 종료
        done = True
    else:
        # 그 외에는 한 스텝당 작은 패널티 부여(-0.01)
        # → 조금이라도 더 빨리 목표에 가도록 유도
        reward = -0.01
        done = False

    # (다음 상태, 보상, 종료 여부) 반환
    return next_state, reward, done

def reset():
    # 에피소드 시작 시 항상 상태 0에서 시작
    return 0

def state_to_onehot(state):
    # 정수 상태를 one-hot 벡터로 변환
    # 예: 상태 2 → [0, 0, 1, 0, 0]
    x = np.zeros((1, n_states), dtype=np.float32)
    x[0, state] = 1.0
    return x

# =====================================================
# 2. DQN 신경망 구조 정의 (NumPy로 구현)
#    - 입력: 상태(one-hot, 5차원)
#    - 은닉층: 32 노드, ReLU
#    - 출력: 각 행동(2개)에 대한 Q값
# =====================================================
input_dim  = n_states   # 5
hidden_dim = 32         # 은닉 노드 수
output_dim = n_actions  # 2

# 온라인 Q-네트워크 파라미터 (학습 대상)
W1 = 0.1 * np.random.randn(input_dim, hidden_dim)  # 1층 가중치
b1 = np.zeros((1, hidden_dim))                     # 1층 편향
W2 = 0.1 * np.random.randn(hidden_dim, output_dim) # 2층 가중치
b2 = np.zeros((1, output_dim))                     # 2층 편향

# 타깃 Q-네트워크 파라미터 (온라인 네트워크를 주기적으로 복사)
W1_tgt = W1.copy()
b1_tgt = b1.copy()
W2_tgt = W2.copy()
b2_tgt = b2.copy()

def relu(x):
    # ReLU 활성화 함수: 음수는 0, 양수는 그대로
    return np.maximum(0, x)

def relu_deriv(x):
    # ReLU 도함수: x>0이면 1, 아니면 0
    return (x > 0).astype(np.float32)

def forward_network(x, use_target=False):
    # DQN 순전파 함수
    # x        : 입력 상태 배치 (배치 크기 B, 차원 input_dim)
    # use_target: True면 타깃 네트워크 사용, False면 온라인 네트워크 사용
    # 반환값   : (은닉층 전개전 z1, 은닉층 활성값 h, 출력 Q값)
    if not use_target:
        # 온라인 네트워크 사용
        z1 = x @ W1 + b1          # 1층 선형
        h  = relu(z1)             # ReLU
        out = h @ W2 + b2         # 2층 선형 (Q값)
    else:
        # 타깃 네트워크 사용
        z1 = x @ W1_tgt + b1_tgt
        h  = relu(z1)
        out = h @ W2_tgt + b2_tgt

    return z1, h, out

# =====================================================
# 3. Prioritized Experience Replay(PER) 버퍼 구현
#    - 리플레이 버퍼 + 각각의 transition에 priority 부여
#    - 샘플링 시 priority 비례 확률로 샘플
# =====================================================
buffer_capacity = 1000   # 리플레이 버퍼 최대 크기
replay_buffer  = []      # (state, action, reward, next_state, done) 튜플 목록
priorities     = []      # 각 transition의 priority (같은 인덱스)

alpha = 0.6              # PER에서 priority의 비중 (0=균등, 1=완전 priority)
beta  = 0.4              # IS(importance sampling) 보정 계수
priority_eps = 1e-6      # priority가 0이 되는 것을 방지하는 작은 값

def add_to_buffer(state, action, reward, next_state, done):
    # 새 transition을 리플레이 버퍼에 추가
    # priority는 현재까지의 최대 priority로 초기화 (새로운 경험을 우선 학습)
    if len(priorities) > 0:
        max_prio = max(priorities)
    else:
        max_prio = 1.0

    # 버퍼가 가득 차면 가장 오래된 transition 제거(FIFO)
    if len(replay_buffer) >= buffer_capacity:
        replay_buffer.pop(0)
        priorities.pop(0)

    # 새 transition 및 priority 추가
    replay_buffer.append((state, action, reward, next_state, done))
    priorities.append(max_prio)

def sample_from_buffer_per(batch_size):
    # Prioritized Experience Replay 방식으로 미니배치 샘플링
    # 1) priority 리스트를 기반으로 확률 분포 P(i) 계산
    # 2) P(i)에 따라 인덱스 샘플링
    # 3) IS weight 계산

    # priority 배열로 변환
    prios = np.array(priorities, dtype=np.float32)  # shape: (N,)

    # priority^alpha (alpha가 클수록 큰 TD오차에 더 집중)
    scaled_prios = prios ** alpha

    # 확률 분포로 정규화
    P = scaled_prios / (np.sum(scaled_prios) + 1e-8)

    # P를 이용해 인덱스 샘플링
    indices = np.random.choice(len(replay_buffer), size=batch_size, p=P, replace=False)

    # IS(importance sampling) weight 계산
    # w_i = (N * P(i))^-beta / max_j (N * P(j))^-beta
    N = len(replay_buffer)
    weights = (N * P[indices]) ** (-beta)
    weights = weights / (np.max(weights) + 1e-8)
    weights = weights.astype(np.float32)

    # 샘플링된 transition들
    batch = [replay_buffer[i] for i in indices]

    return batch, indices, weights

# =====================================================
# 4. 하이퍼파라미터 설정
# =====================================================
gamma = 0.9             # 할인율
learning_rate = 0.001   # 학습률
n_episodes = 500        # 에피소드 수
max_steps  = 20         # 에피소드당 최대 스텝 수

batch_size        = 32  # 미니배치 크기
warmup_steps      = 100 # 최소 이 정도 샘플이 쌓여야 학습 시작
target_update_ep  = 20  # 타깃 네트워크 동기화 주기(에피소드 단위)

epsilon     = 0.1       # ε-greedy에서 탐험 확률 (PER 자체는 탐험이 아님)
epsilon_min = 0.05
epsilon_decay = 0.995   # 필요시 서서히 줄일 수 있음

# =====================================================
# 5. ε-greedy 행동 선택 함수 (DQN + PER)
# =====================================================
def choose_action(state, epsilon):
    # 확률 epsilon으로 무작위 행동 -> 탐험
    if np.random.rand() < epsilon:
        return np.random.randint(n_actions)

    # 그렇지 않으면 현재 Q값 기준으로 greedy 행동 선택
    x = state_to_onehot(state)                      # (1, n_states)
    _, _, q_values = forward_network(x, use_target=False)  # 온라인 네트워크 사용
    action = int(np.argmax(q_values, axis=1)[0])   # Q값이 가장 큰 행동 선택
    return action

# =====================================================
# 6. PER + DQN 학습 함수
# =====================================================
def train_dqn_per(batch_size):
    # 전역 변수로 선언 (이 함수 안에서 수정할 예정)
    global W1, b1, W2, b2, priorities

    # PER 방식으로 미니배치 샘플링
    batch, indices, is_weights = sample_from_buffer_per(batch_size)

    # 배치를 각각 분리 (numpy 배열로 변환)
    states      = np.array([s  for (s, a, r, ns, d) in batch], dtype=np.int32)
    actions     = np.array([a  for (s, a, r, ns, d) in batch], dtype=np.int32)
    rewards     = np.array([r  for (s, a, r, ns, d) in batch], dtype=np.float32)
    next_states = np.array([ns for (s, a, r, ns, d) in batch], dtype=np.int32)
    dones       = np.array([d  for (s, a, r, ns, d) in batch], dtype=bool)

    # 상태를 one-hot 벡터로 변환
    X      = np.vstack([state_to_onehot(s)  for s in states])      # (B, n_states)
    X_next = np.vstack([state_to_onehot(ns) for ns in next_states])# (B, n_states)
    B = X.shape[0]

    # 배치 인덱스 (0 ~ B-1)
    batch_idx = np.arange(B)

    # (1) 온라인 네트워크로 현재 상태의 Q값 계산
    z1, h, q_values = forward_network(X, use_target=False)  # q_values: (B, n_actions)

    # 현재 상태에서 실제로 선택된 행동에 대한 Q값만 추출
    q_pred = q_values[batch_idx, actions]                   # (B,)

    # (2) 타깃 네트워크로 다음 상태의 Q값 계산
    _, _, q_next = forward_network(X_next, use_target=True) # (B, n_actions)

    # 다음 상태에서의 최대 Q값 선택 (Double DQN이 아니라 단순 DQN 방식)
    q_next_max = np.max(q_next, axis=1)                     # (B,)

    # 종료 상태이면 미래 보상이 없으므로 0 처리
    not_dones = (~dones).astype(np.float32)                 # (B,)

    # TD target 계산: r + γ * (1-done) * max_a' Q(s',a')
    td_target = rewards + gamma * not_dones * q_next_max    # (B,)

    # TD오차 (δ = target - prediction)
    td_error = td_target - q_pred                           # (B,)

    # MSE 손실의 gradient (IS weight 포함):
    # L = mean( w_i * (td_error_i)^2 )
    # dL/dq_pred = -2 * w_i * td_error_i / B
    is_w = is_weights                                      # (B,)
    dL_dq_pred = -2.0 * is_w * td_error / B               # (B,)

    # Q값 전체(q_values)에 대한 gradient 초기화 (0으로)
    dQ = np.zeros_like(q_values)                           # (B, n_actions)
    dQ[batch_idx, actions] = dL_dq_pred                    # 선택된 행동 위치에만 gradient 반영

    # 2층(출력층)에 대한 gradient 계산
    # q_values = h @ W2 + b2
    dW2 = h.T @ dQ                                         # (hidden_dim, n_actions)
    db2 = np.sum(dQ, axis=0, keepdims=True)               # (1, n_actions)

    # 은닉층으로 gradient 전파
    dh = dQ @ W2.T                                         # (B, hidden_dim)

    # 1층 z1에 대한 gradient (ReLU 도함수 곱)
    dz1 = dh * relu_deriv(z1)                              # (B, hidden_dim)

    # 1층에 대한 gradient 계산
    dW1 = X.T @ dz1                                        # (input_dim, hidden_dim)
    db1 = np.sum(dz1, axis=0, keepdims=True)               # (1, hidden_dim)

    # 파라미터 업데이트 (경사하강법)
    W2 -= learning_rate * dW2
    b2 -= learning_rate * db2
    W1 -= learning_rate * dW1
    b1 -= learning_rate * db1

    # PER의 priority 업데이트: |TD오차| + 작은 epsilon
    new_priorities = np.abs(td_error) + priority_eps
    for idx_buf, pr in zip(indices, new_priorities):
        priorities[idx_buf] = pr

# =====================================================
# 7. 학습 루프 (DQN + PER)
# =====================================================
reward_history = []  # 에피소드별 총 보상을 저장
total_steps    = 0   # 전체 스텝 수

print("=== 1차원 선형 월드에서의 DQN + PER 학습 시작 ===")

for episode in range(1, n_episodes + 1):
    # 에피소드 시작 시 상태 초기화
    state = reset()
    total_reward = 0.0

    for step_idx in range(max_steps):
        total_steps += 1

        # 1) ε-greedy 정책으로 행동 선택
        action = choose_action(state, epsilon)

        # 2) 환경에 행동 적용
        next_state, reward, done = step(state, action)

        # 3) 리플레이 버퍼에 transition 추가
        add_to_buffer(state, action, reward, next_state, done)

        # 4) 리플레이 버퍼에 충분한 데이터가 쌓이면 PER 기반 학습 수행
        if len(replay_buffer) >= max(batch_size, warmup_steps):
            train_dqn_per(batch_size)

        # 5) 누적 보상 업데이트
        total_reward += reward

        # 6) 상태 이동
        state = next_state

        # 7) 목표 상태 도달 시 에피소드 종료
        if done:
            break

    # 에피소드별 보상 기록
    reward_history.append(total_reward)

    # ε 서서히 감소 (원하면 사용 / 여기서는 거의 고정)
    epsilon = max(epsilon_min, epsilon * epsilon_decay)

    # 타깃 네트워크 주기적으로 업데이트
    if episode % target_update_ep == 0:
        W1_tgt = W1.copy()
        b1_tgt = b1.copy()
        W2_tgt = W2.copy()
        b2_tgt = b2.copy()

    # 로그 출력 (50 에피소드마다)
    if episode % 50 == 0:
        avg_reward = np.mean(reward_history[-50:])
        print(f"[Episode {episode:4d}] 최근 50 에피소드 평균 리워드 = {avg_reward:.3f}, epsilon = {epsilon:.3f}")

print("\n=== 학습 종료 ===\n")

# =====================================================
# 8. 학습된 근사 Q-테이블 출력
#    - 각 상태 s에 대해 one-hot을 넣고 Q(s,a)를 추정
# =====================================================
print("▶ DQN + PER 근사 Q-테이블 (행: 상태, 열: 행동[←,→])")

for s in range(n_states):
    x = state_to_onehot(s)                     # (1, n_states)
    _, _, q_vals = forward_network(x, use_target=False)
    q_row = q_vals[0]
    print(f"상태 {s}: {q_row}")

# =====================================================
# 9. 학습된 정책( greedy ) 출력
# =====================================================
action_symbols = {0: "←", 1: "→"}

print("\n▶ 학습된 정책(Policy: greedy w.r.t Q)")
policy_str = ""
for s in range(n_states):
    if s == n_states - 1:
        policy_str += " G "
    else:
        x = state_to_onehot(s)
        _, _, q_vals = forward_network(x, use_target=False)
        best_action = int(np.argmax(q_vals, axis=1)[0])
        policy_str += f" {action_symbols[best_action]} "

print("상태 0  1  2  3  4")
print("     " + policy_str)

# =====================================================
# 10. 학습된 정책으로 1회 에피소드 실행
# =====================================================
print("\n▶ 학습된 정책으로 1회 에피소드 실행 예시")

state = reset()
trajectory = [state]

for step_idx in range(max_steps):
    # 탐험 없이 항상 greedy 행동 선택
    x = state_to_onehot(state)
    _, _, q_vals = forward_network(x, use_target=False)
    action = int(np.argmax(q_vals, axis=1)[0])

    next_state, reward, done = step(state, action)
    trajectory.append(next_state)
    state = next_state

    if done:
        break

print("방문한 상태들:", trajectory)
print("스텝 수:", len(trajectory) - 1)
print("마지막 상태가 목표(4)면 학습 성공!")


=== 1차원 선형 월드에서의 DQN + PER 학습 시작 ===
[Episode   50] 최근 50 에피소드 평균 리워드 = 0.965, epsilon = 0.078
[Episode  100] 최근 50 에피소드 평균 리워드 = 0.967, epsilon = 0.061
[Episode  150] 최근 50 에피소드 평균 리워드 = 0.967, epsilon = 0.050
[Episode  200] 최근 50 에피소드 평균 리워드 = 0.970, epsilon = 0.050
[Episode  250] 최근 50 에피소드 평균 리워드 = 0.969, epsilon = 0.050
[Episode  300] 최근 50 에피소드 평균 리워드 = 0.969, epsilon = 0.050
[Episode  350] 최근 50 에피소드 평균 리워드 = 0.968, epsilon = 0.050
[Episode  400] 최근 50 에피소드 평균 리워드 = 0.968, epsilon = 0.050
[Episode  450] 최근 50 에피소드 평균 리워드 = 0.969, epsilon = 0.050
[Episode  500] 최근 50 에피소드 평균 리워드 = 0.968, epsilon = 0.050

=== 학습 종료 ===

▶ DQN + PER 근사 Q-테이블 (행: 상태, 열: 행동[←,→])
상태 0: [0.06371025 0.55240762]
상태 1: [0.04028242 0.55052092]
상태 2: [0.02519569 0.64619044]
상태 3: [0.05680502 0.84578399]
상태 4: [0.04570611 0.53712368]

▶ 학습된 정책(Policy: greedy w.r.t Q)
상태 0  1  2  3  4
      →  →  →  →  G 

▶ 학습된 정책으로 1회 에피소드 실행 예시
방문한 상태들: [0, 1, 2, 3, 4]
스텝 수: 4
마지막 상태가 목표(4)면 학습 성공!


In [14]:

########################################################################################################
## (1-13) HER(Hindsight Experience Replay) : 목표 달성을 학습할 수 있도록 과거 경험을 재사용하는 기법
########################################################################################################

import numpy as np  # 수치 계산용 numpy

# ============================================
# 0. 난수 시드 고정 (항상 같은 결과가 나오도록)
# ============================================
np.random.seed(42)

# ============================================
# 1. 환경 정의 (1차원 선형 월드)
#    - 실제 물리 상태 s: 0,1,2,3,4  (자리 위치)
#    - 목표 상태 g: 0~4 (HER에서는 목표도 상태공간 위에 놓음)
#    - 여기서는 "실제 환경 목표"는 항상 4로 고정
# ============================================
n_states = 5          # 상태 개수: 0~4
n_actions = 2         # 행동 개수: 0(←), 1(→)
env_real_goal = 4     # 환경에서 실제 목표 상태 (고정)

def env_step(state, action):
    # ------------------------------------------------
    # 환경 dynamics (목표와 무관한 순수 위치 이동)
    #  - action=0: 왼쪽 한 칸,   상태는 최소 0
    #  - action=1: 오른쪽 한 칸, 상태는 최대 4
    # ------------------------------------------------
    if action == 0:
        next_state = max(0, state - 1)
    else:
        next_state = min(n_states - 1, state + 1)

    # ------------------------------------------------
    # 환경의 “실제 목표”에 대한 보상
    #  - HER 학습 시에는 이 보상뿐 아니라
    #    사후적으로 정의한 가짜 목표들에 대해서도
    #    보상을 다시 계산해서 사용함
    # ------------------------------------------------
    if next_state == env_real_goal:
        reward = 1.0      # 목표 도달
        done = True
    else:
        reward = 0.0      # 실패(스파스 보상)
        done = False

    return next_state, reward, done

def reset():
    # 에피소드 시작 시 항상 상태 0에서 시작
    return 0

# ============================================
# 2. HER + Tabular Q-Learning 설정
#    - Q(s, g, a): (상태 s, 목표 g, 행동 a)에 대한 가치
#    - s, g ∈ {0,1,2,3,4}, a ∈ {0,1}
#    - 테이블 크기: [5(상태) x 5(목표) x 2(행동)]
# ============================================
Q = np.zeros((n_states, n_states, n_actions), dtype=np.float32)

alpha = 0.1     # 학습률
gamma = 0.9     # 할인율
epsilon = 1.0   # 초기 ε-greedy 탐험 비율
epsilon_min = 0.05
epsilon_decay = 0.995

n_episodes = 500  # 에피소드 수
max_steps  = 20   # 에피소드당 최대 스텝 수

# ============================================
# 3. ε-greedy 정책 (goal-conditioned)
#    - 현재 상태 s와 목표 g에 대해 Q(s,g,·)를 보고 행동 선택
# ============================================
def choose_action(state, goal, epsilon):
    # ------------------------------------------------
    # 확률 epsilon으로 무작위 행동 (탐험)
    # ------------------------------------------------
    if np.random.rand() < epsilon:
        return np.random.randint(n_actions)

    # ------------------------------------------------
    # 그 외에는 Q(s,g,a)가 최대인 행동 선택 (이용)
    # ------------------------------------------------
    q_vals = Q[state, goal]          # shape: (n_actions,)
    best_action = int(np.argmax(q_vals))
    return best_action

# ============================================
# 4. HER 업데이트 함수
#    - 한 에피소드에서 수집된 trajectory를 이용해
#      (1) 실제 목표(4)에 대한 Q 업데이트
#      (2) 사후 목표(trajectory 중 나중에 도달한 상태)를
#          인공적인 목표로 삼아 Q를 추가 업데이트
# ============================================
def her_update(episode_transitions):
    # episode_transitions: 리스트
    #  각 원소: (s, a, s_next)
    #  (실제 환경에서는 s_next == env_real_goal 이면 done=True로 종료)

    # 에피소드 길이
    T = len(episode_transitions)

    # ------------------------------------------------
    # 1) 실제 목표(env_real_goal=4)에 대한 Q 업데이트
    # ------------------------------------------------
    for t in range(T):
        s, a, s_next = episode_transitions[t]

        # 실제 목표 g_real에 대한 보상 (스파스 보상)
        if s_next == env_real_goal:
            r_real = 1.0
            done_real = True
        else:
            r_real = 0.0
            done_real = False

        # TD target 계산 (실제 목표)
        if done_real:
            target_real = r_real
        else:
            target_real = r_real + gamma * np.max(Q[s_next, env_real_goal])

        # TD 오차 및 Q 업데이트
        td_error = target_real - Q[s, env_real_goal, a]
        Q[s, env_real_goal, a] += alpha * td_error

    # ------------------------------------------------
    # 2) HER (Hindsight Experience Replay)
    #    - 각 시점 t에 대해, “나중에 실제로 도달한 상태들”을
    #      목표 g_her로 삼아 다시 Q를 업데이트
    #    - 여기서는 설명을 위해 가장 단순하게
    #      “모든 미래 시점 t' >= t”를 사용
    # ------------------------------------------------
    for t in range(T):
        s, a, s_next = episode_transitions[t]

        # t 이후에 도달한 모든 s'들을 목표로 사용
        for future_t in range(t, T):
            # HER에서 사용하는 인공 목표 (achieved goal)
            _, _, g_her = episode_transitions[future_t]  # 그 시점의 next_state를 목표로 사용

            # HER용 보상: "그 목표에 도달했는가?"
            if s_next == g_her:
                r_her = 1.0
                done_her = True
            else:
                r_her = 0.0
                done_her = False

            # TD target (HER 목표 g_her에 대한)
            if done_her:
                target_her = r_her
            else:
                target_her = r_her + gamma * np.max(Q[s_next, g_her])

            td_error_her = target_her - Q[s, g_her, a]
            Q[s, g_her, a] += alpha * td_error_her

# ============================================
# 5. HER + Tabular Q-Learning 학습 루프
# ============================================
reward_history = []

print("=== 1차원 선형 월드에서의 HER (Hindsight Experience Replay) + Tabular Q-Learning 학습 시작 ===")

for episode in range(1, n_episodes + 1):
    # --------------------------------------------
    # 에피소드 시작: 상태 초기화
    # --------------------------------------------
    state = reset()
    total_reward = 0.0

    # 이 에피소드에서의 (s, a, s_next) trajectory를 저장
    episode_transitions = []

    for step_idx in range(max_steps):
        # ----------------------------------------
        # 1) 현재 "실제 목표" env_real_goal(=4)에 대해 행동 선택
        #    (goal-conditioned Q(s,g,a) 중 g=4)
        # ----------------------------------------
        action = choose_action(state, env_real_goal, epsilon)

        # ----------------------------------------
        # 2) 환경에 행동 적용
        # ----------------------------------------
        next_state, reward, done = env_step(state, action)

        # trajectory에 저장 (HER에서 목표 재구성에 활용)
        episode_transitions.append((state, action, next_state))

        # 실제 목표에 대한 보상으로 누적 보상 계산
        total_reward += reward

        # 다음 상태로 이동
        state = next_state

        # 목표 도달 시 에피소드 종료
        if done:
            break

    # --------------------------------------------
    # 3) 한 에피소드가 끝난 뒤 HER 업데이트 수행
    #    - 실제 목표 + HER 목표들에 대해 Q를 모두 학습
    # --------------------------------------------
    her_update(episode_transitions)

    # --------------------------------------------
    # 4) ε 감소 및 로그 기록
    # --------------------------------------------
    reward_history.append(total_reward)
    epsilon = max(epsilon_min, epsilon * epsilon_decay)

    if episode % 50 == 0:
        avg_reward = np.mean(reward_history[-50:])
        print(f"[Episode {episode:4d}] 최근 50 에피소드 평균 리워드 = {avg_reward:.3f}, epsilon = {epsilon:.3f}")

print("\n=== 학습 종료 ===\n")

# ============================================
# 6. 목표 g=4에 대한 Q(s,g=4,a)만 따로 출력
#    - 이제까지 다른 알고리즘과 비교하기 위해
#      g=4일 때의 Q 테이블을 꺼내서 보여줌
# ============================================
print("▶ HER + Q-Learning 근사 Q-테이블 (행: 상태 s, 열: 행동[←,→], 목표 g=4 기준)")
for s in range(n_states):
    print(f"상태 {s}, 목표 4: {Q[s, env_real_goal]}")

# ============================================
# 7. 목표 g=4에 대한 greedy 정책 출력
# ============================================
action_symbols = {0: "←", 1: "→"}

print("\n▶ 학습된 정책(Policy: greedy w.r.t Q(s,g=4,a))")
policy_str = ""
for s in range(n_states):
    if s == env_real_goal:
        policy_str += " G "
    else:
        best_action = int(np.argmax(Q[s, env_real_goal]))
        policy_str += f" {action_symbols[best_action]} "

print("상태 0  1  2  3  4")
print("     " + policy_str)

# ============================================
# 8. 학습된 정책으로 1회 에피소드 실행 예시 (목표 g=4)
# ============================================
print("\n▶ 학습된 정책으로 1회 에피소드 실행 예시 (목표 g=4)")

state = reset()
trajectory = [state]

for step_idx in range(max_steps):
    best_action = int(np.argmax(Q[state, env_real_goal]))
    next_state, reward, done = env_step(state, best_action)
    trajectory.append(next_state)
    state = next_state
    if done:
        break

print("방문한 상태들:", trajectory)
print("스텝 수:", len(trajectory) - 1)
print("마지막 상태가 목표(4)면 학습 성공!")


=== 1차원 선형 월드에서의 HER (Hindsight Experience Replay) + Tabular Q-Learning 학습 시작 ===
[Episode   50] 최근 50 에피소드 평균 리워드 = 0.720, epsilon = 0.778
[Episode  100] 최근 50 에피소드 평균 리워드 = 0.940, epsilon = 0.606
[Episode  150] 최근 50 에피소드 평균 리워드 = 0.980, epsilon = 0.471
[Episode  200] 최근 50 에피소드 평균 리워드 = 1.000, epsilon = 0.367
[Episode  250] 최근 50 에피소드 평균 리워드 = 1.000, epsilon = 0.286
[Episode  300] 최근 50 에피소드 평균 리워드 = 1.000, epsilon = 0.222
[Episode  350] 최근 50 에피소드 평균 리워드 = 1.000, epsilon = 0.173
[Episode  400] 최근 50 에피소드 평균 리워드 = 1.000, epsilon = 0.135
[Episode  450] 최근 50 에피소드 평균 리워드 = 1.000, epsilon = 0.105
[Episode  500] 최근 50 에피소드 평균 리워드 = 1.000, epsilon = 0.082

=== 학습 종료 ===

▶ HER + Q-Learning 근사 Q-테이블 (행: 상태 s, 열: 행동[←,→], 목표 g=4 기준)
상태 0, 목표 4: [0.65609884 0.728999  ]
상태 1, 목표 4: [0.65609884 0.8099992 ]
상태 2, 목표 4: [0.728999  0.8999995]
상태 3, 목표 4: [0.8099992  0.99999976]
상태 4, 목표 4: [0. 0.]

▶ 학습된 정책(Policy: greedy w.r.t Q(s,g=4,a))
상태 0  1  2  3  4
      →  →  →  →  G 

▶ 학습된 정책으로 1회 에피소

In [15]:
########################################################################################################
## (1-14) NoisyNet : 신경망 가중치에 노이즈를 추가해 탐색 효율성을 높이는 방식
########################################################################################################
import numpy as np  # 수치 계산용 numpy

# ============================================
# 0. 난수 시드 고정 (실행마다 같은 결과를 얻기 위해)
# ============================================
np.random.seed(42)

# ============================================
# 1. 환경 정의 (1차원 선형 월드)
#    - 상태: 0,1,2,3,4  (5개)
#    - 행동: 0=왼쪽(←), 1=오른쪽(→)
#    - 목표 상태: 4에 도달하면 보상 +1, 에피소드 종료
#    - 그 외에는 보상 -0.01 (빨리 도달하도록 유도)
# ============================================
n_states  = 5                 # 상태 개수
n_actions = 2                 # 행동 개수 (←, →)
goal_state = 4                # 목표 상태

def step(state, action):
    # 환경 dynamics:
    #  - action=0: 왼쪽으로 한 칸 이동 (최소 0)
    #  - action=1: 오른쪽으로 한 칸 이동 (최대 4)
    if action == 0:
        next_state = max(0, state - 1)
    else:
        next_state = min(n_states - 1, state + 1)

    # 보상 설계:
    #  - 목표 상태(4)에 도달하면 +1, 에피소드 종료
    #  - 그 외에는 작은 패널티(-0.01)
    if next_state == goal_state:
        reward = 1.0
        done   = True
    else:
        reward = -0.01
        done   = False

    return next_state, reward, done

def reset():
    # 항상 상태 0에서 에피소드 시작
    return 0

def state_to_onehot(state):
    # 정수 상태를 one-hot 벡터로 변환
    # 예: 상태 2 → [0, 0, 1, 0, 0]
    x = np.zeros((1, n_states), dtype=np.float32)
    x[0, state] = 1.0
    return x

# ============================================
# 2. NoisyNet DQN 신경망 구조 정의 (NumPy로 구현)
#    - 입력: 상태(one-hot, 5차원)
#    - 은닉층: 32 노드, Noisy Linear + ReLU
#    - 출력층: 2 노드, Noisy Linear (각 행동의 Q값)
#    - Noisy Linear:
#        W = W_mu + W_sigma ⊙ eps_W
#        b = b_mu + b_sigma ⊙ eps_b
# ============================================
input_dim  = n_states   # 5
hidden_dim = 32         # 은닉 노드 수
output_dim = n_actions  # 2

# 2-1. 온라인 네트워크 파라미터 (학습 대상)
#   - 각 층마다 (mu, sigma) 쌍으로 구성
# 1층 가중치/편향 (입력 → 은닉)
W1_mu    = 0.1 * np.random.randn(input_dim, hidden_dim)
W1_sigma = 0.017 * np.ones((input_dim, hidden_dim), dtype=np.float32)  # 초기 sigma는 작은 값으로
b1_mu    = np.zeros((1, hidden_dim), dtype=np.float32)
b1_sigma = 0.017 * np.ones((1, hidden_dim), dtype=np.float32)

# 2층 가중치/편향 (은닉 → 출력)
W2_mu    = 0.1 * np.random.randn(hidden_dim, output_dim)
W2_sigma = 0.017 * np.ones((hidden_dim, output_dim), dtype=np.float32)
b2_mu    = np.zeros((1, output_dim), dtype=np.float32)
b2_sigma = 0.017 * np.ones((1, output_dim), dtype=np.float32)

# 2-2. 타깃 네트워크 파라미터 (mu만 복사해서 사용, sigma는 사용하지 않음)
W1_tgt_mu = W1_mu.copy()
b1_tgt_mu = b1_mu.copy()
W2_tgt_mu = W2_mu.copy()
b2_tgt_mu = b2_mu.copy()

# 2-3. NoisyNet에서 사용한 노이즈를 저장할 변수 (역전파 시 필요)
last_eps_W1 = None
last_eps_b1 = None
last_eps_W2 = None
last_eps_b2 = None

def relu(x):
    # ReLU 활성화 함수: 음수는 0, 양수는 그대로
    return np.maximum(0, x)

def relu_deriv(x):
    # ReLU 도함수: x>0이면 1, 아니면 0
    return (x > 0).astype(np.float32)

def forward_noisy(x):
    # ---------------------------------------------
    # NoisyNet 순전파 (온라인 네트워크)
    #  - 각 층에서 새로운 노이즈를 샘플링하여
    #    가중치/편향에 더해줌
    # ---------------------------------------------
    global last_eps_W1, last_eps_b1, last_eps_W2, last_eps_b2

    # 1층 노이즈 샘플링 (정규분포 N(0,1))
    eps_W1 = np.random.randn(*W1_mu.shape).astype(np.float32)
    eps_b1 = np.random.randn(*b1_mu.shape).astype(np.float32)

    # 1층 실제 가중치/편향 계산
    W1 = W1_mu + W1_sigma * eps_W1
    b1 = b1_mu + b1_sigma * eps_b1

    # 1층 순전파
    z1 = x @ W1 + b1
    h  = relu(z1)

    # 2층 노이즈 샘플링
    eps_W2 = np.random.randn(*W2_mu.shape).astype(np.float32)
    eps_b2 = np.random.randn(*b2_mu.shape).astype(np.float32)

    # 2층 실제 가중치/편향 계산
    W2 = W2_mu + W2_sigma * eps_W2
    b2 = b2_mu + b2_sigma * eps_b2

    # 2층 순전파 (Q값)
    q = h @ W2 + b2

    # 사용한 노이즈를 전역 변수에 저장 (역전파에서 사용)
    last_eps_W1 = eps_W1
    last_eps_b1 = eps_b1
    last_eps_W2 = eps_W2
    last_eps_b2 = eps_b2

    return z1, h, q

def forward_target(x):
    # ---------------------------------------------
    # 타깃 네트워크 순전파
    #  - 여기서는 단순히 mu 파라미터만 사용 (deterministic)
    # ---------------------------------------------
    z1 = x @ W1_tgt_mu + b1_tgt_mu
    h  = relu(z1)
    q  = h @ W2_tgt_mu + b2_tgt_mu
    return z1, h, q

# ============================================
# 3. 리플레이 버퍼 (일반 DQN과 동일, PER 아님)
# ============================================
buffer_capacity = 1000      # 버퍼 최대 크기
replay_buffer   = []        # (state, action, reward, next_state, done)

def add_to_buffer(state, action, reward, next_state, done):
    # 버퍼가 찼으면 가장 오래된 transition 제거
    if len(replay_buffer) >= buffer_capacity:
        replay_buffer.pop(0)
    # 새로운 transition 추가
    replay_buffer.append((state, action, reward, next_state, done))

def sample_from_buffer(batch_size):
    # 랜덤하게 mini-batch 샘플링
    idx = np.random.choice(len(replay_buffer), size=batch_size, replace=False)
    batch = [replay_buffer[i] for i in idx]
    return batch

# ============================================
# 4. 하이퍼파라미터 설정
# ============================================
gamma         = 0.9        # 할인율
learning_rate = 0.001      # 학습률

n_episodes = 500           # 에피소드 수
max_steps  = 20            # 에피소드당 최대 스텝 수

batch_size       = 32      # mini-batch 크기
warmup_steps     = 100     # 학습 시작 전 최소 transition 수
target_update_ep = 20      # 타깃 네트워크 업데이트 주기(에피소드 단위)

# NoisyNet에서는 보통 ε-greedy를 쓰지 않지만,
# 여기서는 완전한 NoisyNet 스타일로 ε = 0으로 두고 사용
epsilon = 0.0              # ε-greedy 사용 안 함

# ============================================
# 5. 행동 선택 (NoisyNet 기반, ε-greedy 없음)
# ============================================
def choose_action(state):
    # 상태를 one-hot 벡터로 변환
    x = state_to_onehot(state)       # (1, n_states)
    # NoisyNet 순전파로 Q값 계산 (노이즈 포함)
    _, _, q_values = forward_noisy(x)
    # Q값이 최대인 행동 선택
    action = int(np.argmax(q_values, axis=1)[0])
    return action

# ============================================
# 6. NoisyNet DQN 학습 함수
# ============================================
def train_dqn_noisynet(batch_size):
    global W1_mu, W1_sigma, b1_mu, b1_sigma
    global W2_mu, W2_sigma, b2_mu, b2_sigma

    # 리플레이 버퍼에서 mini-batch 샘플링
    batch = sample_from_buffer(batch_size)

    # 배치를 각각 분리
    states      = np.array([s  for (s, a, r, ns, d) in batch], dtype=np.int32)
    actions     = np.array([a  for (s, a, r, ns, d) in batch], dtype=np.int32)
    rewards     = np.array([r  for (s, a, r, ns, d) in batch], dtype=np.float32)
    next_states = np.array([ns for (s, a, r, ns, d) in batch], dtype=np.int32)
    dones       = np.array([d  for (s, a, r, ns, d) in batch], dtype=bool)

    # 상태/다음 상태를 one-hot으로 변환
    X      = np.vstack([state_to_onehot(s)  for s in states])      # (B, n_states)
    X_next = np.vstack([state_to_onehot(ns) for ns in next_states])# (B, n_states)
    B = X.shape[0]

    batch_idx = np.arange(B)

    # ------------------------------
    # (1) 온라인 NoisyNet 순전파 (Q(s,a))
    # ------------------------------
    z1, h, q_values = forward_noisy(X)      # q_values: (B, n_actions)
    q_pred = q_values[batch_idx, actions]   # 선택한 행동에 대한 Q(s,a)만 뽑기

    # ------------------------------
    # (2) 타깃 네트워크 순전파 (Q_target(s',a'))
    # ------------------------------
    _, _, q_next = forward_target(X_next)   # (B, n_actions)
    q_next_max = np.max(q_next, axis=1)     # max_a' Q_target(s',a')

    not_dones = (~dones).astype(np.float32)

    # TD target 계산: r + γ * (1-done) * max_a' Q_target(s',a')
    td_target = rewards + gamma * not_dones * q_next_max

    # TD 오차 (δ)
    td_error = td_target - q_pred          # (B,)

    # MSE 손실의 gradient: L = mean(δ^2)
    # dL/dq_pred = -2 * δ / B
    dL_dq_pred = -2.0 * td_error / B       # (B,)

    # Q값 전체에 대한 gradient (선택한 행동 위치에만 존재)
    dQ = np.zeros_like(q_values)           # (B, n_actions)
    dQ[batch_idx, actions] = dL_dq_pred

    # ------------------------------
    # (3) 2층(출력층) 역전파 (Noisy Linear)
    # ------------------------------
    # q_values = h @ W2 + b2  (W2, b2는 noisy된 것)
    # 하지만 우리는 mu, sigma에 대한 gradient가 필요함
    # dL/dW2_eff = h^T @ dQ  (W2_eff = W2_mu + W2_sigma ⊙ eps_W2)
    dW2_eff = h.T @ dQ                         # (hidden_dim, output_dim)
    db2_eff = np.sum(dQ, axis=0, keepdims=True)

    # eps 노이즈는 forward_noisy에서 저장된 것을 사용
    eps_W2 = last_eps_W2
    eps_b2 = last_eps_b2

    # mu와 sigma에 대한 gradient
    dW2_mu    = dW2_eff
    dW2_sigma = dW2_eff * eps_W2
    b2_mu_grad    = db2_eff
    b2_sigma_grad = db2_eff * eps_b2

    # 은닉층으로 gradient 전파
    dh = dQ @ (W2_mu + W2_sigma * eps_W2).T    # (B, hidden_dim)

    # ------------------------------
    # (4) 1층 역전파 (Noisy Linear)
    # ------------------------------
    dz1 = dh * relu_deriv(z1)                  # (B, hidden_dim)
    dW1_eff = X.T @ dz1                        # (input_dim, hidden_dim)
    db1_eff = np.sum(dz1, axis=0, keepdims=True)

    eps_W1 = last_eps_W1
    eps_b1 = last_eps_b1

    dW1_mu    = dW1_eff
    dW1_sigma = dW1_eff * eps_W1
    b1_mu_grad    = db1_eff
    b1_sigma_grad = db1_eff * eps_b1

    # ------------------------------
    # (5) 파라미터 업데이트 (경사하강법)
    # ------------------------------
    W2_mu    -= learning_rate * dW2_mu
    W2_sigma -= learning_rate * dW2_sigma
    b2_mu    -= learning_rate * b2_mu_grad
    b2_sigma -= learning_rate * b2_sigma_grad

    W1_mu    -= learning_rate * dW1_mu
    W1_sigma -= learning_rate * dW1_sigma
    b1_mu    -= learning_rate * b1_mu_grad
    b1_sigma -= learning_rate * b1_sigma_grad

# ============================================
# 7. 학습 루프 (NoisyNet DQN)
# ============================================
reward_history = []
total_steps    = 0

print("=== 1차원 선형 월드에서의 NoisyNet DQN(NumPy) 학습 시작 ===")

for episode in range(1, n_episodes + 1):
    state = reset()
    total_reward = 0.0

    for step_idx in range(max_steps):
        total_steps += 1

        # 1) NoisyNet 기반 행동 선택 (ε-greedy 없이)
        action = choose_action(state)

        # 2) 환경 한 스텝 진행
        next_state, reward, done = step(state, action)

        # 3) 리플레이 버퍼에 transition 추가
        add_to_buffer(state, action, reward, next_state, done)

        # 4) 일정량 이상 쌓이면 학습 진행
        if len(replay_buffer) >= max(batch_size, warmup_steps):
            train_dqn_noisynet(batch_size)

        # 5) 누적 보상 업데이트
        total_reward += reward

        # 6) 상태 업데이트
        state = next_state

        # 7) 목표 도달 시 에피소드 종료
        if done:
            break

    # 에피소드별 보상 기록
    reward_history.append(total_reward)

    # 타깃 네트워크 주기적 동기화 (mu 파라미터 복사)
    if episode % target_update_ep == 0:
        W1_tgt_mu = W1_mu.copy()
        b1_tgt_mu = b1_mu.copy()
        W2_tgt_mu = W2_mu.copy()
        b2_tgt_mu = b2_mu.copy()

    # 50 에피소드마다 로그 출력
    if episode % 50 == 0:
        avg_reward = np.mean(reward_history[-50:])
        print(f"[Episode {episode:4d}] 최근 50 에피소드 평균 리워드 = {avg_reward:.3f}")

print("\n=== 학습 종료 ===\n")

# ============================================
# 8. 학습된 NoisyNet Q값 (mu 기준) 출력
#    - 비교를 위해, 노이즈 없이 mu 파라미터만 사용
# ============================================
print("▶ NoisyNet DQN 근사 Q-테이블 (행: 상태, 열: 행동[←,→])")

for s in range(n_states):
    x = state_to_onehot(s)
    # 타깃 네트워크 대신, 온라인 mu 파라미터를 사용해 deterministic Q를 계산
    z1 = x @ W1_mu + b1_mu
    h  = relu(z1)
    q_vals = h @ W2_mu + b2_mu
    q_row = q_vals[0]
    print(f"상태 {s}: {q_row}")

# ============================================
# 9. 학습된 정책( greedy ) 출력
# ============================================
action_symbols = {0: "←", 1: "→"}

print("\n▶ 학습된 정책(Policy: greedy w.r.t Q(mu))")
policy_str = ""
for s in range(n_states):
    if s == goal_state:
        policy_str += " G "
    else:
        x = state_to_onehot(s)
        z1 = x @ W1_mu + b1_mu
        h  = relu(z1)
        q_vals = h @ W2_mu + b2_mu
        best_action = int(np.argmax(q_vals, axis=1)[0])
        policy_str += f" {action_symbols[best_action]} "

print("상태 0  1  2  3  4")
print("     " + policy_str)

# ============================================
# 10. 학습된 정책으로 1회 에피소드 실행 예시
# ============================================
print("\n▶ 학습된 정책으로 1회 에피소드 실행 예시")

state = reset()
trajectory = [state]

for step_idx in range(max_steps):
    x = state_to_onehot(state)
    z1 = x @ W1_mu + b1_mu
    h  = relu(z1)
    q_vals = h @ W2_mu + b2_mu
    action = int(np.argmax(q_vals, axis=1)[0])

    next_state, reward, done = step(state, action)
    trajectory.append(next_state)
    state = next_state

    if done:
        break

print("방문한 상태들:", trajectory)
print("스텝 수:", len(trajectory) - 1)
print("마지막 상태가 목표(4)면 학습 성공!")


=== 1차원 선형 월드에서의 NoisyNet DQN(NumPy) 학습 시작 ===
[Episode   50] 최근 50 에피소드 평균 리워드 = 0.962
[Episode  100] 최근 50 에피소드 평균 리워드 = 0.970
[Episode  150] 최근 50 에피소드 평균 리워드 = 0.970
[Episode  200] 최근 50 에피소드 평균 리워드 = 0.970
[Episode  250] 최근 50 에피소드 평균 리워드 = 0.970
[Episode  300] 최근 50 에피소드 평균 리워드 = 0.970
[Episode  350] 최근 50 에피소드 평균 리워드 = 0.970
[Episode  400] 최근 50 에피소드 평균 리워드 = 0.970
[Episode  450] 최근 50 에피소드 평균 리워드 = 0.970
[Episode  500] 최근 50 에피소드 평균 리워드 = 0.970

=== 학습 종료 ===

▶ NoisyNet DQN 근사 Q-테이블 (행: 상태, 열: 행동[←,→])
상태 0: [0.04956277 0.55301418]
상태 1: [0.02100446 0.56870416]
상태 2: [0.0087512  0.67418015]
상태 3: [0.03827106 0.87848686]
상태 4: [0.03131548 0.55306848]

▶ 학습된 정책(Policy: greedy w.r.t Q(mu))
상태 0  1  2  3  4
      →  →  →  →  G 

▶ 학습된 정책으로 1회 에피소드 실행 예시
방문한 상태들: [0, 1, 2, 3, 4]
스텝 수: 4
마지막 상태가 목표(4)면 학습 성공!


# [2] Model-free RL : Policy Iteration

	기본 Policy Gradient 및 Actor-Critic 계열
		(2-1) REINFORCE: 정책 경사법의 기본 형태
		(2-2) Actor-Critic: 기본적인 Actor-Critic 구조, Policy Gradient와 Critic의 Q값 평가 결합
		(2-3) NAC(Natural Actor-Critic): Natural Gradient를 적용해 정책의 효율적 업데이트를 수행
	Advantage Actor-Critic 및 분산 학습 계열
		(2-4) A2C/A3C(Advantage Actor-Critic): 분산형 Actor-Critic 모델
		(2-5) ACER(Actor-Critic with Experience Replay): 경험 리플레이를 추가한 Actor-Critic 방법
		(2-6) IMPALA(Importance Weighted Actor-Learner Architecture): 분산 학습에 최적화된 구조
		(2-7) Off-PAC(Off-Policy Actor-Critic): 오프폴리시 데이터를 활용하는 Actor-Critic 기법
	신뢰 구간 기반 정책 최적화 계열
		(2-8) PPO(Proximal Policy Optimization): 신뢰 구간을 사용해 안정적으로 정책을 업데이트
		(2-9) TRPO(Trust Region Policy Optimization): 정책 급변을 방지하는 최적화 기법
	연속 행동 공간 최적화 계열
		(2-10) DDPG(Deep Deterministic Policy Gradient): 연속적 행동 공간에서 학습하는 Actor-Critic 모델
		(2-11) TD3(Twin Delayed DDPG): DDPG의 한계점을 극복하기 위한 개선된 모델
		(2-12) SAC(Soft Actor-Critic): 탐색과 활용의 균형을 유지하도록 설계된 정책 학습 모델
	전문가 시범 데이터 활용 계열
		(2-13) BC(Behavioral Cloning): 데이터를 기반으로 정책을 모방하는 방식
		(2-14) DDPGfD(DDPG from Demonstrations): 전문가의 시범을 사용해 DDPG 성능을 개선

In [16]:
###################################################################
## (2-1) REINFORCE : 정책 경사법의 기본 형태
###################################################################
import numpy as np  # 수치 계산을 위한 NumPy 불러오기

# ==============================
# 1. 환경 정의 (1차원 선형 월드)
# ==============================
n_states = 5        # 상태 개수: 0,1,2,3,4 (4가 목표 상태)
n_actions = 2       # 행동 개수: 0=왼쪽, 1=오른쪽

def step(state, action):
    # 주어진 상태 state에서 행동 action을 했을 때
    # 다음 상태(next_state), 보상(reward), 종료 여부(done)를 반환하는 함수

    # 행동이 0이면 왼쪽으로 한 칸 이동, 1이면 오른쪽으로 한 칸 이동
    if action == 0:  # 왼쪽 이동
        next_state = max(0, state - 1)     # 상태 0보다 왼쪽으로는 가지 않도록 최소값 제한
    else:            # 오른쪽 이동
        next_state = min(n_states - 1, state + 1)  # 상태 4보다 오른쪽으로는 가지 않도록 최대값 제한

    # 보상과 종료 조건 설정
    if next_state == n_states - 1:         # 목표 상태(4)에 도달한 경우
        reward = 1.0                       # 성공 보상 1.0 부여
        done = True                        # 에피소드 종료
    else:
        reward = -0.01                     # 그 외 상태에서는 시간 패널티 -0.01 부여 (빨리 도달 유도)
        done = False                       # 에피소드 계속 진행

    return next_state, reward, done        # 결과 반환

def reset():
    # 에피소드 시작 시 초기 상태를 반환하는 함수
    # 여기서는 항상 상태 0에서 시작
    return 0


# ==============================
# 2. REINFORCE 하이퍼파라미터
# ==============================
np.random.seed(42)   # 난수 시드 고정 (실행결과 재현 가능)

gamma = 0.99         # 할인율 (미래 보상을 얼마나 반영할지 결정)
alpha = 0.05         # 학습률 (정책 파라미터 업데이트 크기)
n_episodes = 500     # 전체 학습 에피소드 수
max_steps  = 20      # 한 에피소드에서 허용하는 최대 스텝 수 (무한 루프 방지)

# 정책 파라미터 theta: (상태 x 행동) 크기의 실수 행렬
# 각 상태에서 두 행동(왼쪽, 오른쪽)에 대한 "선호도(preference)"를 나타냄
theta = np.zeros((n_states, n_actions))    # 처음에는 모든 선호도를 0으로 시작


# ==============================
# 3. 정책 관련 함수 정의
# ==============================
def softmax(logits):
    # 주어진 선호도 벡터 logits를 softmax를 이용해 확률 분포로 변환하는 함수
    c = np.max(logits)                     # 수치 안정성을 위해 최대값을 빼줌
    exp = np.exp(logits - c)               # 지수 함수 적용
    return exp / np.sum(exp)               # 전체 합으로 나누어 확률 분포로 변환

def policy_probs(state):
    # 주어진 상태 state에서의 행동 확률 π(a|s; θ)를 반환하는 함수
    # theta[state]는 해당 상태에서 각 행동에 대한 선호도 벡터
    return softmax(theta[state])           # softmax를 적용하여 확률로 변환

def sample_action(state):
    # 현재 정책에 따라 행동을 하나 샘플링하는 함수
    probs = policy_probs(state)            # 행동 확률 π(a|s; θ) 구하기
    return np.random.choice(n_actions, p=probs)  # 해당 확률에 따라 0 또는 1 선택

def grad_log_policy(state, action):
    # ∇θ log π(a|s; θ) 를 계산하는 함수
    # 반환값은 길이 2의 벡터 (각 행동에 대한 gradient)
    probs = policy_probs(state)            # π(a|s; θ) 계산
    grad = -probs.copy()                   # 기본값은 -π(a|s; θ)
    grad[action] += 1.0                    # 선택된 행동 a에 대해서는 +1 더해줌
    # 결과적으로:
    # grad[k] = 1 - π(k|s)  if k == action
    # grad[k] = -π(k|s)     if k != action
    return grad                           # shape: (2,)


# ==============================
# 4. REINFORCE 학습 루프
# ==============================
return_history = []                       # 각 에피소드의 총 Return(G_0)을 기록할 리스트

print("=== 1차원 선형 월드에서의 REINFORCE 학습 시작 ===")

for episode in range(1, n_episodes + 1):
    # 한 에피소드에 대해 (상태, 행동, 보상) 시퀀스를 저장하기 위한 리스트 초기화
    states = []                           # 방문한 상태들을 순서대로 저장
    actions = []                          # 각 시점에서 선택한 행동들을 저장
    rewards = []                          # 각 시점에서 받은 보상을 저장

    # 에피소드 시작: 초기 상태로 리셋
    state = reset()

    # 1) 정책에 따라 에피소드 한 번 실행하여 trajectory 수집
    for t in range(max_steps):
        action = sample_action(state)     # 현재 정책에 따라 행동 샘플링
        next_state, reward, done = step(state, action)  # 환경에 행동 적용

        states.append(state)             # 시점 t의 상태 기록
        actions.append(action)           # 시점 t의 행동 기록
        rewards.append(reward)           # 시점 t의 보상 기록

        state = next_state               # 다음 상태로 전이

        if done:                         # 목표 상태 도달 시 에피소드 종료
            break

    # 2) 에피소드 내 각 시점 t에 대한 Return G_t 계산 (뒤에서부터 누적)
    T = len(rewards)                     # 실제로 진행된 스텝 수
    returns = np.zeros(T)                # 각 시점 t의 G_t를 저장할 배열
    G = 0.0                              # 뒤에서부터 누적할 Return 값
    for t in reversed(range(T)):         # 마지막 시점에서부터 거꾸로 진행
        G = rewards[t] + gamma * G       # G_t = r_t + γ G_{t+1}
        returns[t] = G                   # 계산된 G_t 저장

    # 3) 정책 파라미터 θ 업데이트 (Monte Carlo Policy Gradient)
    #    θ ← θ + α ∑_t ∇θ log π(a_t|s_t; θ) * G_t
    for t in range(T):
        s = states[t]                    # 시점 t의 상태
        a = actions[t]                   # 시점 t의 행동
        G_t = returns[t]                 # 시점 t의 Return G_t

        grad = grad_log_policy(s, a)     # ∇θ(s,:) log π(a|s; θ) 계산 (shape: (2,))
        theta[s] += alpha * grad * G_t   # 해당 상태 s의 파라미터에 gradient ascent 적용

    # 4) 에피소드의 시작 시점 Return(G_0)을 기록
    episode_return = returns[0]          # G_0: 에피소드 전체 Return
    return_history.append(episode_return)

    # 50 에피소드마다 최근 50개 Return 평균을 출력 (학습 진행 확인 용도)
    if episode % 50 == 0:
        avg_ret = np.mean(return_history[-50:])
        print(f"[Episode {episode:4d}] 최근 50 에피소드 평균 Return = {avg_ret:.3f}")

print("\n=== REINFORCE 학습 종료 ===\n")


# ==============================
# 5. 학습된 정책(상태별 행동 확률) 출력
# ==============================
print("▶ 학습된 정책 π(a|s; θ) (행: 상태, 열: 행동[←,→])")
for s in range(n_states):
    probs = policy_probs(s)              # 상태 s에서의 행동 확률
    print(f"상태 {s}: {probs}")

# Greedy 정책으로 사람이 보기 쉽게 화살표로 표현 (확률이 더 큰 행동 선택)
action_symbols = {0: "←", 1: "→"}        # 0은 왼쪽 화살표, 1은 오른쪽 화살표

print("\n▶ Greedy 기준 학습된 정책(Policy)")
policy_str = ""
for s in range(n_states):
    if s == n_states - 1:                # 목표 상태인 경우
        policy_str += " G "              # Goal 표기
    else:
        best_a = np.argmax(policy_probs(s))  # 가장 확률이 높은 행동 선택
        policy_str += f" {action_symbols[best_a]} "
print("상태 0  1  2  3  4")
print("     " + policy_str)


# ==============================
# 6. 학습된 정책으로 1회 테스트 실행
# ==============================
print("\n▶ 학습된 정책으로 1회 에피소드 실행 예시 (Greedy 정책 사용)")

state = reset()                          # 초기 상태로 리셋
trajectory = [state]                     # 방문한 상태들을 기록하기 위한 리스트

for step_idx in range(max_steps):
    probs = policy_probs(state)          # 현재 상태에서의 정책 확률
    action = np.argmax(probs)            # 가장 확률이 높은 행동을 선택 (탐험 없이 greedy)
    next_state, reward, done = step(state, action)  # 환경에 행동 적용

    trajectory.append(next_state)        # 방문한 상태 기록
    state = next_state                   # 다음 상태로 이동

    if done:                             # 목표에 도달하면 종료
        break

print("방문한 상태들:", trajectory)
print("스텝 수:", len(trajectory) - 1)
print("마지막 상태가 목표(4)면 학습 성공!")


=== 1차원 선형 월드에서의 REINFORCE 학습 시작 ===
[Episode   50] 최근 50 에피소드 평균 Return = 0.663
[Episode  100] 최근 50 에피소드 평균 Return = 0.828
[Episode  150] 최근 50 에피소드 평균 Return = 0.867
[Episode  200] 최근 50 에피소드 평균 Return = 0.898
[Episode  250] 최근 50 에피소드 평균 Return = 0.883
[Episode  300] 최근 50 에피소드 평균 Return = 0.905
[Episode  350] 최근 50 에피소드 평균 Return = 0.893
[Episode  400] 최근 50 에피소드 평균 Return = 0.896
[Episode  450] 최근 50 에피소드 평균 Return = 0.900
[Episode  500] 최근 50 에피소드 평균 Return = 0.892

=== REINFORCE 학습 종료 ===

▶ 학습된 정책 π(a|s; θ) (행: 상태, 열: 행동[←,→])
상태 0: [0.10977985 0.89022015]
상태 1: [0.43454976 0.56545024]
상태 2: [0.10441219 0.89558781]
상태 3: [0.20137174 0.79862826]
상태 4: [0.5 0.5]

▶ Greedy 기준 학습된 정책(Policy)
상태 0  1  2  3  4
      →  →  →  →  G 

▶ 학습된 정책으로 1회 에피소드 실행 예시 (Greedy 정책 사용)
방문한 상태들: [0, 1, 2, 3, 4]
스텝 수: 4
마지막 상태가 목표(4)면 학습 성공!


In [17]:
###################################################################
## (2-2) Actor-Critic: 기본적인 Actor-Critic 구조, Policy Gradient와 Critic의 Q값 평가 결합
###################################################################
import numpy as np  # 수치 계산을 위한 NumPy 불러오기

# ==============================
# 1. 환경 정의 (1차원 선형 월드)
# ==============================
n_states = 5        # 상태 개수: 0,1,2,3,4 (4가 목표 상태)
n_actions = 2       # 행동 개수: 0=왼쪽, 1=오른쪽

def step(state, action):
    # 주어진 상태 state에서 행동 action을 했을 때
    # 다음 상태(next_state), 보상(reward), 종료 여부(done)를 반환하는 함수

    # 행동이 0이면 왼쪽으로 한 칸 이동, 1이면 오른쪽으로 한 칸 이동
    if action == 0:  # 왼쪽 이동
        next_state = max(0, state - 1)     # 상태 0보다 왼쪽으로는 가지 않도록 최소값 제한
    else:            # 오른쪽 이동
        next_state = min(n_states - 1, state + 1)  # 상태 4보다 오른쪽으로는 가지 않도록 최대값 제한

    # 보상과 종료 조건 설정
    if next_state == n_states - 1:         # 목표 상태(4)에 도달한 경우
        reward = 1.0                       # 성공 보상 1.0 부여
        done = True                        # 에피소드 종료
    else:
        reward = -0.01                     # 그 외 상태에서는 시간 패널티 -0.01 부여 (빨리 도달 유도)
        done = False                       # 에피소드 계속 진행

    return next_state, reward, done        # 결과 반환

def reset():
    # 에피소드 시작 시 초기 상태를 반환하는 함수
    # 여기서는 항상 상태 0에서 시작
    return 0


# ==============================
# 2. Actor-Critic 하이퍼파라미터
# ==============================
np.random.seed(42)   # 난수 시드 고정 (실행결과 재현 가능)

gamma = 0.99         # 할인율
alpha_theta = 0.05   # Actor(정책 파라미터) 학습률
alpha_v = 0.1        # Critic(가치함수) 학습률
n_episodes = 500     # 전체 학습 에피소드 수
max_steps  = 20      # 한 에피소드에서 허용하는 최대 스텝 수

# 정책 파라미터 theta: (상태 x 행동) 크기의 실수 행렬
# 각 상태에서 두 행동(왼쪽, 오른쪽)에 대한 "선호도(preference)"를 나타냄
theta = np.zeros((n_states, n_actions))    # 처음에는 모든 선호도를 0으로 시작

# 가치함수 파라미터 v: 각 상태의 상태가치 V(s)를 나타내는 벡터
v = np.zeros(n_states)                     # 초기에는 모든 상태가치를 0으로 시작


# ==============================
# 3. 정책 관련 함수 정의
# ==============================
def softmax(logits):
    # 주어진 선호도 벡터 logits를 softmax를 이용해 확률 분포로 변환하는 함수
    c = np.max(logits)                     # 수치 안정성을 위해 최대값을 빼줌
    exp = np.exp(logits - c)               # 지수 함수 적용
    return exp / np.sum(exp)               # 전체 합으로 나누어 확률 분포로 변환

def policy_probs(state):
    # 주어진 상태 state에서의 행동 확률 π(a|s; θ)를 반환하는 함수
    # theta[state]는 해당 상태에서 각 행동에 대한 선호도 벡터
    return softmax(theta[state])           # softmax를 적용하여 확률로 변환

def sample_action(state):
    # 현재 정책에 따라 행동을 하나 샘플링하는 함수
    probs = policy_probs(state)            # 행동 확률 π(a|s; θ) 구하기
    return np.random.choice(n_actions, p=probs)  # 해당 확률에 따라 0 또는 1 선택

def grad_log_policy(state, action):
    # ∇θ log π(a|s; θ) 를 계산하는 함수
    # 반환값은 길이 2의 벡터 (각 행동에 대한 gradient)
    probs = policy_probs(state)            # π(a|s; θ) 계산
    grad = -probs.copy()                   # 기본값은 -π(a|s; θ)
    grad[action] += 1.0                    # 선택된 행동 a에 대해서는 +1 더해줌
    # 결과적으로:
    # grad[k] = 1 - π(k|s)  if k == action
    # grad[k] = -π(k|s)     if k != action
    return grad                           # shape: (2,)


# ==============================
# 4. Actor-Critic 학습 루프
# ==============================
return_history = []                       # 각 에피소드의 총 Return(G_0)을 기록할 리스트

print("=== 1차원 선형 월드에서의 Actor-Critic 학습 시작 ===")

for episode in range(1, n_episodes + 1):
    # 한 에피소드에 대해 총 Return을 계산하기 위한 변수 초기화
    state = reset()                       # 에피소드 시작 상태
    G0 = 0.0                              # 시작 시점의 Return 누적용 (로그용)
    discount = 1.0                        # γ^t 계수 누적용

    for t in range(max_steps):
        # 1) 상태에서 정책에 따라 행동 선택
        action = sample_action(state)     # 현재 Actor(정책)에 따른 행동 샘플링

        # 2) 환경에 행동 적용
        next_state, reward, done = step(state, action)

        # 3) TD 오차(delta) 계산
        #    δ = r + γ V(s') - V(s)
        v_s = v[state]                    # 현재 상태의 가치 V(s)
        v_s_next = v[next_state] if not done else 0.0  # 종료 시에는 V(s') = 0으로 처리
        delta = reward + gamma * v_s_next - v_s

        # 4) Critic 업데이트: V(s) ← V(s) + α_v * δ
        v[state] += alpha_v * delta

        # 5) Actor 업데이트: θ ← θ + α_θ * δ * ∇θ log π(a|s; θ)
        grad = grad_log_policy(state, action)
        theta[state] += alpha_theta * delta * grad

        # 6) 에피소드 Return(G_0) 추적 (로그용)
        #    G_0 = Σ_t γ^t r_t
        G0 += discount * reward
        discount *= gamma

        # 7) 다음 시점으로 전이
        state = next_state

        if done:
            # 목표 상태에 도달하면 에피소드 종료
            break

    # 한 에피소드가 끝나면 Return(G_0)을 기록
    return_history.append(G0)

    # 50 에피소드마다 최근 50개 Return 평균을 출력 (학습 진행 확인 용도)
    if episode % 50 == 0:
        avg_ret = np.mean(return_history[-50:])
        print(f"[Episode {episode:4d}] 최근 50 에피소드 평균 Return = {avg_ret:.3f}")

print("\n=== Actor-Critic 학습 종료 ===\n")


# ==============================
# 5. 학습된 정책(상태별 행동 확률) 출력
# ==============================
print("▶ 학습된 정책 π(a|s; θ) (행: 상태, 열: 행동[←,→])")
for s in range(n_states):
    probs = policy_probs(s)              # 상태 s에서의 행동 확률
    print(f"상태 {s}: {probs}")

# Greedy 정책으로 사람이 보기 쉽게 화살표로 표현 (확률이 더 큰 행동 선택)
action_symbols = {0: "←", 1: "→"}        # 0은 왼쪽 화살표, 1은 오른쪽 화살표

print("\n▶ Greedy 기준 학습된 정책(Policy)")
policy_str = ""
for s in range(n_states):
    if s == n_states - 1:                # 목표 상태인 경우
        policy_str += " G "              # Goal 표기
    else:
        best_a = np.argmax(policy_probs(s))  # 가장 확률이 높은 행동 선택
        policy_str += f" {action_symbols[best_a]} "
print("상태 0  1  2  3  4")
print("     " + policy_str)


# ==============================
# 6. 학습된 정책으로 1회 테스트 실행
# ==============================
print("\n▶ 학습된 정책으로 1회 에피소드 실행 예시 (Greedy 정책 사용)")

state = reset()                          # 초기 상태로 리셋
trajectory = [state]                     # 방문한 상태들을 기록하기 위한 리스트

for step_idx in range(max_steps):
    probs = policy_probs(state)          # 현재 상태에서의 정책 확률
    action = np.argmax(probs)            # 가장 확률이 높은 행동을 선택 (탐험 없이 greedy)
    next_state, reward, done = step(state, action)  # 환경에 행동 적용

    trajectory.append(next_state)        # 방문한 상태 기록
    state = next_state                   # 다음 상태로 이동

    if done:                             # 목표에 도달하면 종료
        break

print("방문한 상태들:", trajectory)
print("스텝 수:", len(trajectory) - 1)
print("마지막 상태가 목표(4)면 학습 성공!")


=== 1차원 선형 월드에서의 Actor-Critic 학습 시작 ===
[Episode   50] 최근 50 에피소드 평균 Return = 0.684
[Episode  100] 최근 50 에피소드 평균 Return = 0.820
[Episode  150] 최근 50 에피소드 평균 Return = 0.862
[Episode  200] 최근 50 에피소드 평균 Return = 0.890
[Episode  250] 최근 50 에피소드 평균 Return = 0.886
[Episode  300] 최근 50 에피소드 평균 Return = 0.895
[Episode  350] 최근 50 에피소드 평균 Return = 0.899
[Episode  400] 최근 50 에피소드 평균 Return = 0.907
[Episode  450] 최근 50 에피소드 평균 Return = 0.910
[Episode  500] 최근 50 에피소드 평균 Return = 0.909

=== Actor-Critic 학습 종료 ===

▶ 학습된 정책 π(a|s; θ) (행: 상태, 열: 행동[←,→])
상태 0: [0.26635851 0.73364149]
상태 1: [0.15665809 0.84334191]
상태 2: [0.13892206 0.86107794]
상태 3: [0.17650638 0.82349362]
상태 4: [0.5 0.5]

▶ Greedy 기준 학습된 정책(Policy)
상태 0  1  2  3  4
      →  →  →  →  G 

▶ 학습된 정책으로 1회 에피소드 실행 예시 (Greedy 정책 사용)
방문한 상태들: [0, 1, 2, 3, 4]
스텝 수: 4
마지막 상태가 목표(4)면 학습 성공!


In [18]:
###################################################################
## (2-3) NAC(Natural Actor-Critic) : Natural Gradient를 적용해 정책의 효율적 업데이트를 수행
###################################################################
import numpy as np  # 수치 계산을 위한 NumPy 불러오기

# ==============================
# 1. 환경 정의 (1차원 선형 월드)
# ==============================
n_states = 5        # 상태 개수: 0,1,2,3,4 (4가 목표 상태)
n_actions = 2       # 행동 개수: 0=왼쪽, 1=오른쪽

def step(state, action):
    # 주어진 상태 state에서 행동 action을 했을 때
    # 다음 상태(next_state), 보상(reward), 종료 여부(done)를 반환하는 함수

    # 행동이 0이면 왼쪽으로 한 칸 이동, 1이면 오른쪽으로 한 칸 이동
    if action == 0:  # 왼쪽 이동
        next_state = max(0, state - 1)     # 상태 0보다 왼쪽으로는 가지 않도록 최소값 제한
    else:            # 오른쪽 이동
        next_state = min(n_states - 1, state + 1)  # 상태 4보다 오른쪽으로는 가지 않도록 최대값 제한

    # 보상과 종료 조건 설정
    if next_state == n_states - 1:         # 목표 상태(4)에 도달한 경우
        reward = 1.0                       # 성공 보상 1.0 부여
        done = True                        # 에피소드 종료
    else:
        reward = -0.01                     # 그 외 상태에서는 시간 패널티 -0.01 부여 (빨리 도달 유도)
        done = False                       # 에피소드 계속 진행

    return next_state, reward, done        # 결과 반환

def reset():
    # 에피소드 시작 시 초기 상태를 반환하는 함수
    # 여기서는 항상 상태 0에서 시작
    return 0


# ==============================
# 2. NAC 하이퍼파라미터
# ==============================
np.random.seed(42)   # 난수 시드 고정 (실행결과 재현 가능)

gamma = 0.99         # 할인율 (미래 보상 반영 정도)
alpha = 0.05         # 학습률 (Natural Gradient 업데이트 크기)
n_episodes = 500     # 전체 학습 에피소드 수
max_steps  = 20      # 한 에피소드에서 허용하는 최대 스텝 수

# 정책 파라미터 theta: (상태 x 행동) 크기의 실수 행렬
# 각 상태에서 두 행동(왼쪽, 오른쪽)에 대한 "선호도(preference)"를 나타냄
theta = np.zeros((n_states, n_actions))    # 처음에는 모든 선호도를 0으로 시작


# ==============================
# 3. 정책 관련 함수 정의
# ==============================
def softmax(logits):
    # 주어진 선호도 벡터 logits를 softmax를 이용해 확률 분포로 변환하는 함수
    c = np.max(logits)                     # 수치 안정성을 위해 최대값을 빼줌
    exp = np.exp(logits - c)               # 지수 함수 적용
    return exp / np.sum(exp)               # 전체 합으로 나누어 확률 분포로 변환

def policy_probs(state):
    # 주어진 상태 state에서의 행동 확률 π(a|s; θ)를 반환하는 함수
    # theta[state]는 해당 상태에서 각 행동에 대한 선호도 벡터
    return softmax(theta[state])           # softmax를 적용하여 확률로 변환

def sample_action(state):
    # 현재 정책에 따라 행동을 하나 샘플링하는 함수
    probs = policy_probs(state)            # 행동 확률 π(a|s; θ) 구하기
    return np.random.choice(n_actions, p=probs)  # 해당 확률에 따라 0 또는 1 선택

def grad_log_policy(state, action):
    # ∇θ log π(a|s; θ) 를 계산하는 함수
    # 반환값은 길이 2의 벡터 (각 행동에 대한 gradient)
    probs = policy_probs(state)            # π(a|s; θ) 계산
    grad = -probs.copy()                   # 기본값은 -π(a|s; θ)
    grad[action] += 1.0                    # 선택된 행동 a에 대해서는 +1 더해줌
    # 결과적으로:
    # grad[k] = 1 - π(k|s)  if k == action
    # grad[k] = -π(k|s)     if k != action
    return grad                           # shape: (2,)


# ==============================
# 4. NAC 학습 루프
# ==============================
return_history = []                       # 각 에피소드의 총 Return(G_0)을 기록할 리스트

print("=== 1차원 선형 월드에서의 NAC (Natural Actor-Critic) 학습 시작 ===")

for episode in range(1, n_episodes + 1):
    # 한 에피소드에 대해 (상태, 행동, 보상) 시퀀스를 저장하기 위한 리스트 초기화
    states = []                           # 방문한 상태들을 순서대로 저장
    actions = []                          # 각 시점에서 선택한 행동들을 저장
    rewards = []                          # 각 시점에서 받은 보상을 저장

    state = reset()                       # 에피소드 시작: 초기 상태로 리셋

    # 1) 정책에 따라 에피소드 한 번 실행하여 trajectory 수집
    for t in range(max_steps):
        action = sample_action(state)     # 현재 정책에 따라 행동 샘플링
        next_state, reward, done = step(state, action)  # 환경에 행동 적용

        states.append(state)             # 시점 t의 상태 기록
        actions.append(action)           # 시점 t의 행동 기록
        rewards.append(reward)           # 시점 t의 보상 기록

        state = next_state               # 다음 상태로 전이

        if done:                         # 목표 상태 도달 시 에피소드 종료
            break

    # 2) 에피소드 내 각 시점 t에 대한 Return G_t 계산 (뒤에서부터 누적)
    T = len(rewards)                     # 실제로 진행된 스텝 수
    returns = np.zeros(T)                # 각 시점 t의 G_t를 저장할 배열
    G = 0.0                              # 뒤에서부터 누적할 Return 값
    for t in reversed(range(T)):         # 마지막 시점에서부터 거꾸로 진행
        G = rewards[t] + gamma * G       # G_t = r_t + γ G_{t+1}
        returns[t] = G                   # 계산된 G_t 저장

    # 3) 상태별 policy gradient / Fisher matrix 누적
    #    각 상태 s에 대해:
    #    g_s = Σ_t ∇θ log π(a_t|s_t=s) * G_t
    #    F_s = Σ_t ∇θ log π(a_t|s_t=s) ∇θ log π(a_t|s_t=s)^T
    g_per_state = np.zeros((n_states, n_actions))            # 각 상태별 gradient 벡터
    F_per_state = np.zeros((n_states, n_actions, n_actions)) # 각 상태별 Fisher(2x2)

    for t in range(T):
        s = states[t]                    # 시점 t의 상태
        a = actions[t]                   # 시점 t의 행동
        G_t = returns[t]                 # 시점 t의 Return G_t

        grad = grad_log_policy(s, a)     # ∇θ log π(a|s; θ) 계산 (shape: (2,))
        g_per_state[s] += grad * G_t     # g_s 누적
        F_per_state[s] += np.outer(grad, grad)  # F_s 누적 (2x2 outer product)

    # 4) Natural Gradient 업데이트
    #    θ_s ← θ_s + α * F_s^{-1} * g_s
    #    2x2 행렬이므로 상태별로 간단히 계산 가능
    for s in range(n_states):
        g_s = g_per_state[s]             # shape: (2,)
        F_s = F_per_state[s]             # shape: (2,2)

        # 에피소드 동안 해당 상태를 방문하지 않았다면 건너뜀
        if np.all(F_s == 0):
            continue

        # 수치 안정성을 위한 정규화(작은 항 추가): F_s_reg = F_s + λI
        lam = 1e-3
        F_s_reg = F_s + lam * np.eye(n_actions)

        # 2x2 행렬이므로 역행렬 계산 시도
        try:
            F_inv = np.linalg.inv(F_s_reg)
        except np.linalg.LinAlgError:
            # 역행렬 계산 실패 시, 대각 성분만 이용한 근사 역행렬 사용
            F_inv = np.diag(1.0 / np.diag(F_s_reg))

        natural_grad = F_inv @ g_s       # 자연 그래디언트: F_s^{-1} g_s  (shape: (2,))
        theta[s] += alpha * natural_grad # 상태 s에 대한 정책 파라미터 업데이트

    # 5) 에피소드 시작 시점 Return(G_0)을 기록 (로그용)
    episode_return = returns[0]          # G_0: 에피소드 전체 Return
    return_history.append(episode_return)

    # 50 에피소드마다 최근 50개 Return 평균을 출력 (학습 진행 확인 용도)
    if episode % 50 == 0:
        avg_ret = np.mean(return_history[-50:])
        print(f"[Episode {episode:4d}] 최근 50 에피소드 평균 Return = {avg_ret:.3f}")

print("\n=== NAC 학습 종료 ===\n")


# ==============================
# 5. 학습된 정책(상태별 행동 확률) 출력
# ==============================
print("▶ 학습된 정책 π(a|s; θ) (행: 상태, 열: 행동[←,→])")
for s in range(n_states):
    probs = policy_probs(s)              # 상태 s에서의 행동 확률
    print(f"상태 {s}: {probs}")

# Greedy 정책으로 사람이 보기 쉽게 화살표로 표현 (확률이 더 큰 행동 선택)
action_symbols = {0: "←", 1: "→"}        # 0은 왼쪽 화살표, 1은 오른쪽 화살표

print("\n▶ Greedy 기준 학습된 정책(Policy)")
policy_str = ""
for s in range(n_states):
    if s == n_states - 1:                # 목표 상태인 경우
        policy_str += " G "              # Goal 표기
    else:
        best_a = np.argmax(policy_probs(s))  # 가장 확률이 높은 행동 선택
        policy_str += f" {action_symbols[best_a]} "
print("상태 0  1  2  3  4")
print("     " + policy_str)


# ==============================
# 6. 학습된 정책으로 1회 테스트 실행
# ==============================
print("\n▶ 학습된 정책으로 1회 에피소드 실행 예시 (Greedy 정책 사용)")

state = reset()                          # 초기 상태로 리셋
trajectory = [state]                     # 방문한 상태들을 기록하기 위한 리스트

for step_idx in range(max_steps):
    probs = policy_probs(state)          # 현재 상태에서의 정책 확률
    action = np.argmax(probs)            # 가장 확률이 높은 행동을 선택 (탐험 없이 greedy)
    next_state, reward, done = step(state, action)  # 환경에 행동 적용

    trajectory.append(next_state)        # 방문한 상태 기록
    state = next_state                   # 다음 상태로 이동

    if done:                             # 목표에 도달하면 종료
        break

print("방문한 상태들:", trajectory)
print("스텝 수:", len(trajectory) - 1)
print("마지막 상태가 목표(4)면 학습 성공!")


=== 1차원 선형 월드에서의 NAC (Natural Actor-Critic) 학습 시작 ===
[Episode   50] 최근 50 에피소드 평균 Return = 0.725
[Episode  100] 최근 50 에피소드 평균 Return = 0.941
[Episode  150] 최근 50 에피소드 평균 Return = 0.941
[Episode  200] 최근 50 에피소드 평균 Return = 0.941
[Episode  250] 최근 50 에피소드 평균 Return = 0.941
[Episode  300] 최근 50 에피소드 평균 Return = 0.941
[Episode  350] 최근 50 에피소드 평균 Return = 0.941
[Episode  400] 최근 50 에피소드 평균 Return = 0.941
[Episode  450] 최근 50 에피소드 평균 Return = 0.941
[Episode  500] 최근 50 에피소드 평균 Return = 0.941

=== NAC 학습 종료 ===

▶ 학습된 정책 π(a|s; θ) (행: 상태, 열: 행동[←,→])
상태 0: [2.27874868e-05 9.99977213e-01]
상태 1: [2.21007898e-05 9.99977899e-01]
상태 2: [2.16594454e-05 9.99978341e-01]
상태 3: [2.11265621e-05 9.99978873e-01]
상태 4: [0.5 0.5]

▶ Greedy 기준 학습된 정책(Policy)
상태 0  1  2  3  4
      →  →  →  →  G 

▶ 학습된 정책으로 1회 에피소드 실행 예시 (Greedy 정책 사용)
방문한 상태들: [0, 1, 2, 3, 4]
스텝 수: 4
마지막 상태가 목표(4)면 학습 성공!


In [19]:
###################################################################
## (2-4) A2C/A3C(Advantage Actor-Critic): 분산형 Actor-Critic 모델
###################################################################
import numpy as np  # 수치 계산을 위한 NumPy 불러오기

# ==============================
# 1. 환경 정의 (1차원 선형 월드)
# ==============================
n_states = 5        # 상태 개수: 0,1,2,3,4 (4가 목표 상태)
n_actions = 2       # 행동 개수: 0=왼쪽, 1=오른쪽

def step(state, action):
    # 주어진 상태 state에서 행동 action을 했을 때
    # 다음 상태(next_state), 보상(reward), 종료 여부(done)를 반환하는 함수

    # 행동이 0이면 왼쪽으로 한 칸 이동, 1이면 오른쪽으로 한 칸 이동
    if action == 0:  # 왼쪽 이동
        next_state = max(0, state - 1)     # 상태 0보다 왼쪽으로는 가지 않도록 최소값 제한
    else:            # 오른쪽 이동
        next_state = min(n_states - 1, state + 1)  # 상태 4보다 오른쪽으로는 가지 않도록 최대값 제한

    # 보상과 종료 조건 설정
    if next_state == n_states - 1:         # 목표 상태(4)에 도달한 경우
        reward = 1.0                       # 성공 보상 1.0 부여
        done = True                        # 에피소드 종료
    else:
        reward = -0.01                     # 그 외 상태에서는 시간 패널티 -0.01 부여 (빨리 도달 유도)
        done = False                       # 에피소드 계속 진행

    return next_state, reward, done        # 결과 반환

def reset():
    # 에피소드 시작 시 초기 상태를 반환하는 함수
    # 여기서는 항상 상태 0에서 시작
    return 0


# ==============================
# 2. A2C/A3C 하이퍼파라미터
# ==============================
np.random.seed(42)   # 난수 시드 고정 (실행결과 재현 가능)

gamma = 0.99         # 할인율
alpha_theta = 0.05   # Actor(정책 파라미터) 학습률
alpha_v = 0.1        # Critic(가치함수) 학습률
n_episodes = 500     # 전체 학습 에피소드 수
max_steps  = 20      # 한 에피소드에서 허용하는 최대 스텝 수

t_max = 5            # A2C/A3C 스타일 n-step rollout 길이 (멀티스텝 Advantage 계산용)

# 정책 파라미터 theta: (상태 x 행동) 크기의 실수 행렬
# 각 상태에서 두 행동(왼쪽, 오른쪽)에 대한 "선호도(preference)"를 나타냄
theta = np.zeros((n_states, n_actions))    # 처음에는 모든 선호도를 0으로 시작

# 가치함수 파라미터 v: 각 상태의 상태가치 V(s)를 나타내는 벡터
v = np.zeros(n_states)                     # 초기에는 모든 상태가치를 0으로 시작


# ==============================
# 3. 정책 관련 함수 정의
# ==============================
def softmax(logits):
    # 주어진 선호도 벡터 logits를 softmax를 이용해 확률 분포로 변환하는 함수
    c = np.max(logits)                     # 수치 안정성을 위해 최대값을 빼줌
    exp = np.exp(logits - c)               # 지수 함수 적용
    return exp / np.sum(exp)               # 전체 합으로 나누어 확률 분포로 변환

def policy_probs(state):
    # 주어진 상태 state에서의 행동 확률 π(a|s; θ)를 반환하는 함수
    # theta[state]는 해당 상태에서 각 행동에 대한 선호도 벡터
    return softmax(theta[state])           # softmax를 적용하여 확률로 변환

def sample_action(state):
    # 현재 정책에 따라 행동을 하나 샘플링하는 함수
    probs = policy_probs(state)            # 행동 확률 π(a|s; θ) 구하기
    return np.random.choice(n_actions, p=probs)  # 해당 확률에 따라 0 또는 1 선택

def grad_log_policy(state, action):
    # ∇θ log π(a|s; θ) 를 계산하는 함수
    # 반환값은 길이 2의 벡터 (각 행동에 대한 gradient)
    probs = policy_probs(state)            # π(a|s; θ) 계산
    grad = -probs.copy()                   # 기본값은 -π(a|s; θ)
    grad[action] += 1.0                    # 선택된 행동 a에 대해서는 +1 더해줌
    # 결과적으로:
    # grad[k] = 1 - π(k|s)  if k == action
    # grad[k] = -π(k|s)     if k != action
    return grad                           # shape: (2,)


# ==============================
# 4. A2C/A3C 스타일 Advantage Actor-Critic 학습 루프
# ==============================
return_history = []                       # 각 에피소드의 총 Return(G_0)을 기록할 리스트

print("=== 1차원 선형 월드에서의 A2C/A3C(Advantage Actor-Critic) 학습 시작 ===")

for episode in range(1, n_episodes + 1):
    state = reset()                       # 에피소드 시작 상태
    G0 = 0.0                              # 에피소드 시작 Return(G_0) 추적용
    discount = 1.0                        # γ^t 계수 누적용

    step_count = 0                        # 에피소드 내 전체 스텝 카운터

    while step_count < max_steps:
        # 1) 최대 t_max 길이까지 rollout을 수행하며 (s_t, a_t, r_t, done) 시퀀스를 모음
        states = []                       # 미니배치 내부 상태 리스트
        actions = []                      # 미니배치 내부 행동 리스트
        rewards = []                      # 미니배치 내부 보상 리스트
        dones = []                        # 미니배치 내부 done 플래그 리스트

        for t in range(t_max):
            # 현재 상태에서 정책에 따라 행동 샘플링
            action = sample_action(state)

            # 환경에 행동 적용
            next_state, reward, done = step(state, action)

            # rollout 시퀀스에 기록
            states.append(state)
            actions.append(action)
            rewards.append(reward)
            dones.append(done)

            # 에피소드 Return(G_0) 추적 (로그용)
            G0 += discount * reward
            discount *= gamma

            step_count += 1               # 전체 스텝 수 증가
            state = next_state            # 다음 상태로 전이

            if done or step_count >= max_steps:
                # 목표 도달 또는 최대 스텝 도달 시 rollout 종료
                break

        # 2) rollout의 마지막 상태에서 bootstrap 값 R 초기화
        #    A2C/A3C 스타일 n-step return:
        #    R = V(s_{t_end})  (단, terminal이면 0)
        if len(states) == 0:
            # 방어적 코드: rollout이 비어 있으면 다음 에피소드로
            break

        if dones[-1]:                     # 마지막 스텝에서 에피소드가 끝났다면
            R = 0.0                       # terminal state 이므로 bootstrap 없음
        else:
            R = v[state]                  # 아직 끝나지 않았다면 V(s_last)를 bootstrap 값으로 사용

        # 3) rollout을 거꾸로 돌면서 n-step Return과 Advantage를 계산하고
        #    Actor와 Critic을 동시에 업데이트
        for i in reversed(range(len(states))):
            s = states[i]                 # 시점 i의 상태
            a = actions[i]                # 시점 i의 행동
            r = rewards[i]                # 시점 i의 보상

            # n-step return R_t 계산: R_t = r_t + γ R_{t+1}
            R = r + gamma * R

            # Advantage A_t = R_t - V(s_t)
            V_s = v[s]
            advantage = R - V_s

            # Critic 업데이트: V(s) ← V(s) + α_v * Advantage
            v[s] += alpha_v * advantage

            # Actor 업데이트: θ ← θ + α_θ * Advantage * ∇θ log π(a|s; θ)
            grad = grad_log_policy(s, a)
            theta[s] += alpha_theta * advantage * grad

        # rollout 종료 후, 에피소드가 종료되었다면 while 루프 탈출
        if dones[-1]:
            break

    # 한 에피소드가 끝나면 Return(G_0)을 기록
    return_history.append(G0)

    # 50 에피소드마다 최근 50개 Return 평균을 출력 (학습 진행 확인 용도)
    if episode % 50 == 0:
        avg_ret = np.mean(return_history[-50:])
        print(f"[Episode {episode:4d}] 최근 50 에피소드 평균 Return = {avg_ret:.3f}")

print("\n=== A2C/A3C 학습 종료 ===\n")


# ==============================
# 5. 학습된 정책(상태별 행동 확률) 출력
# ==============================
print("▶ 학습된 정책 π(a|s; θ) (행: 상태, 열: 행동[←,→])")
for s in range(n_states):
    probs = policy_probs(s)              # 상태 s에서의 행동 확률
    print(f"상태 {s}: {probs}")

# Greedy 정책으로 사람이 보기 쉽게 화살표로 표현 (확률이 더 큰 행동 선택)
action_symbols = {0: "←", 1: "→"}        # 0은 왼쪽 화살표, 1은 오른쪽 화살표

print("\n▶ Greedy 기준 학습된 정책(Policy)")
policy_str = ""
for s in range(n_states):
    if s == n_states - 1:                # 목표 상태인 경우
        policy_str += " G "              # Goal 표기
    else:
        best_a = np.argmax(policy_probs(s))  # 가장 확률이 높은 행동 선택
        policy_str += f" {action_symbols[best_a]} "
print("상태 0  1  2  3  4")
print("     " + policy_str)


# ==============================
# 6. 학습된 정책으로 1회 테스트 실행
# ==============================
print("\n▶ 학습된 정책으로 1회 에피소드 실행 예시 (Greedy 정책 사용)")

state = reset()                          # 초기 상태로 리셋
trajectory = [state]                     # 방문한 상태들을 기록하기 위한 리스트

for step_idx in range(max_steps):
    probs = policy_probs(state)          # 현재 상태에서의 정책 확률
    action = np.argmax(probs)            # 가장 확률이 높은 행동을 선택 (탐험 없이 greedy)
    next_state, reward, done = step(state, action)  # 환경에 행동 적용

    trajectory.append(next_state)        # 방문한 상태 기록
    state = next_state                   # 다음 상태로 이동

    if done:                             # 목표에 도달하면 종료
        break

print("방문한 상태들:", trajectory)
print("스텝 수:", len(trajectory) - 1)
print("마지막 상태가 목표(4)면 학습 성공!")


=== 1차원 선형 월드에서의 A2C/A3C(Advantage Actor-Critic) 학습 시작 ===
[Episode   50] 최근 50 에피소드 평균 Return = 0.661
[Episode  100] 최근 50 에피소드 평균 Return = 0.770
[Episode  150] 최근 50 에피소드 평균 Return = 0.849
[Episode  200] 최근 50 에피소드 평균 Return = 0.853
[Episode  250] 최근 50 에피소드 평균 Return = 0.880
[Episode  300] 최근 50 에피소드 평균 Return = 0.894
[Episode  350] 최근 50 에피소드 평균 Return = 0.893
[Episode  400] 최근 50 에피소드 평균 Return = 0.900
[Episode  450] 최근 50 에피소드 평균 Return = 0.884
[Episode  500] 최근 50 에피소드 평균 Return = 0.897

=== A2C/A3C 학습 종료 ===

▶ 학습된 정책 π(a|s; θ) (행: 상태, 열: 행동[←,→])
상태 0: [0.29026512 0.70973488]
상태 1: [0.17356323 0.82643677]
상태 2: [0.16023098 0.83976902]
상태 3: [0.19077707 0.80922293]
상태 4: [0.5 0.5]

▶ Greedy 기준 학습된 정책(Policy)
상태 0  1  2  3  4
      →  →  →  →  G 

▶ 학습된 정책으로 1회 에피소드 실행 예시 (Greedy 정책 사용)
방문한 상태들: [0, 1, 2, 3, 4]
스텝 수: 4
마지막 상태가 목표(4)면 학습 성공!


In [20]:
###################################################################
## (2-5) ACER(Actor-Critic with Experience Replay): 경험 리플레이를 추가한 Actor-Critic 방법
###################################################################
import numpy as np  # 수치 계산을 위한 NumPy 불러오기

# ==============================
# 1. 환경 정의 (1차원 선형 월드)
# ==============================
n_states = 5        # 상태 개수: 0,1,2,3,4 (4가 목표 상태)
n_actions = 2       # 행동 개수: 0=왼쪽, 1=오른쪽

def step(state, action):
    # 주어진 상태 state에서 행동 action을 했을 때
    # 다음 상태(next_state), 보상(reward), 종료 여부(done)를 반환하는 함수

    # 행동이 0이면 왼쪽으로 한 칸 이동, 1이면 오른쪽으로 한 칸 이동
    if action == 0:  # 왼쪽 이동
        next_state = max(0, state - 1)     # 상태 0보다 왼쪽으로는 가지 않도록 최소값 제한
    else:            # 오른쪽 이동
        next_state = min(n_states - 1, state + 1)  # 상태 4보다 오른쪽으로는 가지 않도록 최대값 제한

    # 보상과 종료 조건 설정
    if next_state == n_states - 1:         # 목표 상태(4)에 도달한 경우
        reward = 1.0                       # 성공 보상 1.0 부여
        done = True                        # 에피소드 종료
    else:
        reward = -0.01                     # 그 외 상태에서는 시간 패널티 -0.01 부여 (빨리 도달 유도)
        done = False                       # 에피소드 계속 진행

    return next_state, reward, done        # 결과 반환

def reset():
    # 에피소드 시작 시 초기 상태를 반환하는 함수
    # 여기서는 항상 상태 0에서 시작
    return 0


# ==============================
# 2. ACER 하이퍼파라미터
# ==============================
np.random.seed(42)   # 난수 시드 고정 (실행결과 재현 가능)

gamma = 0.99         # 할인율
alpha_theta = 0.05   # Actor(정책 파라미터) 학습률
alpha_v = 0.1        # Critic(가치함수) 학습률

n_episodes = 500     # 전체 학습 에피소드 수
max_steps  = 20      # 한 에피소드에서 허용하는 최대 스텝 수

c_bar = 10.0         # 중요도 비율(truncated importance sampling) 상한값

# 정책 파라미터 theta: (상태 x 행동) 크기의 실수 행렬
# 각 상태에서 두 행동(왼쪽, 오른쪽)에 대한 "선호도(preference)"를 나타냄
theta = np.zeros((n_states, n_actions))    # 처음에는 모든 선호도를 0으로 시작

# 가치함수 파라미터 v: 각 상태의 상태가치 V(s)를 나타내는 벡터
v = np.zeros(n_states)                     # 초기에는 모든 상태가치를 0으로 시작

# 리플레이 버퍼 (간단하게 파이썬 리스트로 구현)
# 각 원소는 (s, a, r, s_next, done, mu_a) 튜플
replay_buffer = []
buffer_capacity = 10000                    # 버퍼 최대 크기
min_buffer_size = 50                       # 리플레이 시작을 위한 최소 저장 개수


# ==============================
# 3. 정책 관련 함수 정의
# ==============================
def softmax(logits):
    # 주어진 선호도 벡터 logits를 softmax를 이용해 확률 분포로 변환하는 함수
    c = np.max(logits)                     # 수치 안정성을 위해 최대값을 빼줌
    exp = np.exp(logits - c)               # 지수 함수 적용
    return exp / np.sum(exp)               # 전체 합으로 나누어 확률 분포로 변환

def policy_probs(state):
    # 주어진 상태 state에서의 행동 확률 π(a|s; θ)를 반환하는 함수
    # theta[state]는 해당 상태에서 각 행동에 대한 선호도 벡터
    return softmax(theta[state])           # softmax를 적용하여 확률로 변환

def sample_action(state):
    # 현재 정책에 따라 행동을 하나 샘플링하는 함수
    probs = policy_probs(state)            # 행동 확률 π(a|s; θ) 구하기
    return np.random.choice(n_actions, p=probs), probs  # 행동과 그 때의 확률분포 반환

def grad_log_policy(state, action):
    # ∇θ log π(a|s; θ) 를 계산하는 함수
    # 반환값은 길이 2의 벡터 (각 행동에 대한 gradient)
    probs = policy_probs(state)            # π(a|s; θ) 계산
    grad = -probs.copy()                   # 기본값은 -π(a|s; θ)
    grad[action] += 1.0                    # 선택된 행동 a에 대해서는 +1 더해줌
    # 결과적으로:
    # grad[k] = 1 - π(k|s)  if k == action
    # grad[k] = -π(k|s)     if k != action
    return grad                           # shape: (2,)


# ==============================
# 4. 리플레이 버퍼에서 샘플링하여 ACER 업데이트 수행
# ==============================
def acer_replay_update():
    # 리플레이 버퍼가 충분히 쌓이지 않았으면 업데이트를 하지 않음
    if len(replay_buffer) < min_buffer_size:
        return

    # 리플레이 버퍼에서 하나의 transition을 무작위 샘플링
    idx = np.random.randint(len(replay_buffer))
    s, a, r, s_next, done, mu_a = replay_buffer[idx]

    # 현재 정책 하에서의 행동 확률 π(a|s; θ)
    probs = policy_probs(s)
    pi_a = probs[a]

    # 중요도 비율 ρ = π(a|s) / μ(a|s)
    rho = pi_a / (mu_a + 1e-8)

    # truncated importance sampling: c = min(c_bar, ρ)
    c = min(c_bar, rho)

    # TD 타깃 및 Advantage 계산
    v_s = v[s]
    v_s_next = v[s_next] if not done else 0.0
    target = r + gamma * v_s_next
    advantage = target - v_s

    # Critic 업데이트: V(s) ← V(s) + α_v * Advantage
    v[s] += alpha_v * advantage

    # Actor 업데이트 (off-policy 보정): θ ← θ + α_θ * c * Advantage * ∇θ log π(a|s; θ)
    grad = grad_log_policy(s, a)
    theta[s] += alpha_theta * c * advantage * grad


# ==============================
# 5. ACER 학습 루프
# ==============================
return_history = []                       # 각 에피소드의 총 Return(G_0)을 기록할 리스트

print("=== 1차원 선형 월드에서의 ACER(Actor-Critic with Experience Replay) 학습 시작 ===")

for episode in range(1, n_episodes + 1):
    state = reset()                       # 에피소드 시작 상태
    G0 = 0.0                              # 에피소드 시작 Return(G_0) 추적용
    discount = 1.0                        # γ^t 계수 누적용

    for step_idx in range(max_steps):
        # 1) 현재 정책(behavior policy)로부터 행동을 샘플링
        action, probs = sample_action(state)
        mu_a = probs[action]              # 행동 a를 선택한 behavior policy 확률 μ(a|s)

        # 2) 환경에 행동 적용
        next_state, reward, done = step(state, action)

        # 3) on-policy TD 업데이트 (기본 Actor-Critic 업데이트)
        v_s = v[state]
        v_s_next = v[next_state] if not done else 0.0
        target = reward + gamma * v_s_next
        td_error = target - v_s

        # Critic 업데이트
        v[state] += alpha_v * td_error

        # Actor 업데이트 (on-policy): θ ← θ + α_θ * δ * ∇θ log π(a|s; θ)
        grad = grad_log_policy(state, action)
        theta[state] += alpha_theta * td_error * grad

        # 4) 리플레이 버퍼에 transition 추가
        if len(replay_buffer) >= buffer_capacity:
            # 버퍼가 가득 찬 경우, 가장 오래된 데이터를 제거
            replay_buffer.pop(0)
        replay_buffer.append((state, action, reward, next_state, done, mu_a))

        # 5) 리플레이 버퍼에서 off-policy ACER 업데이트 한 번 수행
        acer_replay_update()

        # 6) 에피소드 Return(G_0) 추적 (로그용)
        G0 += discount * reward
        discount *= gamma

        state = next_state                # 다음 상태로 전이

        if done:
            # 목표 상태 도달 시 에피소드 종료
            break

    # 한 에피소드가 끝나면 Return(G_0)을 기록
    return_history.append(G0)

    # 50 에피소드마다 최근 50개 Return 평균을 출력 (학습 진행 확인 용도)
    if episode % 50 == 0:
        avg_ret = np.mean(return_history[-50:])
        print(f"[Episode {episode:4d}] 최근 50 에피소드 평균 Return = {avg_ret:.3f}")

print("\n=== ACER 학습 종료 ===\n")


# ==============================
# 6. 학습된 정책(상태별 행동 확률) 출력
# ==============================
print("▶ 학습된 정책 π(a|s; θ) (행: 상태, 열: 행동[←,→])")
for s in range(n_states):
    probs = policy_probs(s)              # 상태 s에서의 행동 확률
    print(f"상태 {s}: {probs}")

# Greedy 정책으로 사람이 보기 쉽게 화살표로 표현 (확률이 더 큰 행동 선택)
action_symbols = {0: "←", 1: "→"}        # 0은 왼쪽 화살표, 1은 오른쪽 화살표

print("\n▶ Greedy 기준 학습된 정책(Policy)")
policy_str = ""
for s in range(n_states):
    if s == n_states - 1:                # 목표 상태인 경우
        policy_str += " G "              # Goal 표기
    else:
        best_a = np.argmax(policy_probs(s))  # 가장 확률이 높은 행동 선택
        policy_str += f" {action_symbols[best_a]} "
print("상태 0  1  2  3  4")
print("     " + policy_str)


# ==============================
# 7. 학습된 정책으로 1회 테스트 실행
# ==============================
print("\n▶ 학습된 정책으로 1회 에피소드 실행 예시 (Greedy 정책 사용)")

state = reset()                          # 초기 상태로 리셋
trajectory = [state]                     # 방문한 상태들을 기록하기 위한 리스트

for step_idx in range(max_steps):
    probs = policy_probs(state)          # 현재 상태에서의 정책 확률
    action = np.argmax(probs)            # 가장 확률이 높은 행동을 선택 (탐험 없이 greedy)
    next_state, reward, done = step(state, action)  # 환경에 행동 적용

    trajectory.append(next_state)        # 방문한 상태 기록
    state = next_state                   # 다음 상태로 이동

    if done:                             # 목표에 도달하면 종료
        break

print("방문한 상태들:", trajectory)
print("스텝 수:", len(trajectory) - 1)
print("마지막 상태가 목표(4)면 학습 성공!")


=== 1차원 선형 월드에서의 ACER(Actor-Critic with Experience Replay) 학습 시작 ===
[Episode   50] 최근 50 에피소드 평균 Return = 0.665
[Episode  100] 최근 50 에피소드 평균 Return = 0.886
[Episode  150] 최근 50 에피소드 평균 Return = 0.898
[Episode  200] 최근 50 에피소드 평균 Return = 0.917
[Episode  250] 최근 50 에피소드 평균 Return = 0.891
[Episode  300] 최근 50 에피소드 평균 Return = 0.917
[Episode  350] 최근 50 에피소드 평균 Return = 0.916
[Episode  400] 최근 50 에피소드 평균 Return = 0.911
[Episode  450] 최근 50 에피소드 평균 Return = 0.923
[Episode  500] 최근 50 에피소드 평균 Return = 0.923

=== ACER 학습 종료 ===

▶ 학습된 정책 π(a|s; θ) (행: 상태, 열: 행동[←,→])
상태 0: [0.18338427 0.81661573]
상태 1: [0.10233127 0.89766873]
상태 2: [0.10469859 0.89530141]
상태 3: [0.11631745 0.88368255]
상태 4: [0.5 0.5]

▶ Greedy 기준 학습된 정책(Policy)
상태 0  1  2  3  4
      →  →  →  →  G 

▶ 학습된 정책으로 1회 에피소드 실행 예시 (Greedy 정책 사용)
방문한 상태들: [0, 1, 2, 3, 4]
스텝 수: 4
마지막 상태가 목표(4)면 학습 성공!


In [21]:
###################################################################
## (2-6) IMPALA(Importance Weighted Actor-Learner Architecture): 분산 학습에 최적화된 구조
###################################################################
import numpy as np  # 수치 계산을 위한 NumPy 불러오기

# ==============================
# 1. 환경 정의 (1차원 선형 월드)
# ==============================
n_states = 5        # 상태 개수: 0,1,2,3,4 (4가 목표 상태)
n_actions = 2       # 행동 개수: 0=왼쪽, 1=오른쪽

def step(state, action):
    # 주어진 상태 state에서 행동 action을 했을 때
    # 다음 상태(next_state), 보상(reward), 종료 여부(done)를 반환하는 함수

    # 행동이 0이면 왼쪽으로 한 칸 이동, 1이면 오른쪽으로 한 칸 이동
    if action == 0:  # 왼쪽 이동
        next_state = max(0, state - 1)     # 상태 0보다 왼쪽으로는 가지 않도록 최소값 제한
    else:            # 오른쪽 이동
        next_state = min(n_states - 1, state + 1)  # 상태 4보다 오른쪽으로는 가지 않도록 최대값 제한

    # 보상과 종료 조건 설정
    if next_state == n_states - 1:         # 목표 상태(4)에 도달한 경우
        reward = 1.0                       # 성공 보상 1.0 부여
        done = True                        # 에피소드 종료
    else:
        reward = -0.01                     # 그 외 상태에서는 시간 패널티 -0.01 부여 (빨리 도달 유도)
        done = False                       # 에피소드 계속 진행

    return next_state, reward, done        # 결과 반환

def reset():
    # 에피소드 시작 시 초기 상태를 반환하는 함수
    # 여기서는 항상 상태 0에서 시작
    return 0


# ==============================
# 2. IMPALA 하이퍼파라미터
# ==============================
np.random.seed(42)    # 난수 시드 고정 (실행결과 재현 가능)

gamma = 0.99          # 할인율
alpha_theta = 0.05    # Actor(정책 파라미터) 학습률
alpha_v = 0.1         # Critic(가치함수) 학습률

n_episodes = 500      # 전체 학습 에피소드 수
max_steps  = 20       # 한 에피소드에서 허용하는 최대 스텝 수

rho_bar = 1.0         # 중요도 비율 ρ̄ 상한 (정책 gradient에 사용)
c_bar   = 1.0         # c_t 상한 (V-trace 보정에 사용)

sync_rate = 0.1       # learner → behavior policy로 옮기는 속도 (지연 정도를 만들기 위함)

# 학습용 정책 파라미터(learner): theta_learner[s, a]
theta_learner = np.zeros((n_states, n_actions))   # 초기값 0

# 데이터 수집용 정책 파라미터(behavior): theta_behavior[s, a]
theta_behavior = np.zeros((n_states, n_actions))  # 초기에는 learner와 동일

# 가치함수 파라미터 v: 각 상태의 상태가치 V(s)를 나타내는 벡터
v = np.zeros(n_states)                            # 초기에는 모든 상태가치 0


# ==============================
# 3. 정책 관련 함수 정의
# ==============================
def softmax(logits):
    # 주어진 선호도 벡터 logits를 softmax를 이용해 확률 분포로 변환하는 함수
    c = np.max(logits)                      # 수치 안정성을 위해 최대값을 빼줌
    exp = np.exp(logits - c)                # 지수 함수 적용
    return exp / np.sum(exp)                # 전체 합으로 나누어 확률 분포로 변환

def policy_probs(theta, state):
    # 주어진 파라미터 theta와 상태 state에서의 행동 확률 π(a|s; θ)를 반환하는 함수
    # theta[state]는 해당 상태에서 각 행동에 대한 선호도 벡터
    return softmax(theta[state])            # softmax를 적용하여 확률로 변환

def sample_action(theta, state):
    # 주어진 정책 파라미터 theta를 사용하여 행동을 샘플링하는 함수
    probs = policy_probs(theta, state)      # 행동 확률 π(a|s; θ) 계산
    action = np.random.choice(n_actions, p=probs)  # 해당 확률에 따라 0 또는 1 선택
    return action, probs                    # 선택한 행동과 확률 분포 반환

def grad_log_policy(theta, state, action):
    # ∇θ log π(a|s; θ) 를 계산하는 함수
    # 반환값은 길이 2의 벡터 (각 행동에 대한 gradient)
    probs = policy_probs(theta, state)      # π(a|s; θ) 계산
    grad = -probs.copy()                    # 기본값은 -π(a|s; θ)
    grad[action] += 1.0                     # 선택된 행동 a에 대해서는 +1 더해줌
    # 결과적으로:
    # grad[k] = 1 - π(k|s)  if k == action
    # grad[k] = -π(k|s)     if k != action
    return grad                             # shape: (2,)


# ==============================
# 4. IMPALA 스타일 V-trace 업데이트
# ==============================
def impala_update_trajectory(states, actions, rewards, dones, behavior_probs_seq):
    # 하나의 trajectory(에피소드 또는 일부 구간)에 대해
    # IMPALA의 V-trace 아이디어를 이용하여 learner 파라미터를 업데이트하는 함수
    global theta_learner, v                # 외부에서 정의한 파라미터 사용

    # trajectory 길이 계산
    T = len(states)                        # 시퀀스의 길이 (0 ~ T-1)

    # 각 시점에서 learner 정책과 behavior 정책의 확률, 중요도 비율 계산
    rhos = np.zeros(T)                     # 중요도 비율 ρ_t = π(a_t|s_t) / μ(a_t|s_t)
    cs   = np.zeros(T)                     # V-trace 보정 계수 c_t = min(c_bar, ρ_t)

    # 각 시점에서의 V(s_t)와 V(s_{t+1})를 미리 계산
    V_s      = np.zeros(T)                 # V(s_t)
    V_s_next = np.zeros(T)                 # V(s_{t+1}), terminal인 경우 0

    for t in range(T):
        s = states[t]                      # 시점 t의 상태
        a = actions[t]                     # 시점 t의 행동

        # learner 정책에서의 행동 확률 π(a_t|s_t)
        pi_probs = policy_probs(theta_learner, s)
        pi_a = pi_probs[a]

        # behavior 정책에서의 행동 확률 μ(a_t|s_t)
        mu_probs = behavior_probs_seq[t]
        mu_a = mu_probs[a]

        # 중요도 비율 ρ_t 계산 (작은 epsilon으로 0 나누기 방지)
        rho = pi_a / (mu_a + 1e-8)
        rhos[t] = min(rho_bar, rho)        # ρ̄_t = min(ρ_bar, ρ_t)
        cs[t]   = min(c_bar, rho)          # c_t   = min(c_bar, ρ_t)

        # 가치함수 값 계산
        V_s[t] = v[s]                      # V(s_t)
        if dones[t]:
            V_s_next[t] = 0.0             # terminal이면 V(s_{t+1}) = 0
        else:
            V_s_next[t] = v[states[t+1]] if t+1 < T else v[states[t]]  # 마지막이면 대충 자기 자신

    # V-trace 타깃 v_target_t를 뒤에서부터 역순으로 계산
    v_target = np.zeros(T)                 # 각 시점의 v_target_t
    # 마지막 시점부터 역순으로 계산
    for t in reversed(range(T)):
        # δ^V_t = ρ̄_t * (r_t + γ V(s_{t+1}) - V(s_t))
        delta_v = rhos[t] * (rewards[t] + gamma * V_s_next[t] - V_s[t])
        if t == T - 1:
            # 마지막 시점에서는 v_target_T = V(s_T) + δ^V_T
            v_target[t] = V_s[t] + delta_v
        else:
            # v_target_t = V(s_t) + δ^V_t + γ c_t (v_target_{t+1} - V(s_{t+1}))
            v_target[t] = V_s[t] + delta_v + gamma * cs[t] * (v_target[t+1] - V_s_next[t])

    # 이제 v_target을 사용하여 Critic와 Actor(learner)를 업데이트
    for t in range(T):
        s = states[t]                      # 상태 s_t
        a = actions[t]                     # 행동 a_t

        # Advantage_t = v_target_t - V(s_t)
        advantage = v_target[t] - v[s]

        # Critic 업데이트: V(s) ← V(s) + α_v * Advantage_t
        v[s] += alpha_v * advantage

        # Actor(learner) 업데이트:
        # θ ← θ + α_θ * ρ̄_t * Advantage_t * ∇θ log π(a_t|s_t; θ)
        grad = grad_log_policy(theta_learner, s, a)
        theta_learner[s] += alpha_theta * rhos[t] * advantage * grad


# ==============================
# 5. IMPALA 학습 루프
# ==============================
return_history = []                       # 각 에피소드의 총 Return(G_0)을 기록할 리스트

print("=== 1차원 선형 월드에서의 IMPALA(Importance Weighted Actor-Learner Architecture) 학습 시작 ===")

for episode in range(1, n_episodes + 1):
    state = reset()                       # 에피소드 시작 상태
    G0 = 0.0                              # 에피소드 Return(G_0) 추적용
    discount = 1.0                        # γ^t 계수 누적용

    # 하나의 trajectory(에피소드 전체)를 behavior policy로 수집
    states = []                           # 상태 시퀀스
    actions = []                          # 행동 시퀀스
    rewards = []                          # 보상 시퀀스
    dones = []                            # done 플래그 시퀀스
    behavior_probs_seq = []               # 각 시점의 behavior 정책 확률 분포 μ(·|s_t)

    for step_idx in range(max_steps):
        # behavior policy(theta_behavior)를 사용하여 행동 샘플링
        action, probs_b = sample_action(theta_behavior, state)

        # 환경에 행동 적용
        next_state, reward, done = step(state, action)

        # 시퀀스에 저장
        states.append(state)
        actions.append(action)
        rewards.append(reward)
        dones.append(done)
        behavior_probs_seq.append(probs_b)

        # Return(G_0) 계산용
        G0 += discount * reward
        discount *= gamma

        state = next_state                # 다음 상태로 전이

        if done:
            # 목표 도달 시 에피소드 종료
            break

    # 수집된 trajectory를 이용해 IMPALA 스타일 V-trace 업데이트 수행
    impala_update_trajectory(states, actions, rewards, dones, behavior_probs_seq)

    # learner 정책을 behavior 정책 쪽으로 부분 반영 (지연된 actor 효과)
    theta_behavior = (1.0 - sync_rate) * theta_behavior + sync_rate * theta_learner

    # 에피소드 Return 기록
    return_history.append(G0)

    # 50 에피소드마다 최근 50개 Return 평균 출력
    if episode % 50 == 0:
        avg_ret = np.mean(return_history[-50:])
        print(f"[Episode {episode:4d}] 최근 50 에피소드 평균 Return = {avg_ret:.3f}")

print("\n=== IMPALA 학습 종료 ===\n")


# ==============================
# 6. 학습된 정책(상태별 행동 확률) 출력
# ==============================
print("▶ 학습된 정책 π(a|s; θ_learner) (행: 상태, 열: 행동[←,→])")
for s in range(n_states):
    probs = policy_probs(theta_learner, s)  # learner 정책에서의 행동 확률
    print(f"상태 {s}: {probs}")

# Greedy 정책으로 사람이 보기 쉽게 화살표로 표현 (확률이 더 큰 행동 선택)
action_symbols = {0: "←", 1: "→"}        # 0은 왼쪽 화살표, 1은 오른쪽 화살표

print("\n▶ Greedy 기준 학습된 정책(Policy)")
policy_str = ""
for s in range(n_states):
    if s == n_states - 1:                # 목표 상태인 경우
        policy_str += " G "              # Goal 표기
    else:
        best_a = np.argmax(policy_probs(theta_learner, s))  # learner 기준 greedy 행동
        policy_str += f" {action_symbols[best_a]} "
print("상태 0  1  2  3  4")
print("     " + policy_str)


# ==============================
# 7. 학습된 정책으로 1회 테스트 실행
# ==============================
print("\n▶ 학습된 정책으로 1회 에피소드 실행 예시 (Greedy 정책 사용, learner 기준)")

state = reset()                          # 초기 상태로 리셋
trajectory = [state]                     # 방문한 상태들을 기록하기 위한 리스트

for step_idx in range(max_steps):
    probs = policy_probs(theta_learner, state)  # learner 정책에서의 행동 확률
    action = np.argmax(probs)                  # 가장 확률이 높은 행동 선택 (탐험 없이 greedy)
    next_state, reward, done = step(state, action)  # 환경에 행동 적용

    trajectory.append(next_state)        # 방문한 상태 기록
    state = next_state                   # 다음 상태로 이동

    if done:                             # 목표에 도달하면 종료
        break

print("방문한 상태들:", trajectory)
print("스텝 수:", len(trajectory) - 1)
print("마지막 상태가 목표(4)면 학습 성공!")


=== 1차원 선형 월드에서의 IMPALA(Importance Weighted Actor-Learner Architecture) 학습 시작 ===
[Episode   50] 최근 50 에피소드 평균 Return = 0.535
[Episode  100] 최근 50 에피소드 평균 Return = 0.688
[Episode  150] 최근 50 에피소드 평균 Return = 0.753
[Episode  200] 최근 50 에피소드 평균 Return = 0.855
[Episode  250] 최근 50 에피소드 평균 Return = 0.884
[Episode  300] 최근 50 에피소드 평균 Return = 0.883
[Episode  350] 최근 50 에피소드 평균 Return = 0.888
[Episode  400] 최근 50 에피소드 평균 Return = 0.868
[Episode  450] 최근 50 에피소드 평균 Return = 0.890
[Episode  500] 최근 50 에피소드 평균 Return = 0.890

=== IMPALA 학습 종료 ===

▶ 학습된 정책 π(a|s; θ_learner) (행: 상태, 열: 행동[←,→])
상태 0: [0.33174848 0.66825152]
상태 1: [0.18933591 0.81066409]
상태 2: [0.18348034 0.81651966]
상태 3: [0.18936821 0.81063179]
상태 4: [0.5 0.5]

▶ Greedy 기준 학습된 정책(Policy)
상태 0  1  2  3  4
      →  →  →  →  G 

▶ 학습된 정책으로 1회 에피소드 실행 예시 (Greedy 정책 사용, learner 기준)
방문한 상태들: [0, 1, 2, 3, 4]
스텝 수: 4
마지막 상태가 목표(4)면 학습 성공!


In [22]:
###################################################################
## (2-7) Off-PAC(Off-Policy Actor-Critic): 오프폴리시 데이터를 활용하는 Actor-Critic 기법
###################################################################
import numpy as np  # 수치 계산을 위한 NumPy 불러오기

# ==============================
# 1. 환경 정의 (1차원 선형 월드)
# ==============================
n_states = 5        # 상태 개수: 0,1,2,3,4 (4가 목표 상태)
n_actions = 2       # 행동 개수: 0=왼쪽, 1=오른쪽

def step(state, action):
    # 주어진 상태 state에서 행동 action을 했을 때
    # 다음 상태(next_state), 보상(reward), 종료 여부(done)를 반환하는 함수

    # 행동이 0이면 왼쪽으로 한 칸 이동, 1이면 오른쪽으로 한 칸 이동
    if action == 0:  # 왼쪽 이동
        next_state = max(0, state - 1)     # 상태 0보다 왼쪽으로는 가지 않도록 최소값 제한
    else:            # 오른쪽 이동
        next_state = min(n_states - 1, state + 1)  # 상태 4보다 오른쪽으로는 가지 않도록 최대값 제한

    # 보상과 종료 조건 설정
    if next_state == n_states - 1:         # 목표 상태(4)에 도달한 경우
        reward = 1.0                       # 성공 보상 1.0 부여
        done = True                        # 에피소드 종료
    else:
        reward = -0.01                     # 그 외 상태에서는 시간 패널티 -0.01 부여 (빨리 도달 유도)
        done = False                       # 에피소드 계속 진행

    return next_state, reward, done        # 결과 반환

def reset():
    # 에피소드 시작 시 초기 상태를 반환하는 함수
    # 여기서는 항상 상태 0에서 시작
    return 0


# ==============================
# 2. Off-PAC 하이퍼파라미터
# ==============================
np.random.seed(42)    # 난수 시드 고정 (실행결과 재현 가능)

gamma = 0.99          # 할인율
alpha_theta = 0.05    # Actor(정책 파라미터) 학습률
alpha_v = 0.1         # Critic(가치함수) 학습률

n_episodes = 500      # 전체 학습 에피소드 수
max_steps  = 20       # 한 에피소드에서 허용하는 최대 스텝 수

epsilon_behavior = 0.2  # behavior policy의 ε-greedy 탐험 비율
rho_bar = 5.0           # 중요도 비율 ρ 상한 (굉장히 큰 값 방지용)


# ==============================
# 3. 파라미터 초기화
# ==============================
# 타깃 정책 파라미터 theta[s, a]
theta = np.zeros((n_states, n_actions))   # 초기에는 모든 선호도를 0으로 시작

# Critic: 상태가치 V(s)를 나타내는 벡터
v = np.zeros(n_states)                    # 초기에는 모든 상태가치를 0으로 시작


# ==============================
# 4. 정책 관련 함수 정의
# ==============================
def softmax(logits):
    # 주어진 선호도 벡터 logits를 softmax를 이용해 확률 분포로 변환하는 함수
    c = np.max(logits)                      # 수치 안정성을 위해 최대값을 빼줌
    exp = np.exp(logits - c)                # 지수 함수 적용
    return exp / np.sum(exp)                # 전체 합으로 나누어 확률 분포로 변환

def target_policy_probs(state):
    # 타깃 정책 π(a|s; θ) 의 행동 확률을 반환하는 함수
    return softmax(theta[state])            # theta[state]에 softmax 적용

def behavior_policy_probs(state):
    # behavior 정책 μ(a|s) 의 행동 확률을 반환하는 함수
    # 여기서는 타깃 정책의 greedy 행동을 기준으로 ε-greedy 형태로 구성
    pi_probs = target_policy_probs(state)   # 타깃 정책 확률
    greedy_action = np.argmax(pi_probs)     # 가장 확률이 높은 행동 (greedy)
    probs = np.ones(n_actions) * (epsilon_behavior / n_actions)  # 기본적으로 균등 분포에 ε 나누기
    probs[greedy_action] += (1.0 - epsilon_behavior)             # greedy 행동에 1-ε 추가
    return probs

def sample_behavior_action(state):
    # behavior 정책 μ(a|s)를 이용하여 행동을 샘플링하는 함수
    probs = behavior_policy_probs(state)    # μ(a|s) 확률 분포
    action = np.random.choice(n_actions, p=probs)  # 그 확률에 따라 행동 선택
    return action, probs                    # 선택한 행동과 해당 시점의 μ 분포 반환

def grad_log_target_policy(state, action):
    # 타깃 정책에 대한 ∇θ log π(a|s; θ)를 계산하는 함수
    probs = target_policy_probs(state)      # π(a|s; θ)
    grad = -probs.copy()                    # 기본값은 -π(a|s; θ)
    grad[action] += 1.0                     # 선택된 행동 a에 대해서는 +1 더해줌
    # 결과적으로:
    # grad[k] = 1 - π(k|s)  if k == action
    # grad[k] = -π(k|s)     if k != action
    return grad                             # shape: (2,)


# ==============================
# 5. Off-PAC 학습 루프
# ==============================
return_history = []                       # 각 에피소드의 총 Return(G_0)을 기록할 리스트

print("=== 1차원 선형 월드에서의 Off-PAC(Off-Policy Actor-Critic) 학습 시작 ===")

for episode in range(1, n_episodes + 1):
    state = reset()                       # 에피소드 시작 상태
    G0 = 0.0                              # 에피소드 Return(G_0) 추적용
    discount = 1.0                        # γ^t 계수 누적용

    for step_idx in range(max_steps):
        # 1) behavior 정책 μ(a|s)에서 행동 샘플링 (off-policy)
        action, mu_probs = sample_behavior_action(state)
        mu_a = mu_probs[action]           # 실제 선택된 행동 a의 behavior 확률 μ(a|s)

        # 2) 환경에 행동 적용
        next_state, reward, done = step(state, action)

        # 3) Critic 업데이트를 위한 TD 오차 계산
        v_s = v[state]
        v_s_next = v[next_state] if not done else 0.0
        td_target = reward + gamma * v_s_next
        td_error = td_target - v_s         # δ_t = r + γV(s') - V(s)

        # 4) 타깃 정책 π(a|s; θ)의 행동 확률 계산
        pi_probs = target_policy_probs(state)
        pi_a = pi_probs[action]           # π(a|s) 중 실제 취한 행동의 확률

        # 5) 중요도 비율 ρ_t = π(a|s) / μ(a|s)
        rho = pi_a / (mu_a + 1e-8)        # 0 나누기 방지를 위한 작은 상수 추가
        rho = min(rho_bar, rho)           # 너무 큰 값이 되지 않도록 상한 적용

        # 6) Critic 업데이트 (off-policy 가중치 적용)
        #    V(s) ← V(s) + α_v * ρ * δ_t
        v[state] += alpha_v * rho * td_error

        # 7) Actor 업데이트 (Off-PAC의 핵심)
        #    θ ← θ + α_θ * ρ * δ_t * ∇θ log π(a|s; θ)
        grad = grad_log_target_policy(state, action)
        theta[state] += alpha_theta * rho * td_error * grad

        # 8) Return(G_0) 추적 (로그용)
        G0 += discount * reward
        discount *= gamma

        state = next_state                # 다음 상태로 전이

        if done:
            # 목표 상태 도달 시 에피소드 종료
            break

    # 한 에피소드가 끝나면 Return(G_0)을 기록
    return_history.append(G0)

    # 50 에피소드마다 최근 50개 Return 평균을 출력 (학습 진행 확인 용도)
    if episode % 50 == 0:
        avg_ret = np.mean(return_history[-50:])
        print(f"[Episode {episode:4d}] 최근 50 에피소드 평균 Return = {avg_ret:.3f}")

print("\n=== Off-PAC 학습 종료 ===\n")


# ==============================
# 6. 학습된 타깃 정책(상태별 행동 확률) 출력
# ==============================
print("▶ 학습된 타깃 정책 π(a|s; θ) (행: 상태, 열: 행동[←,→])")
for s in range(n_states):
    probs = target_policy_probs(s)        # 상태 s에서의 타깃 정책 확률
    print(f"상태 {s}: {probs}")

# Greedy 정책으로 사람이 보기 쉽게 화살표로 표현 (타깃 정책 기준)
action_symbols = {0: "←", 1: "→"}        # 0은 왼쪽 화살표, 1은 오른쪽 화살표

print("\n▶ Greedy 기준 학습된 타깃 정책(Policy)")
policy_str = ""
for s in range(n_states):
    if s == n_states - 1:                # 목표 상태인 경우
        policy_str += " G "              # Goal 표기
    else:
        best_a = np.argmax(target_policy_probs(s))  # 타깃 정책 기준 greedy 행동
        policy_str += f" {action_symbols[best_a]} "
print("상태 0  1  2  3  4")
print("     " + policy_str)


# ==============================
# 7. 학습된 타깃 정책으로 1회 테스트 실행
# ==============================
print("\n▶ 학습된 타깃 정책으로 1회 에피소드 실행 예시 (Greedy 정책 사용)")

state = reset()                          # 초기 상태로 리셋
trajectory = [state]                     # 방문한 상태들을 기록하기 위한 리스트

for step_idx in range(max_steps):
    probs = target_policy_probs(state)   # 현재 상태에서의 타깃 정책 확률
    action = np.argmax(probs)            # 가장 확률이 높은 행동을 선택 (탐험 없이 greedy)
    next_state, reward, done = step(state, action)  # 환경에 행동 적용

    trajectory.append(next_state)        # 방문한 상태 기록
    state = next_state                   # 다음 상태로 이동

    if done:                             # 목표에 도달하면 종료
        break

print("방문한 상태들:", trajectory)
print("스텝 수:", len(trajectory) - 1)
print("마지막 상태가 목표(4)면 학습 성공!")


=== 1차원 선형 월드에서의 Off-PAC(Off-Policy Actor-Critic) 학습 시작 ===
[Episode   50] 최근 50 에피소드 평균 Return = 0.915
[Episode  100] 최근 50 에피소드 평균 Return = 0.915
[Episode  150] 최근 50 에피소드 평균 Return = 0.930
[Episode  200] 최근 50 에피소드 평균 Return = 0.923
[Episode  250] 최근 50 에피소드 평균 Return = 0.924
[Episode  300] 최근 50 에피소드 평균 Return = 0.922
[Episode  350] 최근 50 에피소드 평균 Return = 0.922
[Episode  400] 최근 50 에피소드 평균 Return = 0.922
[Episode  450] 최근 50 에피소드 평균 Return = 0.926
[Episode  500] 최근 50 에피소드 평균 Return = 0.923

=== Off-PAC 학습 종료 ===

▶ 학습된 타깃 정책 π(a|s; θ) (행: 상태, 열: 행동[←,→])
상태 0: [0.31338939 0.68661061]
상태 1: [0.16443638 0.83556362]
상태 2: [0.12063528 0.87936472]
상태 3: [0.13076979 0.86923021]
상태 4: [0.5 0.5]

▶ Greedy 기준 학습된 타깃 정책(Policy)
상태 0  1  2  3  4
      →  →  →  →  G 

▶ 학습된 타깃 정책으로 1회 에피소드 실행 예시 (Greedy 정책 사용)
방문한 상태들: [0, 1, 2, 3, 4]
스텝 수: 4
마지막 상태가 목표(4)면 학습 성공!


In [23]:
###################################################################
## (2-8) PPO(Proximal Policy Optimization): 신뢰 구간을 사용해 안정적으로 정책을 업데이트
###################################################################
import numpy as np  # 수치 계산용 NumPy 불러오기

# ==============================
# 1. 환경 정의 (1차원 선형 월드)
# ==============================
n_states = 5        # 상태 개수: 0,1,2,3,4 (4가 목표 상태)
n_actions = 2       # 행동 개수: 0=왼쪽, 1=오른쪽

def step(state, action):
    # 주어진 상태 state에서 행동 action을 했을 때
    # 다음 상태(next_state), 보상(reward), 종료 여부(done)를 반환하는 함수

    # 행동이 0이면 왼쪽으로 한 칸 이동, 1이면 오른쪽으로 한 칸 이동
    if action == 0:  # 왼쪽 이동
        next_state = max(0, state - 1)     # 상태 0보다 왼쪽으로는 가지 않도록 최소값 제한
    else:            # 오른쪽 이동
        next_state = min(n_states - 1, state + 1)  # 상태 4보다 오른쪽으로는 가지 않도록 최대값 제한

    # 보상과 종료 조건 설정
    if next_state == n_states - 1:         # 목표 상태(4)에 도달한 경우
        reward = 1.0                       # 성공 보상 1.0 부여
        done = True                        # 에피소드 종료
    else:
        reward = -0.01                     # 그 외 상태에서는 시간 패널티 -0.01 부여 (빨리 도달 유도)
        done = False                       # 에피소드 계속 진행

    return next_state, reward, done        # 결과 반환

def reset():
    # 에피소드 시작 시 초기 상태를 반환하는 함수
    # 여기서는 항상 상태 0에서 시작
    return 0


# ==============================
# 2. PPO 하이퍼파라미터
# ==============================
np.random.seed(42)    # 난수 시드 고정 (실행 결과 재현 가능)

gamma = 0.99          # 할인율
alpha_theta = 0.05    # Actor(정책 파라미터) 학습률
alpha_v = 0.1         # Critic(가치함수) 학습률

n_episodes = 500      # 전체 학습 에피소드 수
max_steps  = 20       # 한 에피소드에서 허용하는 최대 스텝 수

eps_clip = 0.2        # PPO 클리핑 파라미터 ε (r_t를 1±ε 사이로 제한)
ppo_epochs = 1        # 한 에피소드 수집 후 몇 번 반복 업데이트할지 (최소 구현으로 1회)


# ==============================
# 3. 파라미터 초기화
# ==============================
# 정책 파라미터 theta[s, a]
theta = np.zeros((n_states, n_actions))   # 초기에는 모든 선호도를 0으로 시작

# Critic: 상태가치 V(s)를 나타내는 벡터
v = np.zeros(n_states)                    # 초기에는 모든 상태가치를 0으로 시작


# ==============================
# 4. 정책 관련 함수 정의
# ==============================
def softmax(logits):
    # 주어진 선호도 벡터 logits를 softmax를 이용해 확률 분포로 변환하는 함수
    c = np.max(logits)                      # 수치 안정성을 위해 최대값을 빼줌
    exp = np.exp(logits - c)                # 지수 함수 적용
    return exp / np.sum(exp)                # 전체 합으로 나누어 확률 분포로 변환

def policy_probs(state, theta_param=None):
    # 상태 state에서의 행동 확률 π(a|s; θ)를 반환하는 함수
    # theta_param이 None이면 전역 theta를 사용
    if theta_param is None:
        theta_param = theta
    return softmax(theta_param[state])      # theta[state]에 softmax 적용

def sample_action(state):
    # 현재 정책 π(a|s; θ)를 이용하여 행동을 샘플링하는 함수
    probs = policy_probs(state, theta)      # 상태 s에서의 행동 확률
    action = np.random.choice(n_actions, p=probs)  # 확률에 따라 행동 선택
    return action, probs                    # 선택한 행동과 해당 시점의 정책 확률 분포 반환

def grad_log_policy(state, action):
    # ∇θ log π(a|s; θ)를 계산하는 함수
    probs = policy_probs(state, theta)      # π(a|s; θ)
    grad = -probs.copy()                    # 기본값은 -π(a|s; θ)
    grad[action] += 1.0                     # 선택된 행동 a에 대해서는 +1 더해줌
    # 결과적으로:
    # grad[k] = 1 - π(k|s)  if k == action
    # grad[k] = -π(k|s)     if k != action
    return grad                             # shape: (2,)


# ==============================
# 5. PPO 학습 루프
# ==============================
return_history = []                        # 각 에피소드의 총 Return(G_0)을 기록할 리스트

print("=== 1차원 선형 월드에서의 PPO(Proximal Policy Optimization) 학습 시작 ===")

for episode in range(1, n_episodes + 1):
    state = reset()                        # 에피소드 시작 상태
    G0 = 0.0                               # 에피소드 Return(G_0) 추적용
    discount = 1.0                         # γ^t 계수 누적용

    # 1) 한 에피소드 trajectory 수집
    states  = []                           # 상태 시퀀스
    actions = []                           # 행동 시퀀스
    rewards = []                           # 보상 시퀀스
    dones   = []                           # done 플래그 시퀀스
    old_action_probs = []                  # 각 시점에서의 π_old(a_t|s_t) (스칼라)

    for step_idx in range(max_steps):
        # 현재 정책으로 행동 샘플링
        action, probs = sample_action(state)
        # 행동에 해당하는 확률 π_old(a_t|s_t)를 기록
        old_prob_a = probs[action]

        # 환경에 적용
        next_state, reward, done = step(state, action)

        # trajectory에 저장
        states.append(state)
        actions.append(action)
        rewards.append(reward)
        dones.append(done)
        old_action_probs.append(old_prob_a)

        # Return(G_0) 계산용
        G0 += discount * reward
        discount *= gamma

        state = next_state                 # 다음 상태로 전이

        if done:
            # 목표 도달 시 에피소드 종료
            break

    # 2) 수집된 trajectory로부터 Monte Carlo return 및 Advantage 계산
    T = len(states)                        # 에피소드 길이
    returns = np.zeros(T)                  # 각 시점의 G_t
    G = 0.0                                # 뒤에서부터 누적할 Return

    for t in reversed(range(T)):
        # terminal이면 그대로 보상에서 시작, 아니면 할인 Return 누적
        G = rewards[t] + gamma * G
        returns[t] = G

    # Advantage A_t = G_t - V(s_t)
    advantages = np.zeros(T)
    for t in range(T):
        s = states[t]
        advantages[t] = returns[t] - v[s]

    # 3) Critic(V) 업데이트: V(s) ← V(s) + α_v * (G_t - V(s))
    for t in range(T):
        s = states[t]
        v[s] += alpha_v * (returns[t] - v[s])

    # 4) PPO Actor 업데이트 (최소 구현: 한 번의 에포크만 수행)
    for _ in range(ppo_epochs):
        for t in range(T):
            s = states[t]                  # s_t
            a = actions[t]                 # a_t
            old_prob_a = old_action_probs[t]  # π_old(a_t|s_t)
            A_t = advantages[t]            # Advantage A_t

            # 현재 정책에서의 π_new(a_t|s_t)
            new_probs = policy_probs(s, theta)
            new_prob_a = new_probs[a]

            # 확률 비율 r_t = π_new(a_t|s_t) / π_old(a_t|s_t)
            ratio = new_prob_a / (old_prob_a + 1e-8)

            # 클리핑된 비율 r_t^clip = clip(r_t, 1-ε, 1+ε)
            clipped_ratio = np.clip(ratio, 1.0 - eps_clip, 1.0 + eps_clip)

            # PPO의 핵심: L_t = min(r_t * A_t, r_t^clip * A_t)
            # 여기서는 gradient를 단순화해서, 효과적인 ratio를 선택해 사용
            if A_t >= 0:
                # 이득이 양수면 너무 크게 늘어나지 않도록 상한을 적용
                effective_ratio = min(ratio, clipped_ratio)
            else:
                # 이득이 음수면 너무 크게 줄어들지 않도록 하한을 적용
                effective_ratio = max(ratio, clipped_ratio)

            # 정책 gradient: ∇θ L ≈ effective_ratio * A_t * ∇θ log π(a_t|s_t)
            grad = grad_log_policy(s, a)
            theta[s] += alpha_theta * effective_ratio * A_t * grad

    # 한 에피소드가 끝나면 Return(G_0)을 기록
    return_history.append(G0)

    # 50 에피소드마다 최근 50개 Return 평균을 출력
    if episode % 50 == 0:
        avg_ret = np.mean(return_history[-50:])
        print(f"[Episode {episode:4d}] 최근 50 에피소드 평균 Return = {avg_ret:.3f}")

print("\n=== PPO 학습 종료 ===\n")


# ==============================
# 6. 학습된 정책(상태별 행동 확률) 출력
# ==============================
print("▶ 학습된 정책 π(a|s; θ) (행: 상태, 열: 행동[←,→])")
for s in range(n_states):
    probs = policy_probs(s, theta)        # 상태 s에서의 정책 확률
    print(f"상태 {s}: {probs}")

# Greedy 정책으로 사람이 보기 쉽게 화살표로 표현
action_symbols = {0: "←", 1: "→"}        # 0은 왼쪽 화살표, 1은 오른쪽 화살표

print("\n▶ Greedy 기준 학습된 정책(Policy)")
policy_str = ""
for s in range(n_states):
    if s == n_states - 1:                # 목표 상태인 경우
        policy_str += " G "              # Goal 표기
    else:
        best_a = np.argmax(policy_probs(s, theta))  # greedy 행동
        policy_str += f" {action_symbols[best_a]} "
print("상태 0  1  2  3  4")
print("     " + policy_str)


# ==============================
# 7. 학습된 정책으로 1회 테스트 실행
# ==============================
print("\n▶ 학습된 정책으로 1회 에피소드 실행 예시 (Greedy 정책 사용)")

state = reset()                          # 초기 상태로 리셋
trajectory = [state]                     # 방문한 상태들을 기록하기 위한 리스트

for step_idx in range(max_steps):
    probs = policy_probs(state, theta)   # 현재 상태에서의 정책 확률
    action = np.argmax(probs)            # 가장 확률이 높은 행동 선택 (탐험 없이 greedy)
    next_state, reward, done = step(state, action)  # 환경에 행동 적용

    trajectory.append(next_state)        # 방문한 상태 기록
    state = next_state                   # 다음 상태로 이동

    if done:                             # 목표에 도달하면 종료
        break

print("방문한 상태들:", trajectory)
print("스텝 수:", len(trajectory) - 1)
print("마지막 상태가 목표(4)면 학습 성공!")


=== 1차원 선형 월드에서의 PPO(Proximal Policy Optimization) 학습 시작 ===
[Episode   50] 최근 50 에피소드 평균 Return = 0.662
[Episode  100] 최근 50 에피소드 평균 Return = 0.832
[Episode  150] 최근 50 에피소드 평균 Return = 0.856
[Episode  200] 최근 50 에피소드 평균 Return = 0.848
[Episode  250] 최근 50 에피소드 평균 Return = 0.905
[Episode  300] 최근 50 에피소드 평균 Return = 0.903
[Episode  350] 최근 50 에피소드 평균 Return = 0.901
[Episode  400] 최근 50 에피소드 평균 Return = 0.907
[Episode  450] 최근 50 에피소드 평균 Return = 0.912
[Episode  500] 최근 50 에피소드 평균 Return = 0.914

=== PPO 학습 종료 ===

▶ 학습된 정책 π(a|s; θ) (행: 상태, 열: 행동[←,→])
상태 0: [0.25908815 0.74091185]
상태 1: [0.12482231 0.87517769]
상태 2: [0.09811593 0.90188407]
상태 3: [0.18027466 0.81972534]
상태 4: [0.5 0.5]

▶ Greedy 기준 학습된 정책(Policy)
상태 0  1  2  3  4
      →  →  →  →  G 

▶ 학습된 정책으로 1회 에피소드 실행 예시 (Greedy 정책 사용)
방문한 상태들: [0, 1, 2, 3, 4]
스텝 수: 4
마지막 상태가 목표(4)면 학습 성공!


In [24]:
###################################################################
## (2-9) TRPO(Trust Region Policy Optimization): 정책 급변을 방지하는 최적화 기법
###################################################################
import numpy as np  # 수치 계산용 NumPy 불러오기

# ==============================
# 1. 환경 정의 (1차원 선형 월드)
# ==============================
n_states = 5        # 상태 개수: 0,1,2,3,4 (4가 목표 상태)
n_actions = 2       # 행동 개수: 0=왼쪽, 1=오른쪽

def step(state, action):
    # 주어진 상태 state에서 행동 action을 했을 때
    # 다음 상태(next_state), 보상(reward), 종료 여부(done)를 반환하는 함수

    # 행동이 0이면 왼쪽으로 한 칸 이동, 1이면 오른쪽으로 한 칸 이동
    if action == 0:  # 왼쪽 이동
        next_state = max(0, state - 1)                     # 상태 0보다 왼쪽으로는 가지 않도록 제한
    else:            # 오른쪽 이동
        next_state = min(n_states - 1, state + 1)          # 상태 4보다 오른쪽으로는 가지 않도록 제한

    # 보상과 종료 조건 설정
    if next_state == n_states - 1:                         # 목표 상태(4)에 도달한 경우
        reward = 1.0                                       # 성공 보상 1.0 부여
        done = True                                        # 에피소드 종료
    else:
        reward = -0.01                                     # 그 외 상태에서는 시간 패널티 -0.01 부여 (빨리 도달 유도)
        done = False                                       # 에피소드 계속 진행

    return next_state, reward, done                        # 결과 반환

def reset():
    # 에피소드 시작 시 초기 상태를 반환하는 함수
    # 여기서는 항상 상태 0에서 시작
    return 0


# ==============================
# 2. TRPO 하이퍼파라미터
# ==============================
np.random.seed(42)        # 난수 시드 고정 (실행 결과 재현 가능)

gamma = 0.99              # 할인율
alpha_v = 0.1             # Critic(가치함수) 학습률

n_episodes = 500          # 전체 학습 에피소드 수
max_steps  = 20           # 한 에피소드에서 허용하는 최대 스텝 수

max_kl = 0.01             # Trust region의 최대 KL 제한 (대략적인 상한)


# ==============================
# 3. 파라미터 초기화
# ==============================
# 정책 파라미터 theta[s, a]
theta = np.zeros((n_states, n_actions))   # 초기에는 모든 선호도를 0으로 시작

# Critic: 상태가치 V(s)를 나타내는 벡터
v = np.zeros(n_states)                    # 초기에는 모든 상태가치를 0으로 시작


# ==============================
# 4. 정책 관련 함수 정의
# ==============================
def softmax(logits):
    # 주어진 선호도 벡터 logits를 softmax를 이용해 확률 분포로 변환하는 함수
    c = np.max(logits)                    # 수치 안정성을 위해 최대값을 빼줌
    exp = np.exp(logits - c)              # 지수 함수 적용
    return exp / np.sum(exp)              # 전체 합으로 나누어 확률 분포로 변환

def policy_probs(state, theta_param=None):
    # 상태 state에서의 행동 확률 π(a|s; θ)를 반환하는 함수
    # theta_param이 None이면 전역 theta를 사용
    if theta_param is None:
        theta_param = theta               # 기본적으로 전역 theta 사용
    return softmax(theta_param[state])    # theta[state]에 softmax 적용

def sample_action(state):
    # 현재 정책 π(a|s; θ)를 이용하여 행동을 샘플링하는 함수
    probs = policy_probs(state, theta)    # 상태 s에서의 행동 확률
    action = np.random.choice(n_actions, p=probs)  # 확률에 따라 행동 선택
    return action, probs                  # 선택한 행동과 해당 시점의 정책 확률 분포 반환

def grad_log_policy(state, action):
    # ∇θ log π(a|s; θ)를 계산하는 함수
    probs = policy_probs(state, theta)    # π(a|s; θ)
    grad = -probs.copy()                  # 기본값은 -π(a|s; θ)
    grad[action] += 1.0                   # 선택된 행동 a에 대해서는 +1 더해줌
    # 결과적으로:
    # grad[k] = 1 - π(k|s)  if k == action
    # grad[k] = -π(k|s)     if k != action
    return grad                           # shape: (2,)


def fisher_matrix_from_probs(probs):
    # tabular softmax에서 Fisher 정보 행렬 F = E[∇logπ ∇logπ^T]
    # 이론적으로 F = diag(π) - π π^T 로 쓸 수 있음 (2x2 행렬)
    diag = np.diag(probs)                 # diag(π)
    outer = np.outer(probs, probs)        # π π^T
    F = diag - outer                      # Fisher(2x2)
    return F


# ==============================
# 5. TRPO 학습 루프
# ==============================
return_history = []                      # 각 에피소드의 총 Return(G_0)을 기록할 리스트

print("=== 1차원 선형 월드에서의 TRPO(Trust Region Policy Optimization) 학습 시작 ===")

for episode in range(1, n_episodes + 1):
    state = reset()                      # 에피소드 시작 상태
    G0 = 0.0                             # 에피소드 Return(G_0) 추적용
    discount = 1.0                       # γ^t 계수 누적용

    # 1) 한 에피소드 trajectory 수집
    states  = []                         # 상태 시퀀스
    actions = []                         # 행동 시퀀스
    rewards = []                         # 보상 시퀀스
    dones   = []                         # done 플래그 시퀀스

    for step_idx in range(max_steps):
        # 현재 정책으로 행동 샘플링
        action, probs = sample_action(state)

        # 환경에 적용
        next_state, reward, done = step(state, action)

        # trajectory에 저장
        states.append(state)
        actions.append(action)
        rewards.append(reward)
        dones.append(done)

        # Return(G_0) 계산용
        G0 += discount * reward
        discount *= gamma

        state = next_state               # 다음 상태로 전이

        if done:
            # 목표 도달 시 에피소드 종료
            break

    # 2) 수집된 trajectory로부터 Monte Carlo return 및 Advantage 계산
    T = len(states)                      # 에피소드 길이
    returns = np.zeros(T)                # 각 시점의 G_t
    G = 0.0                              # 뒤에서부터 누적할 Return

    for t in reversed(range(T)):
        # terminal이면 그대로 보상에서 시작, 아니면 할인 Return 누적
        G = rewards[t] + gamma * G
        returns[t] = G

    # Advantage A_t = G_t - V(s_t)
    advantages = np.zeros(T)
    for t in range(T):
        s = states[t]
        advantages[t] = returns[t] - v[s]

    # 3) Critic(V) 업데이트: V(s) ← V(s) + α_v * (G_t - V(s))
    for t in range(T):
        s = states[t]
        v[s] += alpha_v * (returns[t] - v[s])

    # 4) TRPO Actor 업데이트
    #    상태별로 policy gradient g_s와 Fisher F_s를 계산하고,
    #    natural gradient step n_s = F_s^{-1} g_s 를 구한 뒤
    #    trust region 조건 (approx KL <= max_kl)에 맞게 스케일링해서 θ 업데이트

    # 4-1) 상태별 policy gradient g_s 초기화 (각 상태마다 2차원 벡터)
    g = np.zeros_like(theta)             # g[s, a]

    for t in range(T):
        s = states[t]                    # s_t
        a = actions[t]                   # a_t
        A_t = advantages[t]              # Advantage A_t

        # ∇θ log π(a_t|s_t; θ)를 계산
        grad = grad_log_policy(s, a)     # shape: (2,)
        # g_s에 A_t * grad를 누적 (에피소드의 합)
        g[s] += A_t * grad

    # 4-2) 상태별 Fisher F_s와 natural step n_s 계산
    nat_step = np.zeros_like(theta)      # natural gradient step n[s, a]
    reg = 1e-3                           # 수치 안정성을 위한 작은 정규화 항

    # 우선 상태별 F_s, n_s를 구하면서 전체 quad form도 계산
    quad = 0.0                           # δθ^T F δθ (KL 근사에 사용)

    for s in range(n_states):
        # 상태 s에서의 현재 정책 확률
        probs_s = policy_probs(s, theta) # π(a|s; θ)
        # Fisher 행렬 F_s = diag(π) - π π^T
        F_s = fisher_matrix_from_probs(probs_s)

        # F_s에 작은 정규화 항을 더해 역행렬 계산 안정화
        F_s_reg = F_s + reg * np.eye(n_actions)

        # 자연 그래디언트: n_s = F_s^{-1} g_s
        try:
            inv_F_s = np.linalg.inv(F_s_reg)
        except np.linalg.LinAlgError:
            # 혹시라도 역행렬 계산이 실패하면 항등행렬 사용 (fallback)
            inv_F_s = np.eye(n_actions)

        n_s = inv_F_s @ g[s]             # 2차원 벡터
        nat_step[s] = n_s

        # KL 근사에 사용될 δθ^T F δθ 항을 누적 (각 상태별로 더함)
        quad += n_s.T @ (F_s @ n_s)

    # 4-3) trust region 조건에 맞게 전체 step 스케일링
    #      작은 스텝이면 quad가 거의 0일 수 있으므로 방어 코드 포함
    if quad > 0.0:
        # KL ≈ 0.5 * quad <= max_kl  이 되도록 scale 결정
        scale = np.sqrt(2.0 * max_kl / quad)
        scale = min(1.0, scale)         # 1을 넘지 않도록 제한 (너무 크게 업데이트 방지)
    else:
        scale = 1.0                     # quad가 0이면 스케일 그대로 1

    # 4-4) 최종 θ 업데이트
    theta += scale * nat_step           # TRPO 스타일 natural gradient + trust region 적용

    # 5) 한 에피소드가 끝나면 Return(G_0)을 기록
    return_history.append(G0)

    # 50 에피소드마다 최근 50개 Return 평균을 출력 (학습 진행 확인 용도)
    if episode % 50 == 0:
        avg_ret = np.mean(return_history[-50:])
        print(f"[Episode {episode:4d}] 최근 50 에피소드 평균 Return = {avg_ret:.3f}")

print("\n=== TRPO 학습 종료 ===\n")


# ==============================
# 6. 학습된 정책(상태별 행동 확률) 출력
# ==============================
print("▶ 학습된 정책 π(a|s; θ) (행: 상태, 열: 행동[←,→])")
for s in range(n_states):
    probs = policy_probs(s, theta)       # 상태 s에서의 정책 확률
    print(f"상태 {s}: {probs}")

# Greedy 정책으로 사람이 보기 쉽게 화살표로 표현
action_symbols = {0: "←", 1: "→"}       # 0은 왼쪽 화살표, 1은 오른쪽 화살표

print("\n▶ Greedy 기준 학습된 정책(Policy)")
policy_str = ""
for s in range(n_states):
    if s == n_states - 1:               # 목표 상태인 경우
        policy_str += " G "             # Goal 표기
    else:
        best_a = np.argmax(policy_probs(s, theta))  # greedy 행동
        policy_str += f" {action_symbols[best_a]} "
print("상태 0  1  2  3  4")
print("     " + policy_str)


# ==============================
# 7. 학습된 정책으로 1회 테스트 실행
# ==============================
print("\n▶ 학습된 정책으로 1회 에피소드 실행 예시 (Greedy 정책 사용)")

state = reset()                         # 초기 상태로 리셋
trajectory = [state]                    # 방문한 상태들을 기록하기 위한 리스트

for step_idx in range(max_steps):
    probs = policy_probs(state, theta)  # 현재 상태에서의 정책 확률
    action = np.argmax(probs)           # 가장 확률이 높은 행동 선택 (탐험 없이 greedy)
    next_state, reward, done = step(state, action)  # 환경에 행동 적용

    trajectory.append(next_state)       # 방문한 상태 기록
    state = next_state                  # 다음 상태로 이동

    if done:                            # 목표에 도달하면 종료
        break

print("방문한 상태들:", trajectory)
print("스텝 수:", len(trajectory) - 1)
print("마지막 상태가 목표(4)면 학습 성공!")


=== 1차원 선형 월드에서의 TRPO(Trust Region Policy Optimization) 학습 시작 ===
[Episode   50] 최근 50 에피소드 평균 Return = 0.823
[Episode  100] 최근 50 에피소드 평균 Return = 0.937
[Episode  150] 최근 50 에피소드 평균 Return = 0.939
[Episode  200] 최근 50 에피소드 평균 Return = 0.939
[Episode  250] 최근 50 에피소드 평균 Return = 0.940
[Episode  300] 최근 50 에피소드 평균 Return = 0.941
[Episode  350] 최근 50 에피소드 평균 Return = 0.939
[Episode  400] 최근 50 에피소드 평균 Return = 0.941
[Episode  450] 최근 50 에피소드 평균 Return = 0.941
[Episode  500] 최근 50 에피소드 평균 Return = 0.941

=== TRPO 학습 종료 ===

▶ 학습된 정책 π(a|s; θ) (행: 상태, 열: 행동[←,→])
상태 0: [0.00282126 0.99717874]
상태 1: [0.00164994 0.99835006]
상태 2: [0.0072049 0.9927951]
상태 3: [0.00456071 0.99543929]
상태 4: [0.5 0.5]

▶ Greedy 기준 학습된 정책(Policy)
상태 0  1  2  3  4
      →  →  →  →  G 

▶ 학습된 정책으로 1회 에피소드 실행 예시 (Greedy 정책 사용)
방문한 상태들: [0, 1, 2, 3, 4]
스텝 수: 4
마지막 상태가 목표(4)면 학습 성공!


In [25]:
###################################################################
## (2-10) DDPG(Deep Deterministic Policy Gradient) : 연속적 행동 공간에서 학습하는 Actor-Critic 모델
###################################################################
import numpy as np  # 수치 계산을 위한 NumPy 불러오기

# ==============================
# 1. 환경 정의 (1차원 선형 월드)
# ==============================
n_states = 5        # 상태 개수: 0,1,2,3,4 (4가 목표 상태)
n_actions = 1       # DDPG에서는 연속 행동 1차원 (스칼라)

def step(state, env_action):
    # 환경에서 사용하는 행동 env_action은 이산 행동 (0=왼쪽, 1=오른쪽)
    # 주어진 상태 state에서 env_action을 했을 때
    # 다음 상태(next_state), 보상(reward), 종료 여부(done)를 반환하는 함수

    # 행동이 0이면 왼쪽으로 한 칸 이동, 1이면 오른쪽으로 한 칸 이동
    if env_action == 0:  # 왼쪽 이동
        next_state = max(0, state - 1)                     # 상태 0보다 왼쪽으로는 가지 않도록 제한
    else:               # 오른쪽 이동
        next_state = min(n_states - 1, state + 1)          # 상태 4보다 오른쪽으로는 가지 않도록 제한

    # 보상과 종료 조건 설정
    if next_state == n_states - 1:                         # 목표 상태(4)에 도달한 경우
        reward = 1.0                                       # 성공 보상 1.0 부여
        done = True                                        # 에피소드 종료
    else:
        reward = -0.01                                     # 그 외 상태에서는 시간 패널티 -0.01 부여 (빨리 도달 유도)
        done = False                                       # 에피소드 계속 진행

    return next_state, reward, done                        # 결과 반환

def reset():
    # 에피소드 시작 시 초기 상태를 반환하는 함수
    # 여기서는 항상 상태 0에서 시작
    return 0


# ==============================
# 2. 난수 시드 및 하이퍼파라미터
# ==============================
np.random.seed(42)        # 난수 시드 고정 (실행 결과 재현 가능)

gamma = 0.99              # 할인율
tau = 0.01                # 타깃 네트워크 soft update 계수

actor_lr = 0.01           # Actor 학습률
critic_lr = 0.02          # Critic 학습률

n_episodes = 500          # 전체 학습 에피소드 수
max_steps  = 20           # 한 에피소드에서 허용하는 최대 스텝 수

buffer_capacity = 10000   # 리플레이 버퍼 최대 크기
batch_size = 32           # 미니배치 크기
start_learning = 100      # 일정 개수 이상 샘플이 쌓인 뒤부터 학습 시작

noise_std_init = 0.3      # 초기 탐험 노이즈 표준편차
noise_std_min = 0.05      # 최소 노이즈 크기
noise_decay = 0.995       # 에피소드마다 노이즈 감소 비율


# ==============================
# 3. 간단한 MLP 구현 (Critic 용)
# ==============================
class MLP:
    # 입력 차원(in_dim) → hidden_dim → 1 출력 구조의 MLP를 정의
    def __init__(self, in_dim, hidden_dim):
        # Xavier 초기화를 사용하여 가중치 초기화
        self.W1 = np.random.randn(in_dim, hidden_dim) / np.sqrt(in_dim)
        self.b1 = np.zeros(hidden_dim)
        self.W2 = np.random.randn(hidden_dim, 1) / np.sqrt(hidden_dim)
        self.b2 = np.zeros(1)

    def forward(self, x):
        # 순전파 계산: x → h → q
        # x: (in_dim,) 벡터
        z1 = x @ self.W1 + self.b1           # 1층 선형결합
        h = np.tanh(z1)                      # 1층 활성함수: tanh
        z2 = h @ self.W2 + self.b2           # 2층 선형결합
        q = z2[0]                            # 출력 스칼라
        cache = (x, z1, h, z2, q)            # 역전파에 필요한 값들을 캐시에 저장
        return q, cache

    def backward(self, dq, cache, lr):
        # Critic의 파라미터에 대한 역전파 및 파라미터 업데이트
        # dq: dL/dq (스칼라, 여기서 L은 손실)
        x, z1, h, z2, q = cache

        dz2 = dq                             # 출력층 선형노드의 gradient
        dW2 = np.outer(h, dz2)               # W2 gradient
        db2 = dz2                            # b2 gradient

        dh = self.W2.flatten() * dz2         # hidden 노드로 전파된 gradient
        dz1 = dh * (1.0 - np.tanh(z1) ** 2)  # tanh 미분

        dW1 = np.outer(x, dz1)               # W1 gradient
        db1 = dz1                            # b1 gradient

        # 파라미터 업데이트 (gradient descent)
        self.W2 -= lr * dW2
        self.b2 -= lr * db2
        self.W1 -= lr * dW1
        self.b1 -= lr * db1

    def backward_input(self, dq, cache):
        # 입력 x에 대한 gradient (∂L/∂x)를 계산하는 함수
        # Actor 업데이트에서 ∂Q/∂a를 얻기 위해 사용
        x, z1, h, z2, q = cache

        dz2 = dq                             # dL/dz2
        dh = self.W2.flatten() * dz2         # dL/dh
        dz1 = dh * (1.0 - np.tanh(z1) ** 2)  # dL/dz1
        dx = dz1 @ self.W1.T                 # dL/dx
        return dx                            # x와 같은 차원의 벡터


# ==============================
# 4. Actor 네트워크 구현
# ==============================
class Actor:
    # 입력은 상태(one-hot), 출력은 연속 행동 a ∈ [-1, 1]
    def __init__(self, state_dim, hidden_dim):
        in_dim = state_dim
        self.W1 = np.random.randn(in_dim, hidden_dim) / np.sqrt(in_dim)
        self.b1 = np.zeros(hidden_dim)
        self.W2 = np.random.randn(hidden_dim, 1) / np.sqrt(hidden_dim)
        self.b2 = np.zeros(1)

    def forward(self, s_vec):
        # 상태 벡터 s_vec (one-hot) → 행동 a 를 출력
        z1 = s_vec @ self.W1 + self.b1              # 1층 선형
        h = np.tanh(z1)                             # tanh 활성화
        z2 = h @ self.W2 + self.b2                  # 2층 선형
        a = np.tanh(z2[0])                          # 출력도 tanh로 [-1,1] 범위로 제한
        cache = (s_vec, z1, h, z2, a)               # 역전파를 위한 캐시
        return a, cache

    def backward(self, da, cache, lr):
        # Actor에 대한 역전파: da = dJ/da (J는 maximize할 목적함수)
        # 실제 구현에서는 L = -J 로 두고 gradient descent 수행
        # 즉, dL/da = -dJ/da 이므로 입력으로 넘어오는 da는 dL/da로 해석
        s_vec, z1, h, z2, a = cache

        # a = tanh(z2) 이므로 da/dz2 = 1 - a^2
        dz2 = da * (1.0 - a ** 2)                  # dL/dz2
        dW2 = np.outer(h, dz2)                     # W2 gradient
        db2 = dz2                                  # b2 gradient

        dh = self.W2.flatten() * dz2               # dL/dh
        dz1 = dh * (1.0 - np.tanh(z1) ** 2)        # tanh 미분
        dW1 = np.outer(s_vec, dz1)                 # W1 gradient
        db1 = dz1                                  # b1 gradient

        # 파라미터 업데이트 (gradient descent)
        self.W2 -= lr * dW2
        self.b2 -= lr * db2
        self.W1 -= lr * dW1
        self.b1 -= lr * db1


# ==============================
# 5. 타깃 네트워크 soft update 함수
# ==============================
def soft_update(target, source, tau):
    # target 파라미터를 source 파라미터 쪽으로 조금씩 이동시키는 함수
    # θ_target ← τ θ + (1-τ) θ_target
    for attr in ["W1", "b1", "W2", "b2"]:
        target_param = getattr(target, attr)
        source_param = getattr(source, attr)
        new_param = tau * source_param + (1.0 - tau) * target_param
        setattr(target, attr, new_param.copy())


# ==============================
# 6. 리플레이 버퍼 구현
# ==============================
class ReplayBuffer:
    def __init__(self, capacity):
        self.capacity = capacity          # 최대 저장 개수
        self.buffer = []                  # (s, a, r, s', done) 튜플 리스트
        self.position = 0                 # 다음에 덮어쓸 위치 인덱스

    def push(self, s, a, r, ns, done):
        # 새로운 transition을 버퍼에 추가
        data = (s, a, r, ns, done)
        if len(self.buffer) < self.capacity:
            self.buffer.append(data)
        else:
            self.buffer[self.position] = data
        self.position = (self.position + 1) % self.capacity

    def sample(self, batch_size):
        # 버퍼에서 임의의 미니배치 샘플링
        idxs = np.random.choice(len(self.buffer), batch_size, replace=False)
        batch = [self.buffer[i] for i in idxs]
        return batch

    def __len__(self):
        # 현재 버퍼에 쌓인 transition 수 반환
        return len(self.buffer)


# ==============================
# 7. DDPG 초기 설정
# ==============================
state_dim = n_states      # 상태를 one-hot으로 표현
actor = Actor(state_dim, hidden_dim=16)           # 기본 Actor
actor_target = Actor(state_dim, hidden_dim=16)    # 타깃 Actor

critic = MLP(in_dim=state_dim + n_actions, hidden_dim=32)          # 기본 Critic
critic_target = MLP(in_dim=state_dim + n_actions, hidden_dim=32)   # 타깃 Critic

# 타깃 네트워크를 초기에는 동일하게 맞춤
soft_update(actor_target, actor, tau=1.0)
soft_update(critic_target, critic, tau=1.0)

buffer = ReplayBuffer(buffer_capacity)  # 리플레이 버퍼 생성
noise_std = noise_std_init             # 탐험 노이즈 초기값

return_history = []                    # 에피소드별 Return(G_0)을 기록할 리스트

print("=== 1차원 선형 월드에서의 DDPG(Deep Deterministic Policy Gradient, NumPy) 학습 시작 ===")

# ==============================
# 8. 학습 루프
# ==============================
for episode in range(1, n_episodes + 1):
    state = reset()                    # 에피소드 시작 상태
    G0 = 0.0                           # 에피소드 Return(G_0)
    discount = 1.0                     # γ^t 계수 누적용

    for step_idx in range(max_steps):
        # 1) 상태를 one-hot 벡터로 변환
        s_vec = np.zeros(state_dim)
        s_vec[state] = 1.0

        # 2) Actor에서 현재 상태에 대한 행동 a = μ(s) 계산
        a_det, _ = actor.forward(s_vec)

        # 3) 탐험을 위한 가우시안 노이즈 추가
        a = a_det + noise_std * np.random.randn()
        # 행동 범위를 [-1, 1]로 클리핑
        a = np.clip(a, -1.0, 1.0)

        # 4) 연속 행동 a를 이산 행동 env_action으로 매핑
        #    a < 0 이면 왼쪽(0), a >= 0 이면 오른쪽(1)
        env_action = 0 if a < 0.0 else 1

        # 5) 환경에 적용하여 다음 상태, 보상, 종료 여부를 얻음
        next_state, reward, done = step(state, env_action)

        # 6) 리플레이 버퍼에 transition 저장
        buffer.push(state, a, reward, next_state, done)

        # 7) 학습 시작 조건이 되면 DDPG 업데이트 수행
        if len(buffer) >= max(start_learning, batch_size):
            # 미니배치 샘플링
            batch = buffer.sample(batch_size)

            for (s_b, a_b, r_b, ns_b, d_b) in batch:
                # 상태 s_b를 one-hot으로 변환
                s_vec_b = np.zeros(state_dim)
                s_vec_b[s_b] = 1.0

                # 다음 상태 ns_b 또한 one-hot으로 변환
                ns_vec_b = np.zeros(state_dim)
                ns_vec_b[ns_b] = 1.0

                # 타깃 Actor로부터 다음 상태에서의 행동 a'
                a_next, _ = actor_target.forward(ns_vec_b)

                # Critic 타깃 네트워크로부터 Q_target(ns, a')
                x_next = np.concatenate([ns_vec_b, np.array([a_next])])
                q_next, _ = critic_target.forward(x_next)

                # TD Target 계산: y = r + γ * (1-done) * q_next
                y = r_b + (0.0 if d_b else gamma * q_next)

                # 현재 Critic으로부터 Q(s, a)를 계산
                x_curr = np.concatenate([s_vec_b, np.array([a_b])])
                q_curr, cache_c = critic.forward(x_curr)

                # Critic 손실 L = 0.5 * (q_curr - y)^2
                # dL/dq_curr = (q_curr - y)
                dq = (q_curr - y)
                critic.backward(dq, cache_c, critic_lr)

                # Actor 업데이트:
                # J ≈ E_s[ Q(s, μ(s)) ] 를 최대화
                # L_actor = -Q(s, μ(s))
                # dL_actor/dQ = -1
                a_policy, cache_a = actor.forward(s_vec_b)
                x_actor = np.concatenate([s_vec_b, np.array([a_policy])])
                q_for_actor, cache_c2 = critic.forward(x_actor)

                # ∂L_actor/∂x = ∂L_actor/∂Q * ∂Q/∂x = -1 * ∂Q/∂x
                dx = critic.backward_input(-1.0, cache_c2)

                # x = [s_vec, a] 이므로 마지막 원소가 행동 a에 대한 gradient
                da = dx[-1]

                # Actor 파라미터 업데이트 (gradient descent on L_actor)
                actor.backward(da, cache_a, actor_lr)

                # 타깃 네트워크 soft update
                soft_update(actor_target, actor, tau)
                soft_update(critic_target, critic, tau)

        # 8) Return(G_0) 계산용 누적
        G0 += discount * reward
        discount *= gamma

        # 9) 상태 업데이트
        state = next_state

        if done:
            # 목표에 도달하면 에피소드 종료
            break

    # 에피소드가 끝난 후, 에피소드 Return 기록
    return_history.append(G0)

    # 탐험 노이즈 감소
    noise_std = max(noise_std_min, noise_std * noise_decay)

    # 50 에피소드마다 최근 50개 Return 평균을 출력
    if episode % 50 == 0:
        avg_ret = np.mean(return_history[-50:])
        print(f"[Episode {episode:4d}] 최근 50 에피소드 평균 Return = {avg_ret:.3f}, noise_std = {noise_std:.3f}")

print("\n=== DDPG 학습 종료 ===\n")


# ==============================
# 9. 학습된 결정론적 정책 μ(s) 출력
# ==============================
print("▶ 학습된 Actor의 결정론적 정책 μ(s) (상태별 연속 행동 값)")
for s in range(n_states):
    s_vec = np.zeros(state_dim)
    s_vec[s] = 1.0
    a_det, _ = actor.forward(s_vec)
    print(f"상태 {s}: 행동 a = {a_det:.4f}")

# Greedy 정책(연속 행동의 부호를 이산 행동으로 변환) 출력
print("\n▶ Greedy 기준 이산 정책(연속 행동 a의 부호를 기준으로 ←/→ 표시)")
action_symbols = {0: "←", 1: "→"}

policy_str = ""
for s in range(n_states):
    s_vec = np.zeros(state_dim)
    s_vec[s] = 1.0
    a_det, _ = actor.forward(s_vec)
    env_action = 0 if a_det < 0.0 else 1
    if s == n_states - 1:
        policy_str += " G "
    else:
        policy_str += f" {action_symbols[env_action]} "

print("상태 0  1  2  3  4")
print("     " + policy_str)


# ==============================
# 10. 학습된 정책으로 1회 테스트 실행
# ==============================
print("\n▶ 학습된 결정론적 정책으로 1회 에피소드 실행 예시")

state = reset()                      # 초기 상태로 리셋
trajectory = [state]                 # 방문한 상태들을 기록하기 위한 리스트

for step_idx in range(max_steps):
    s_vec = np.zeros(state_dim)
    s_vec[state] = 1.0
    # 탐험 없이 결정론적 정책 a = μ(s) 사용
    a_det, _ = actor.forward(s_vec)
    env_action = 0 if a_det < 0.0 else 1

    next_state, reward, done = step(state, env_action)
    trajectory.append(next_state)
    state = next_state

    if done:
        break

print("방문한 상태들:", trajectory)
print("스텝 수:", len(trajectory) - 1)
print("마지막 상태가 목표(4)면 학습 성공!")


=== 1차원 선형 월드에서의 DDPG(Deep Deterministic Policy Gradient, NumPy) 학습 시작 ===
[Episode   50] 최근 50 에피소드 평균 Return = 0.855, noise_std = 0.233
[Episode  100] 최근 50 에피소드 평균 Return = 0.929, noise_std = 0.182
[Episode  150] 최근 50 에피소드 평균 Return = 0.941, noise_std = 0.141
[Episode  200] 최근 50 에피소드 평균 Return = 0.941, noise_std = 0.110
[Episode  250] 최근 50 에피소드 평균 Return = 0.941, noise_std = 0.086
[Episode  300] 최근 50 에피소드 평균 Return = 0.941, noise_std = 0.067
[Episode  350] 최근 50 에피소드 평균 Return = 0.941, noise_std = 0.052
[Episode  400] 최근 50 에피소드 평균 Return = 0.941, noise_std = 0.050
[Episode  450] 최근 50 에피소드 평균 Return = 0.941, noise_std = 0.050
[Episode  500] 최근 50 에피소드 평균 Return = 0.941, noise_std = 0.050

=== DDPG 학습 종료 ===

▶ 학습된 Actor의 결정론적 정책 μ(s) (상태별 연속 행동 값)
상태 0: 행동 a = 0.6133
상태 1: 행동 a = 0.9966
상태 2: 행동 a = 0.9505
상태 3: 행동 a = 0.4647
상태 4: 행동 a = 0.7957

▶ Greedy 기준 이산 정책(연속 행동 a의 부호를 기준으로 ←/→ 표시)
상태 0  1  2  3  4
      →  →  →  →  G 

▶ 학습된 결정론적 정책으로 1회 에피소드 실행 예시
방문한 상태들: [0, 1, 2, 3

In [26]:
###################################################################
## (2-11) TD3(Twin Delayed DDPG) : DDPG의 한계점을 극복하기 위한 개선된 모델
###################################################################
import numpy as np  # 수치 계산을 위한 NumPy 임포트

# ==============================
# 1. 환경 정의 (1차원 선형 월드)
# ==============================
n_states = 5        # 상태 개수: 0,1,2,3,4 (4가 목표 상태)
n_actions = 1       # TD3에서 사용할 연속 행동 차원 (스칼라)

def step(state, env_action):
    # 주어진 상태 state에서 이산 행동 env_action(0=왼쪽, 1=오른쪽)을 수행했을 때
    # 다음 상태(next_state), 보상(reward), 종료 여부(done)를 반환하는 함수

    # env_action이 0이면 왼쪽 이동, 1이면 오른쪽 이동
    if env_action == 0:  # 왼쪽
        next_state = max(0, state - 1)                 # 상태는 0보다 작아지지 않도록 제한
    else:               # 오른쪽
        next_state = min(n_states - 1, state + 1)      # 상태는 4보다 커지지 않도록 제한

    # 목표 상태(4)에 도달하면 보상 1.0, 에피소드 종료
    if next_state == n_states - 1:
        reward = 1.0
        done = True
    else:
        reward = -0.01                                 # 그 외에는 시간 패널티 -0.01
        done = False

    return next_state, reward, done                    # 다음 상태, 보상, 종료 여부 반환

def reset():
    # 에피소드 시작 시 초기 상태를 반환하는 함수
    # 여기서는 항상 상태 0에서 시작
    return 0


# ==============================
# 2. 난수 시드 및 하이퍼파라미터
# ==============================
np.random.seed(42)        # 난수 시드 고정 (실행 결과 재현성 확보)

gamma = 0.99              # 할인율 (미래 보상 가중치)
tau = 0.01                # 타깃 네트워크 soft update 계수

actor_lr = 0.01           # Actor 학습률
critic_lr = 0.02          # Critic 학습률

n_episodes = 500          # 전체 학습 에피소드 수
max_steps  = 20           # 한 에피소드 최대 스텝 수

buffer_capacity = 10000   # 리플레이 버퍼 최대 크기
batch_size = 32           # 미니배치 크기
start_learning = 100      # 이 이상 샘플이 쌓인 후부터 학습 시작

noise_std_init = 0.3      # 행동 탐험용 가우시안 노이즈 초기 표준편차
noise_std_min = 0.05      # 행동 탐험 노이즈 최소값
noise_decay = 0.995       # 에피소드마다 노이즈 감소 비율

target_noise_std = 0.2    # 타깃 정책에 더하는 노이즈 표준편차 (Target Policy Smoothing)
target_noise_clip = 0.5   # 타깃 정책 노이즈 클리핑 한계

policy_delay = 2          # Critic 여러 번 업데이트 후 1번 Actor 업데이트 (Delayed Policy)


# ==============================
# 3. 간단한 MLP 구현 (Critic 용)
# ==============================
class MLP:
    # 입력(in_dim) → hidden_dim → 1 출력 구조의 단순 MLP
    def __init__(self, in_dim, hidden_dim):
        # Xavier 초기화를 사용하여 가중치 초기화
        self.W1 = np.random.randn(in_dim, hidden_dim) / np.sqrt(in_dim)
        self.b1 = np.zeros(hidden_dim)
        self.W2 = np.random.randn(hidden_dim, 1) / np.sqrt(hidden_dim)
        self.b2 = np.zeros(1)

    def forward(self, x):
        # 순전파 계산: x → h → q
        # x: (in_dim,) 형태의 벡터
        z1 = x @ self.W1 + self.b1           # 1층 선형결합
        h = np.tanh(z1)                      # 1층 활성화 함수: tanh
        z2 = h @ self.W2 + self.b2           # 2층 선형결합
        q = z2[0]                            # 스칼라 출력
        cache = (x, z1, h, z2, q)            # 역전파용 캐시
        return q, cache

    def backward(self, dq, cache, lr):
        # Critic 파라미터에 대한 역전파 및 업데이트
        # dq: dL/dq (스칼라, L은 손실)
        x, z1, h, z2, q = cache

        dz2 = dq                             # dL/dz2
        dW2 = np.outer(h, dz2)               # W2에 대한 gradient
        db2 = dz2                            # b2에 대한 gradient

        dh = self.W2.flatten() * dz2         # hidden 층으로 전파된 gradient
        dz1 = dh * (1.0 - np.tanh(z1) ** 2)  # tanh 미분
        dW1 = np.outer(x, dz1)               # W1 gradient
        db1 = dz1                            # b1 gradient

        # 파라미터 업데이트 (gradient descent)
        self.W2 -= lr * dW2
        self.b2 -= lr * db2
        self.W1 -= lr * dW1
        self.b1 -= lr * db1

    def backward_input(self, dq, cache):
        # 입력 x에 대한 gradient ∂L/∂x 계산
        # Actor 업데이트 시 ∂Q/∂a 를 얻기 위해 사용
        x, z1, h, z2, q = cache

        dz2 = dq                             # dL/dz2
        dh = self.W2.flatten() * dz2         # dL/dh
        dz1 = dh * (1.0 - np.tanh(z1) ** 2)  # dL/dz1
        dx = dz1 @ self.W1.T                 # dL/dx
        return dx


# ==============================
# 4. Actor 네트워크 구현
# ==============================
class Actor:
    # 상태 one-hot 벡터 → 연속 행동 a ∈ [-1, 1] 출력
    def __init__(self, state_dim, hidden_dim):
        in_dim = state_dim
        self.W1 = np.random.randn(in_dim, hidden_dim) / np.sqrt(in_dim)
        self.b1 = np.zeros(hidden_dim)
        self.W2 = np.random.randn(hidden_dim, 1) / np.sqrt(hidden_dim)
        self.b2 = np.zeros(1)

    def forward(self, s_vec):
        # 상태 벡터 s_vec (one-hot) → 행동 a를 출력
        z1 = s_vec @ self.W1 + self.b1               # 1층 선형결합
        h = np.tanh(z1)                              # tanh 활성화
        z2 = h @ self.W2 + self.b2                   # 2층 선형결합
        a = np.tanh(z2[0])                           # 출력 tanh로 [-1,1]로 제한
        cache = (s_vec, z1, h, z2, a)                # 역전파용 캐시
        return a, cache

    def backward(self, da, cache, lr):
        # Actor에 대한 역전파: da = dL/da (L은 최소화할 손실)
        # 여기서는 L_actor = -Q(s, μ(s)) 이므로 dL/da = -∂Q/∂a 와 동일 개념
        s_vec, z1, h, z2, a = cache

        # a = tanh(z2) 이므로 da/dz2 = 1 - a^2
        dz2 = da * (1.0 - a ** 2)                   # dL/dz2
        dW2 = np.outer(h, dz2)                      # W2 gradient
        db2 = dz2                                   # b2 gradient

        dh = self.W2.flatten() * dz2                # dL/dh
        dz1 = dh * (1.0 - np.tanh(z1) ** 2)         # tanh 미분
        dW1 = np.outer(s_vec, dz1)                  # W1 gradient
        db1 = dz1                                   # b1 gradient

        # 파라미터 업데이트
        self.W2 -= lr * dW2
        self.b2 -= lr * db2
        self.W1 -= lr * dW1
        self.b1 -= lr * db1


# ==============================
# 5. 타깃 네트워크 soft update 함수
# ==============================
def soft_update(target, source, tau):
    # θ_target ← τ θ_source + (1 - τ) θ_target 형태의 soft update
    for attr in ["W1", "b1", "W2", "b2"]:
        target_param = getattr(target, attr)
        source_param = getattr(source, attr)
        new_param = tau * source_param + (1.0 - tau) * target_param
        setattr(target, attr, new_param.copy())


# ==============================
# 6. 리플레이 버퍼 구현
# ==============================
class ReplayBuffer:
    def __init__(self, capacity):
        self.capacity = capacity          # 최대 저장 용량
        self.buffer = []                  # (s, a, r, s', done) 저장 리스트
        self.position = 0                 # 덮어쓸 위치 인덱스

    def push(self, s, a, r, ns, done):
        # 새로운 transition을 버퍼에 추가
        data = (s, a, r, ns, done)
        if len(self.buffer) < self.capacity:
            self.buffer.append(data)
        else:
            self.buffer[self.position] = data
        self.position = (self.position + 1) % self.capacity

    def sample(self, batch_size):
        # 버퍼에서 랜덤하게 미니배치를 추출
        idxs = np.random.choice(len(self.buffer), batch_size, replace=False)
        batch = [self.buffer[i] for i in idxs]
        return batch

    def __len__(self):
        # 현재 버퍼에 들어있는 transition 수
        return len(self.buffer)


# ==============================
# 7. TD3 네트워크 초기 설정
# ==============================
state_dim = n_states

# Actor 및 타깃 Actor 생성
actor = Actor(state_dim, hidden_dim=16)
actor_target = Actor(state_dim, hidden_dim=16)

# Critic1, Critic2 및 각 타깃 네트워크 생성 (Twin Critic 구조)
critic1 = MLP(in_dim=state_dim + n_actions, hidden_dim=32)
critic2 = MLP(in_dim=state_dim + n_actions, hidden_dim=32)
critic1_target = MLP(in_dim=state_dim + n_actions, hidden_dim=32)
critic2_target = MLP(in_dim=state_dim + n_actions, hidden_dim=32)

# 타깃 네트워크를 초기에는 원본 네트워크와 동일하게 설정
soft_update(actor_target, actor, tau=1.0)
soft_update(critic1_target, critic1, tau=1.0)
soft_update(critic2_target, critic2, tau=1.0)

buffer = ReplayBuffer(buffer_capacity)    # 리플레이 버퍼 생성
noise_std = noise_std_init               # 행동 탐험 노이즈 초기값

return_history = []                      # 에피소드별 Return(G_0)을 기록할 리스트
update_step = 0                          # Critic 업데이트 횟수(Actor 딜레이 업데이트에 사용)

print("=== 1차원 선형 월드에서의 TD3(Twin Delayed DDPG, NumPy) 학습 시작 ===")

# ==============================
# 8. 학습 루프
# ==============================
for episode in range(1, n_episodes + 1):
    state = reset()           # 에피소드 시작 상태
    G0 = 0.0                  # 에피소드 Return(G_0)
    discount = 1.0            # γ^t 계산용

    for step_idx in range(max_steps):
        # 1) 상태를 one-hot 벡터로 변환
        s_vec = np.zeros(state_dim)
        s_vec[state] = 1.0

        # 2) Actor로부터 결정론적 행동 a_det = μ(s) 계산
        a_det, _ = actor.forward(s_vec)

        # 3) 탐험을 위해 가우시안 노이즈를 추가
        a = a_det + noise_std * np.random.randn()
        a = np.clip(a, -1.0, 1.0)         # 행동 범위를 [-1, 1]로 제한

        # 4) 연속 행동 a를 이산 행동 env_action으로 매핑
        env_action = 0 if a < 0.0 else 1  # a < 0 → 왼쪽, a >= 0 → 오른쪽

        # 5) 환경에 적용하여 다음 상태, 보상, 종료 여부 얻기
        next_state, reward, done = step(state, env_action)

        # 6) 리플레이 버퍼에 (s, a, r, s', done) 저장
        buffer.push(state, a, reward, next_state, done)

        # 7) 충분히 샘플이 쌓이면 TD3 학습 수행
        if len(buffer) >= max(start_learning, batch_size):
            # 미니배치 샘플링
            batch = buffer.sample(batch_size)
            update_step += 1  # Critic 업데이트 횟수 증가

            for (s_b, a_b, r_b, ns_b, d_b) in batch:
                # 상태 s_b, 다음 상태 ns_b를 one-hot 벡터로 변환
                s_vec_b = np.zeros(state_dim)
                s_vec_b[s_b] = 1.0
                ns_vec_b = np.zeros(state_dim)
                ns_vec_b[ns_b] = 1.0

                # 타깃 Actor로부터 다음 상태에서의 행동 a'_det 계산
                a_next_det, _ = actor_target.forward(ns_vec_b)

                # Target Policy Smoothing: a'_det에 노이즈 추가 후 클리핑
                noise = target_noise_std * np.random.randn()
                noise = np.clip(noise, -target_noise_clip, target_noise_clip)
                a_next = a_next_det + noise
                a_next = np.clip(a_next, -1.0, 1.0)

                # Critic 타깃 네트워크들로부터 Q1', Q2' 계산
                x_next = np.concatenate([ns_vec_b, np.array([a_next])])
                q1_next, _ = critic1_target.forward(x_next)
                q2_next, _ = critic2_target.forward(x_next)

                # Twin Critic의 최소값 사용 (TD3의 핵심)
                q_next_min = min(q1_next, q2_next)

                # TD Target 계산: y = r + γ * (1-done) * q_next_min
                y = r_b + (0.0 if d_b else gamma * q_next_min)

                # 현재 Critic1, Critic2로부터 Q1(s,a), Q2(s,a) 계산
                x_curr = np.concatenate([s_vec_b, np.array([a_b])])
                q1_curr, cache_c1 = critic1.forward(x_curr)
                q2_curr, cache_c2 = critic2.forward(x_curr)

                # Critic 손실에 대한 gradient: dL/dQi = (Qi - y)
                dq1 = (q1_curr - y)
                dq2 = (q2_curr - y)

                # Critic1, Critic2 각각 역전파 및 업데이트
                critic1.backward(dq1, cache_c1, critic_lr)
                critic2.backward(dq2, cache_c2, critic_lr)

                # Critic 타깃 네트워크 soft update
                soft_update(critic1_target, critic1, tau)
                soft_update(critic2_target, critic2, tau)

            # Delayed Policy Update: 일정 횟수마다 Actor 및 Actor 타깃 업데이트
            if update_step % policy_delay == 0:
                for (s_b, a_b, r_b, ns_b, d_b) in batch:
                    # Actor 업데이트는 상태 분포에 대해 J ≈ E[Q1(s, μ(s))]를 최대화
                    s_vec_b = np.zeros(state_dim)
                    s_vec_b[s_b] = 1.0

                    # 현재 Actor 정책으로부터 행동 a_policy = μ(s) 계산
                    a_policy, cache_a = actor.forward(s_vec_b)

                    # Critic1으로 Q1(s, μ(s)) 계산
                    x_actor = np.concatenate([s_vec_b, np.array([a_policy])])
                    q_for_actor, cache_c_for_actor = critic1.forward(x_actor)

                    # Actor 손실 L_actor = -Q1(s, μ(s))
                    # dL/dQ = -1 이므로 backward_input에 -1 전달
                    dx = critic1.backward_input(-1.0, cache_c_for_actor)

                    # x = [s_vec, a] 이므로 마지막 원소가 a에 대한 gradient
                    da = dx[-1]

                    # Actor 파라미터 업데이트 (gradient descent on L_actor)
                    actor.backward(da, cache_a, actor_lr)

                # Actor 타깃 네트워크도 soft update
                soft_update(actor_target, actor, tau)

        # 8) 에피소드 Return(G_0) 누적
        G0 += discount * reward
        discount *= gamma

        # 9) 상태 업데이트
        state = next_state

        if done:
            # 목표 상태에 도달하면 에피소드 종료
            break

    # 에피소드 종료 후 Return 기록
    return_history.append(G0)

    # 탐험 노이즈 감소
    noise_std = max(noise_std_min, noise_std * noise_decay)

    # 50 에피소드마다 최근 50개 Return 평균과 현재 노이즈 출력
    if episode % 50 == 0:
        avg_ret = np.mean(return_history[-50:])
        print(f"[Episode {episode:4d}] 최근 50 에피소드 평균 Return = {avg_ret:.3f}, noise_std = {noise_std:.3f}")

print("\n=== TD3 학습 종료 ===\n")


# ==============================
# 9. 학습된 결정론적 정책 μ(s) 출력
# ==============================
print("▶ 학습된 Actor의 결정론적 정책 μ(s) (상태별 연속 행동 값)")
for s in range(n_states):
    s_vec = np.zeros(state_dim)
    s_vec[s] = 1.0
    a_det, _ = actor.forward(s_vec)
    print(f"상태 {s}: 행동 a = {a_det:.4f}")

# Greedy 기준 이산 정책(행동 부호를 기준으로 ←/→) 출력
print("\n▶ Greedy 기준 이산 정책(연속 행동 a의 부호를 기준으로 ←/→ 표시)")
action_symbols = {0: "←", 1: "→"}

policy_str = ""
for s in range(n_states):
    s_vec = np.zeros(state_dim)
    s_vec[s] = 1.0
    a_det, _ = actor.forward(s_vec)
    env_action = 0 if a_det < 0.0 else 1
    if s == n_states - 1:
        policy_str += " G "
    else:
        policy_str += f" {action_symbols[env_action]} "

print("상태 0  1  2  3  4")
print("     " + policy_str)


# ==============================
# 10. 학습된 정책으로 1회 테스트 실행
# ==============================
print("\n▶ 학습된 결정론적 정책으로 1회 에피소드 실행 예시")

state = reset()          # 초기 상태로 리셋
trajectory = [state]     # 방문한 상태를 기록하기 위한 리스트

for step_idx in range(max_steps):
    s_vec = np.zeros(state_dim)
    s_vec[state] = 1.0

    # 탐험 없이 결정론적 정책 a_det = μ(s) 사용
    a_det, _ = actor.forward(s_vec)
    env_action = 0 if a_det < 0.0 else 1

    next_state, reward, done = step(state, env_action)
    trajectory.append(next_state)
    state = next_state

    if done:
        break

print("방문한 상태들:", trajectory)
print("스텝 수:", len(trajectory) - 1)
print("마지막 상태가 목표(4)면 학습 성공!")


=== 1차원 선형 월드에서의 TD3(Twin Delayed DDPG, NumPy) 학습 시작 ===
[Episode   50] 최근 50 에피소드 평균 Return = 0.861, noise_std = 0.233
[Episode  100] 최근 50 에피소드 평균 Return = 0.941, noise_std = 0.182
[Episode  150] 최근 50 에피소드 평균 Return = 0.940, noise_std = 0.141
[Episode  200] 최근 50 에피소드 평균 Return = 0.941, noise_std = 0.110
[Episode  250] 최근 50 에피소드 평균 Return = 0.941, noise_std = 0.086
[Episode  300] 최근 50 에피소드 평균 Return = 0.941, noise_std = 0.067
[Episode  350] 최근 50 에피소드 평균 Return = 0.941, noise_std = 0.052
[Episode  400] 최근 50 에피소드 평균 Return = 0.941, noise_std = 0.050
[Episode  450] 최근 50 에피소드 평균 Return = 0.941, noise_std = 0.050
[Episode  500] 최근 50 에피소드 평균 Return = 0.941, noise_std = 0.050

=== TD3 학습 종료 ===

▶ 학습된 Actor의 결정론적 정책 μ(s) (상태별 연속 행동 값)
상태 0: 행동 a = 0.7023
상태 1: 행동 a = 0.9915
상태 2: 행동 a = 0.9786
상태 3: 행동 a = 0.4965
상태 4: 행동 a = 0.8042

▶ Greedy 기준 이산 정책(연속 행동 a의 부호를 기준으로 ←/→ 표시)
상태 0  1  2  3  4
      →  →  →  →  G 

▶ 학습된 결정론적 정책으로 1회 에피소드 실행 예시
방문한 상태들: [0, 1, 2, 3, 4]
스텝 수: 4
마지막 상태

In [31]:
###################################################################
## (2-12) SAC(Soft Actor-Critic): 탐색과 활용의 균형을 유지하도록 설계된 정책 학습 모델
###################################################################
import numpy as np  # 수치 계산용 NumPy

# ==============================
# 1. 환경 정의 (1차원 선형 월드)
# ==============================
n_states = 5        # 상태: 0,1,2,3,4 (4가 목표 상태)
n_actions = 1       # 연속 행동 차원 (스칼라)

def step(state, env_action):
    # 주어진 상태 state에서 이산 행동 env_action(0=왼쪽, 1=오른쪽)을 수행
    # 다음 상태(next_state), 보상(reward), 종료 여부(done)를 반환

    # 행동에 따라 상태 이동
    if env_action == 0:           # 왼쪽
        next_state = max(0, state - 1)
    else:                         # 오른쪽
        next_state = min(n_states - 1, state + 1)

    # 목표 상태(4)에 도달하면 종료 + 보상 1.0
    if next_state == n_states - 1:
        reward = 1.0
        done = True
    else:
        reward = -0.01            # 그 외에는 시간 패널티 -0.01
        done = False

    return next_state, reward, done

def reset():
    # 에피소드 시작 시 초기 상태 반환
    return 0                      # 항상 상태 0에서 시작


# ==============================
# 2. 난수 시드 및 하이퍼파라미터
# ==============================
np.random.seed(42)                # 시드 고정 (실행 결과 재현성 확보)

gamma = 0.99                      # 할인율
alpha = 0.2                       # 온도 파라미터(Entropy 가중치)

tau = 0.01                        # 타깃 네트워크 soft update 계수

actor_lr = 0.01                   # Actor 학습률
critic_lr = 0.02                  # Critic 학습률

n_episodes = 500                  # 학습 에피소드 수
max_steps  = 20                   # 에피소드당 최대 스텝 수

buffer_capacity = 10000           # 리플레이 버퍼 용량
batch_size = 32                   # 미니배치 크기
start_learning = 100              # 버퍼에 이 이상 쌓이면 학습 시작


# ==============================
# 3. MLP (Critic용) 정의
# ==============================
class MLP:
    # 입력(in_dim) → hidden_dim → 1 출력 스칼라 Q값
    def __init__(self, in_dim, hidden_dim):
        # Xavier 초기화
        self.W1 = np.random.randn(in_dim, hidden_dim) / np.sqrt(in_dim)
        self.b1 = np.zeros(hidden_dim)
        self.W2 = np.random.randn(hidden_dim, 1) / np.sqrt(hidden_dim)
        self.b2 = np.zeros(1)

    def forward(self, x):
        # 순전파: x → h → q
        # x: (in_dim,) 벡터
        z1 = x @ self.W1 + self.b1         # 1층 선형 결합
        h = np.tanh(z1)                    # tanh 활성화
        z2 = h @ self.W2 + self.b2         # 2층 선형 결합
        q = z2[0]                          # 스칼라 출력
        cache = (x, z1, h, z2, q)          # 역전파용 캐시
        return q, cache

    def backward(self, dq, cache, lr):
        # Critic 파라미터에 대한 역전파
        # dq: dL/dq (스칼라)
        x, z1, h, z2, q = cache

        dz2 = dq                           # dL/dz2
        dW2 = np.outer(h, dz2)             # W2 gradient
        db2 = dz2                          # b2 gradient

        dh = self.W2.flatten() * dz2       # hidden gradient
        dz1 = dh * (1.0 - np.tanh(z1) ** 2)  # tanh 미분
        dW1 = np.outer(x, dz1)             # W1 gradient
        db1 = dz1                          # b1 gradient

        # 파라미터 업데이트 (gradient descent)
        self.W2 -= lr * dW2
        self.b2 -= lr * db2
        self.W1 -= lr * dW1
        self.b1 -= lr * db1

    def backward_input(self, dq, cache):
        # 입력 x에 대한 gradient ∂L/∂x 계산
        # Actor 업데이트 시 ∂Q/∂a 얻기 위해 사용
        x, z1, h, z2, q = cache

        dz2 = dq                           # dL/dz2
        dh = self.W2.flatten() * dz2       # dL/dh
        dz1 = dh * (1.0 - np.tanh(z1) ** 2)
        dx = dz1 @ self.W1.T               # dL/dx
        return dx


# ==============================
# 4. Actor(확률 정책, Gaussian) 정의
# ==============================
class ActorSAC:
    # 상태 one-hot → (μ(s), logσ(s)) → z = μ + σ ε → a = clip(z, -1, 1)
    def __init__(self, state_dim, hidden_dim):
        in_dim = state_dim
        self.W1 = np.random.randn(in_dim, hidden_dim) / np.sqrt(in_dim)
        self.b1 = np.zeros(hidden_dim)

        # μ 헤드 파라미터
        self.W2_mu = np.random.randn(hidden_dim, 1) / np.sqrt(hidden_dim)
        self.b2_mu = np.zeros(1)

        # logσ 헤드 파라미터
        self.W2_logstd = np.random.randn(hidden_dim, 1) / np.sqrt(hidden_dim)
        self.b2_logstd = np.zeros(1)

    def forward(self, s_vec):
        # 상태 벡터 s_vec(one-hot) → (μ, logσ, σ, ε, z, a, logπ) 반환
        z1 = s_vec @ self.W1 + self.b1                # 1층 선형 결합
        h = np.tanh(z1)                               # tanh 활성화

        z2_mu = h @ self.W2_mu + self.b2_mu           # μ 출력
        mu = z2_mu[0]

        z2_logstd = h @ self.W2_logstd + self.b2_logstd  # logσ 출력
        log_std = z2_logstd[0]

        # log_std를 적당한 범위로 클리핑 (너무 큰 분산 방지)
        log_std = np.clip(log_std, -2.0, 1.0)
        std = np.exp(log_std)                         # σ = exp(logσ)

        # ε ~ N(0,1) 샘플링 (reparameterization trick)
        eps = np.random.randn()
        z = mu + std * eps                            # 샘플 z
        a = np.clip(z, -1.0, 1.0)                     # [-1,1] 범위로 클리핑

        # Gaussian 로그 확률 밀도 log π(z|μ,σ)
        # 상수항은 무시해도 gradient에는 영향 없음
        var = std ** 2
        log_pi = -0.5 * ((z - mu) ** 2 / var + 2.0 * log_std)

        cache = (s_vec, z1, h, z2_mu, z2_logstd, mu, log_std, std, eps, z, a, log_pi)
        return mu, log_std, std, eps, z, a, log_pi, cache

    def backward(self, dmu, dlogstd, cache, lr):
        # Actor 파라미터에 대한 역전파
        # 입력: dmu = dL/dμ, dlogstd = dL/d(logσ)
        s_vec, z1, h, z2_mu, z2_logstd, mu, log_std, std, eps, z, a, log_pi = cache

        # μ 헤드 역전파 (μ = z2_mu)
        dz2_mu = dmu                                  # dL/dz2_mu
        dW2_mu = np.outer(h, dz2_mu)                  # W2_mu gradient
        db2_mu = dz2_mu                               # b2_mu gradient
        dh_mu = self.W2_mu.flatten() * dz2_mu         # hidden으로의 gradient

        # logσ 헤드 역전파 (logσ = z2_logstd)
        dz2_logstd = dlogstd                          # dL/dz2_logstd
        dW2_logstd = np.outer(h, dz2_logstd)          # W2_logstd gradient
        db2_logstd = dz2_logstd                       # b2_logstd gradient
        dh_logstd = self.W2_logstd.flatten() * dz2_logstd

        # 두 헤드에서 온 gradient를 합산
        dh = dh_mu + dh_logstd                        # dL/dh

        # 1층으로 역전파 (h = tanh(z1))
        dz1 = dh * (1.0 - np.tanh(z1) ** 2)           # dL/dz1
        dW1 = np.outer(s_vec, dz1)                    # W1 gradient
        db1 = dz1                                     # b1 gradient

        # 파라미터 업데이트
        self.W2_mu      -= lr * dW2_mu
        self.b2_mu      -= lr * db2_mu
        self.W2_logstd  -= lr * dW2_logstd
        self.b2_logstd  -= lr * db2_logstd
        self.W1         -= lr * dW1
        self.b1         -= lr * db1


# ==============================
# 5. 타깃 네트워크 soft update
# ==============================
def soft_update(target, source, tau):
    # θ_target ← τ θ_source + (1 - τ) θ_target
    for attr in ["W1", "b1", "W2", "b2"]:
        target_param = getattr(target, attr)
        source_param = getattr(source, attr)
        new_param = tau * source_param + (1.0 - tau) * target_param
        setattr(target, attr, new_param.copy())


# ==============================
# 6. 리플레이 버퍼
# ==============================
class ReplayBuffer:
    def __init__(self, capacity):
        self.capacity = capacity
        self.buffer = []
        self.position = 0

    def push(self, s, a, r, ns, done):
        # (s, a, r, ns, done) transition 저장
        data = (s, a, r, ns, done)
        if len(self.buffer) < self.capacity:
            self.buffer.append(data)
        else:
            self.buffer[self.position] = data
        self.position = (self.position + 1) % self.capacity

    def sample(self, batch_size):
        # 랜덤 미니배치 추출
        idxs = np.random.choice(len(self.buffer), batch_size, replace=False)
        batch = [self.buffer[i] for i in idxs]
        return batch

    def __len__(self):
        return len(self.buffer)


# ==============================
# 7. SAC 네트워크 초기화
# ==============================
state_dim = n_states

# Actor (확률 정책)
actor = ActorSAC(state_dim, hidden_dim=16)

# Twin Q-Critic 및 타깃 Q-Critic
critic1 = MLP(in_dim=state_dim + n_actions, hidden_dim=32)
critic2 = MLP(in_dim=state_dim + n_actions, hidden_dim=32)
critic1_target = MLP(in_dim=state_dim + n_actions, hidden_dim=32)
critic2_target = MLP(in_dim=state_dim + n_actions, hidden_dim=32)

# 타깃 네트워크를 초기에는 원본과 동일하게 설정
soft_update(critic1_target, critic1, tau=1.0)
soft_update(critic2_target, critic2, tau=1.0)

buffer = ReplayBuffer(buffer_capacity)    # 리플레이 버퍼 생성

return_history = []                       # 에피소드별 Return 기록

print("=== 1차원 선형 월드에서의 SAC(Soft Actor-Critic, NumPy) 학습 시작 ===")

# ==============================
# 8. 학습 루프
# ==============================
for episode in range(1, n_episodes + 1):
    state = reset()           # 초기 상태
    G0 = 0.0                  # 에피소드 Return(G_0)
    discount = 1.0            # γ^t 계산용

    for step_idx in range(max_steps):
        # 1) 상태를 one-hot 벡터로 변환
        s_vec = np.zeros(state_dim)
        s_vec[state] = 1.0

        # 2) 현재 정책에서 행동 샘플링 (탐험을 위한 stochastic 정책)
        mu, log_std, std, eps, z, a, log_pi, cache_actor_step = actor.forward(s_vec)

        # 3) 연속 행동 a의 부호를 기준으로 이산 행동 env_action 선택
        env_action = 0 if a < 0.0 else 1

        # 4) 환경 한 스텝 진행
        next_state, reward, done = step(state, env_action)

        # 5) 리플레이 버퍼에 transition 저장
        #    여기서는 연속 행동 a(클리핑된 값)를 저장
        buffer.push(state, a, reward, next_state, done)

        # 6) 버퍼에 충분히 쌓이면 SAC 업데이트 수행
        if len(buffer) >= max(start_learning, batch_size):
            # 미니배치 샘플링
            batch = buffer.sample(batch_size)

            # ----------------------
            # 6-1. Critic(Q1, Q2) 업데이트
            # ----------------------
            for (s_b, a_b, r_b, ns_b, d_b) in batch:
                # 상태 및 다음 상태를 one-hot 벡터로 변환
                s_vec_b = np.zeros(state_dim)
                s_vec_b[s_b] = 1.0
                ns_vec_b = np.zeros(state_dim)
                ns_vec_b[ns_b] = 1.0

                # 다음 상태에서의 정책 샘플 a' 및 log π(a'|s')
                mu_next, log_std_next, std_next, eps_next, z_next, a_next, log_pi_next, cache_actor_next = actor.forward(ns_vec_b)

                # 타깃 Q1', Q2' 계산
                x_next = np.concatenate([ns_vec_b, np.array([a_next])])
                q1_next, _ = critic1_target.forward(x_next)
                q2_next, _ = critic2_target.forward(x_next)
                q_next_min = min(q1_next, q2_next)

                # SAC 타깃: y = r + γ(1-done) * (q_next_min - α logπ_next)
                target = r_b + (0.0 if d_b else gamma * (q_next_min - alpha * log_pi_next))

                # 현재 Q1, Q2 계산
                x_curr = np.concatenate([s_vec_b, np.array([a_b])])
                q1_curr, cache_c1 = critic1.forward(x_curr)
                q2_curr, cache_c2 = critic2.forward(x_curr)

                # MSE 손실의 gradient: dL/dQi = (Qi - target)
                dq1 = (q1_curr - target)
                dq2 = (q2_curr - target)

                # Critic1, Critic2 업데이트
                critic1.backward(dq1, cache_c1, critic_lr)
                critic2.backward(dq2, cache_c2, critic_lr)

                # 타깃 Q 네트워크 soft update
                soft_update(critic1_target, critic1, tau)
                soft_update(critic2_target, critic2, tau)

            # ----------------------
            # 6-2. Actor(정책) 업데이트
            # ----------------------
            for (s_b, a_b, r_b, ns_b, d_b) in batch:
                # 상태를 one-hot 벡터로 변환
                s_vec_b = np.zeros(state_dim)
                s_vec_b[s_b] = 1.0

                # 현재 정책에서 행동 샘플 a_policy 및 log π(a_policy|s)
                mu_b, log_std_b, std_b, eps_b, z_b, a_b_samp, log_pi_b, cache_actor_b = actor.forward(s_vec_b)

                # Q1, Q2에서 Qmin(s, a_policy) 계산
                x_actor = np.concatenate([s_vec_b, np.array([a_b_samp])])
                q1_val, cache_q1_for_actor = critic1.forward(x_actor)
                q2_val, cache_q2_for_actor = critic2.forward(x_actor)

                if q1_val <= q2_val:
                    q_min = q1_val
                    cache_q_min = cache_q1_for_actor
                else:
                    q_min = q2_val
                    cache_q_min = cache_q2_for_actor

                # Actor 손실: L_actor = α logπ - Qmin
                # Qmin 항에 대한 gradient: dL/dQmin = -1
                dx = critic1.backward_input(-1.0, cache_q_min) if q1_val <= q2_val else critic2.backward_input(-1.0, cache_q_min)
                da_from_Q = dx[-1]   # 연속 행동 a에 대한 gradient (Q 항에서 기여)

                # logπ(a|s)에 대한 analytic gradient (Gaussian)
                # logπ = -0.5 * ((z-μ)^2 / σ^2 + 2logσ + const)
                # ∂logπ/∂μ = (z - μ) / σ^2
                # ∂logπ/∂logσ = -1 + ((z - μ)/σ)^2
                diff = z_b - mu_b
                grad_logpi_mu = diff / (std_b ** 2)
                grad_logpi_logstd = -1.0 + (diff / std_b) ** 2

                # z = μ + σ ε, 여기서 ε는 eps_b
                # ∂z/∂μ = 1, ∂z/∂logσ = σ * ε
                dz_dmu = 1.0
                dz_dlogstd = std_b * eps_b

                # Q 항에서 오는 μ, logσ에 대한 gradient
                dmu_Q = da_from_Q * dz_dmu
                dlogstd_Q = da_from_Q * dz_dlogstd

                # 엔트로피(α logπ) 항에서 오는 gradient
                dmu_ent = alpha * grad_logpi_mu
                dlogstd_ent = alpha * grad_logpi_logstd

                # 최종 dL/dμ, dL/dlogσ
                dmu_total = dmu_Q + dmu_ent
                dlogstd_total = dlogstd_Q + dlogstd_ent

                # Actor 파라미터 업데이트
                actor.backward(dmu_total, dlogstd_total, cache_actor_b, actor_lr)

        # 7) 에피소드 Return(G_0) 누적 (할인 보상)
        G0 += discount * reward
        discount *= gamma

        # 8) 상태 업데이트
        state = next_state

        if done:
            # 목표 상태 도달 시 에피소드 종료
            break

    # 에피소드 종료 후 Return 기록
    return_history.append(G0)

    # 50 에피소드마다 최근 50개 Return 평균 출력
    if episode % 50 == 0:
        avg_ret = np.mean(return_history[-50:])
        print(f"[Episode {episode:4d}] 최근 50 에피소드 평균 Return = {avg_ret:.3f}")

print("\n=== SAC 학습 종료 ===\n")

# ==============================
# 9. 학습된 정책(평균 μ 기준) 출력
# ==============================
print("▶ 학습된 Actor의 평균 정책 μ(s) (상태별 연속 행동 평균 값)")
for s in range(n_states):
    s_vec = np.zeros(state_dim)
    s_vec[s] = 1.0
    mu_s, log_std_s, std_s, eps_s, z_s, a_s, log_pi_s, cache_s = actor.forward(s_vec)
    print(f"상태 {s}: μ = {mu_s:.4f}")

# Greedy 기준 이산 정책 (μ의 부호를 기준으로 ←/→)
print("\n▶ Greedy 기준 이산 정책(μ(s)의 부호 기준 ←/→)")
action_symbols = {0: "←", 1: "→"}

policy_str = ""
for s in range(n_states):
    s_vec = np.zeros(state_dim)
    s_vec[s] = 1.0
    mu_s, log_std_s, std_s, eps_s, z_s, a_s, log_pi_s, cache_s = actor.forward(s_vec)
    env_action = 0 if mu_s < 0.0 else 1
    if s == n_states - 1:
        policy_str += " G "
    else:
        policy_str += f" {action_symbols[env_action]} "

print("상태 0  1  2  3  4")
print("     " + policy_str)

# ==============================
# 10. 학습된 정책으로 1회 테스트 실행
# ==============================
print("\n▶ 학습된 평균 정책(μ)을 이용한 1회 에피소드 실행 예시")

state = reset()
trajectory = [state]

for step_idx in range(max_steps):
    s_vec = np.zeros(state_dim)
    s_vec[state] = 1.0

    mu_s, log_std_s, std_s, eps_s, z_s, a_s, log_pi_s, cache_s = actor.forward(s_vec)
    env_action = 0 if mu_s < 0.0 else 1

    next_state, reward, done = step(state, env_action)
    trajectory.append(next_state)
    state = next_state

    if done:
        break

print("방문한 상태들:", trajectory)
print("스텝 수:", len(trajectory) - 1)
print("마지막 상태가 목표(4)면 학습 성공!")


=== 1차원 선형 월드에서의 SAC(Soft Actor-Critic, NumPy) 학습 시작 ===
[Episode   50] 최근 50 에피소드 평균 Return = 0.891
[Episode  100] 최근 50 에피소드 평균 Return = 0.941
[Episode  150] 최근 50 에피소드 평균 Return = 0.941
[Episode  200] 최근 50 에피소드 평균 Return = 0.941
[Episode  250] 최근 50 에피소드 평균 Return = 0.941
[Episode  300] 최근 50 에피소드 평균 Return = 0.941
[Episode  350] 최근 50 에피소드 평균 Return = 0.941
[Episode  400] 최근 50 에피소드 평균 Return = 0.941
[Episode  450] 최근 50 에피소드 평균 Return = 0.941
[Episode  500] 최근 50 에피소드 평균 Return = 0.941

=== SAC 학습 종료 ===

▶ 학습된 Actor의 평균 정책 μ(s) (상태별 연속 행동 평균 값)
상태 0: μ = 918.1782
상태 1: μ = 918.2812
상태 2: μ = 918.2171
상태 3: μ = 918.2215
상태 4: μ = 916.9561

▶ Greedy 기준 이산 정책(μ(s)의 부호 기준 ←/→)
상태 0  1  2  3  4
      →  →  →  →  G 

▶ 학습된 평균 정책(μ)을 이용한 1회 에피소드 실행 예시
방문한 상태들: [0, 1, 2, 3, 4]
스텝 수: 4
마지막 상태가 목표(4)면 학습 성공!


In [32]:
###################################################################
## (2-13) BCBC(Behavioral Cloning) : 데이터를 기반으로 정책을 모방하는 방식
###################################################################
import numpy as np  # 수치 계산용 NumPy

# ==============================
# 1. 환경 정의 (1차원 선형 월드)
# ==============================
n_states = 5        # 상태: 0,1,2,3,4 (4가 목표 상태)
n_actions = 2       # 이산 행동: 0=왼쪽, 1=오른쪽

def step(state, action):
    # 주어진 상태 state에서 action(0=왼쪽, 1=오른쪽)을 수행
    # 다음 상태(next_state), 보상(reward), 종료 여부(done)를 반환

    # 행동에 따른 상태 이동
    if action == 0:   # 왼쪽
        next_state = max(0, state - 1)
    else:             # 오른쪽
        next_state = min(n_states - 1, state + 1)

    # 목표 상태(4)에 도달하면 보상 1.0, 종료
    if next_state == n_states - 1:
        reward = 1.0
        done = True
    else:
        reward = -0.01   # 그 외에는 시간 패널티
        done = False

    return next_state, reward, done

def reset():
    # 에피소드 시작 시 상태 초기화
    return 0            # 항상 0에서 시작


# ==============================
# 2. 난수 시드 및 Behavioral Cloning 설정
# ==============================
np.random.seed(42)          # 시드 고정 (결과 재현성 확보)

n_demo_episodes = 50        # 전문가 데모 에피소드 수
max_steps       = 20        # 에피소드당 최대 스텝 수

lr = 0.1                    # BC(Behavioral Cloning) 학습률
n_epochs = 2000             # 학습 반복(epoch) 수


# ==============================
# 3. 전문가 정책 정의 (Expert Policy)
# ==============================
def expert_policy(state):
    # 전문가 정책: 목표 상태(4)에 도달할 때까지 항상 오른쪽(1)으로 이동
    if state < n_states - 1:
        return 1           # 오른쪽
    else:
        return 0           # 목표 상태에서는 의미 없음 (실제로는 사용 안 됨)


# ==============================
# 4. 전문가 Demonstration 데이터 수집
# ==============================
demo_states = []   # 전문가가 방문한 상태들
demo_actions = []  # 전문가가 선택한 행동들

for ep in range(n_demo_episodes):
    # 에피소드마다 초기화
    state = reset()

    for step_idx in range(max_steps):
        # 전문가 정책으로 행동 선택
        action = expert_policy(state)

        # 상태, 행동을 데모 데이터에 기록
        demo_states.append(state)
        demo_actions.append(action)

        # 환경 한 스텝 진행
        next_state, reward, done = step(state, action)
        state = next_state

        if done:
            # 목표 상태에 도달하면 에피소드 종료
            break

# 리스트를 NumPy 배열로 변환
demo_states = np.array(demo_states)   # (N,)
demo_actions = np.array(demo_actions) # (N,)

# 데모 데이터 크기 확인
N = demo_states.shape[0]


# ==============================
# 5. 입력(X)과 레이블(y) 구성
# ==============================
# 입력 X: 상태를 one-hot 벡터로 표현 (형태: (N, n_states))
X = np.zeros((N, n_states))
for i, s in enumerate(demo_states):
    X[i, s] = 1.0

# 레이블 y: 행동(0 또는 1)
y = demo_actions.copy()   # (N,)


# ==============================
# 6. 정책 모델 정의 (Softmax Regression)
# ==============================
# π(a|s; θ)를 softmax(W^T s + b)로 표현
# W: (n_states, n_actions), b: (n_actions,)
W = np.random.randn(n_states, n_actions) * 0.01   # 작은 랜덤 값으로 초기화
b = np.zeros(n_actions)                           # 편향은 0으로 초기화

def softmax(logits):
    # 입력 logits: (N, n_actions)
    # 출력: 각 행에 대해 softmax 적용 (확률 분포)
    max_logits = np.max(logits, axis=1, keepdims=True)   # overflow 방지용
    exp_logits = np.exp(logits - max_logits)
    probs = exp_logits / np.sum(exp_logits, axis=1, keepdims=True)
    return probs

def forward(X):
    # 순전파: X → logits → softmax 확률
    logits = X @ W + b      # (N, n_actions)
    probs = softmax(logits) # (N, n_actions)
    return logits, probs


# ==============================
# 7. Behavioral Cloning 학습 루프 (지도학습)
# ==============================
print("=== 1차원 선형 월드에서의 Behavioral Cloning(BC, NumPy) 학습 시작 ===")

for epoch in range(1, n_epochs + 1):
    # 1) 순전파: 현재 정책으로 확률 계산
    logits, probs = forward(X)   # probs: (N, n_actions)

    # 2) 교차 엔트로피 손실 계산
    #    L = - (1/N) * sum_i log π(a_i | s_i)
    #    정답 인덱스에 해당하는 확률을 모아서 log 취함
    correct_log_probs = -np.log(probs[np.arange(N), y] + 1e-12)
    loss = np.mean(correct_log_probs)

    # 3) 역전파: gradient 계산
    #    softmax + cross-entropy의 gradient:
    #    dL/dlogits = (probs - y_onehot) / N
    grad_logits = probs.copy()
    grad_logits[np.arange(N), y] -= 1.0
    grad_logits /= N

    # 4) 파라미터 W, b에 대한 gradient 계산
    #    dL/dW = X^T @ grad_logits
    #    dL/db = row-wise sum(grad_logits)
    dW = X.T @ grad_logits              # (n_states, n_actions)
    db = np.sum(grad_logits, axis=0)    # (n_actions,)

    # 5) 파라미터 업데이트 (Gradient Descent)
    W -= lr * dW
    b -= lr * db

    # 6) 학습 과정 모니터링 (간단히 200 epoch마다 출력)
    if epoch % 200 == 0:
        # 현재 정책으로 전문가 데이터에서의 정확도 계산
        preds = np.argmax(probs, axis=1)
        acc = np.mean(preds == y)
        print(f"[Epoch {epoch:4d}] Loss = {loss:.4f}, Training Accuracy = {acc:.3f}")

print("\n=== Behavioral Cloning 학습 종료 ===\n")


# ==============================
# 8. 학습된 정책 π(a|s; θ) 확인
# ==============================
print("▶ 학습된 정책 π(a|s; θ) (행: 상태, 열: 행동[←,→] 확률)")

for s in range(n_states):
    # 상태 s를 one-hot 벡터로 변환
    s_vec = np.zeros((1, n_states))
    s_vec[0, s] = 1.0

    # 정책 확률 계산
    _, probs_s = forward(s_vec)   # (1, n_actions)
    p_left  = probs_s[0, 0]
    p_right = probs_s[0, 1]

    print(f"상태 {s}: [{p_left:.4f}, {p_right:.4f}]")

# Greedy 정책(가장 확률이 높은 행동) 출력
print("\n▶ Greedy 기준 학습된 정책(Policy)")
action_symbols = {0: "←", 1: "→"}

policy_str = ""
for s in range(n_states):
    if s == n_states - 1:
        # 목표 상태는 G 로 표시
        policy_str += " G "
    else:
        s_vec = np.zeros((1, n_states))
        s_vec[0, s] = 1.0
        _, probs_s = forward(s_vec)
        a_greedy = int(np.argmax(probs_s[0]))   # 확률 최대 행동 선택
        policy_str += f" {action_symbols[a_greedy]} "

print("상태 0  1  2  3  4")
print("     " + policy_str)


# ==============================
# 9. 학습된 정책으로 1회 에피소드 실행 (Greedy)
# ==============================
print("\n▶ 학습된 정책(BC, Greedy)으로 1회 에피소드 실행 예시")

state = reset()
trajectory = [state]

for step_idx in range(max_steps):
    # 상태를 one-hot으로 변환
    s_vec = np.zeros((1, n_states))
    s_vec[0, state] = 1.0

    # 정책 확률 계산 후 Greedy 행동 선택
    _, probs_s = forward(s_vec)
    a_greedy = int(np.argmax(probs_s[0]))

    # 환경에 적용
    next_state, reward, done = step(state, a_greedy)
    trajectory.append(next_state)
    state = next_state

    if done:
        break

print("방문한 상태들:", trajectory)
print("스텝 수:", len(trajectory) - 1)
print("마지막 상태가 목표(4)면 학습 성공!")


=== 1차원 선형 월드에서의 Behavioral Cloning(BC, NumPy) 학습 시작 ===
[Epoch  200] Loss = 0.0209, Training Accuracy = 1.000
[Epoch  400] Loss = 0.0103, Training Accuracy = 1.000
[Epoch  600] Loss = 0.0068, Training Accuracy = 1.000
[Epoch  800] Loss = 0.0051, Training Accuracy = 1.000
[Epoch 1000] Loss = 0.0041, Training Accuracy = 1.000
[Epoch 1200] Loss = 0.0034, Training Accuracy = 1.000
[Epoch 1400] Loss = 0.0029, Training Accuracy = 1.000
[Epoch 1600] Loss = 0.0025, Training Accuracy = 1.000
[Epoch 1800] Loss = 0.0022, Training Accuracy = 1.000
[Epoch 2000] Loss = 0.0020, Training Accuracy = 1.000

=== Behavioral Cloning 학습 종료 ===

▶ 학습된 정책 π(a|s; θ) (행: 상태, 열: 행동[←,→] 확률)
상태 0: [0.0020, 0.9980]
상태 1: [0.0020, 0.9980]
상태 2: [0.0020, 0.9980]
상태 3: [0.0020, 0.9980]
상태 4: [0.0069, 0.9931]

▶ Greedy 기준 학습된 정책(Policy)
상태 0  1  2  3  4
      →  →  →  →  G 

▶ 학습된 정책(BC, Greedy)으로 1회 에피소드 실행 예시
방문한 상태들: [0, 1, 2, 3, 4]
스텝 수: 4
마지막 상태가 목표(4)면 학습 성공!


In [33]:
###################################################################
## (2-14) DDPGfD(DDPG from Demonstrations) : 전문가의 시범을 사용해 DDPG 성능을 개선
###################################################################
import numpy as np  # 수치 계산을 위한 NumPy

# ==============================
# 1. 환경 정의 (1차원 선형 월드)
# ==============================
n_states = 5        # 상태: 0, 1, 2, 3, 4 (4가 목표 상태)
# DDPG 내부에서는 연속 행동 a ∈ [-1, 1] 을 쓰고,
# 실제 환경에는 a>0 이면 오른쪽(1), a<=0 이면 왼쪽(0) 으로 이산 행동으로 변환한다.

def step(state, env_action):
    # 이산 행동 env_action(0=왼쪽, 1=오른쪽)을 받아 다음 상태, 보상, 종료 여부를 반환
    if env_action == 0:
        next_state = max(0, state - 1)
    else:
        next_state = min(n_states - 1, state + 1)

    if next_state == n_states - 1:
        # 목표 상태(4)에 도달하면 보상 1.0, 종료
        reward = 1.0
        done = True
    else:
        # 그 외에는 시간 패널티 -0.01
        reward = -0.01
        done = False

    return next_state, reward, done

def reset():
    # 에피소드 시작 시 항상 상태 0에서 시작
    return 0


# ==============================
# 2. 난수 시드 및 하이퍼파라미터
# ==============================
np.random.seed(42)          # 결과 재현을 위한 시드 고정

gamma = 0.99                # 할인율
tau   = 0.01                # 타깃 네트워크 소프트 업데이트 계수

actor_lr  = 0.01            # Actor 학습률
critic_lr = 0.05            # Critic 학습률

n_episodes = 500            # 전체 학습 에피소드 수
max_steps  = 20             # 한 에피소드 최대 스텝 수

buffer_capacity = 10000     # 리플레이 버퍼 최대 크기
batch_size      = 32        # 미니배치 크기

# 행동 탐험용 가우시안 노이즈 (연속 행동 a에 추가)
noise_std_init = 0.3        # 초기 노이즈 표준편차
noise_std_min  = 0.05       # 최소 노이즈 표준편차
noise_decay    = 0.995      # 에피소드마다 노이즈 감소 비율

# DDPGfD 용 전문가 데모 수
n_demo_episodes = 30        # 전문가 데모 에피소드 수 (버퍼 프리필용)


# ==============================
# 3. 전문가 정책 정의 (Expert Policy)
# ==============================
def expert_policy(state):
    # 전문가 정책: 목표에 도달할 때까지 항상 오른쪽으로 이동
    # 연속 행동 a ∈ [-1, 1] 을 출력한다고 가정하면, +1.0 을 사용
    return 1.0   # 항상 오른쪽


# ==============================
# 4. 상태를 one-hot 벡터로 변환하는 함수
# ==============================
def state_to_one_hot(state_idx):
    # 상태 인덱스를 길이 n_states인 one-hot 벡터로 변환
    vec = np.zeros(n_states)
    vec[state_idx] = 1.0
    return vec


# ==============================
# 5. Actor / Critic 네트워크 (선형 모델, NumPy)
# ==============================
# Actor: μ(s) = tanh( s_onehot^T * Wa + ba )
#  - Wa: (n_states,)  -> 상태별 weight
#  - ba: 스칼라        -> bias
# Critic: Q(s,a) = s_onehot^T * Wc_s + a * Wc_a + bc
#  - Wc_s: (n_states,) -> 상태 weight
#  - Wc_a: 스칼라      -> 행동 weight
#  - bc:   스칼라      -> bias

# Actor 파라미터 (메인)
Wa = np.random.randn(n_states) * 0.01
ba = 0.0

# Critic 파라미터 (메인)
Wc_s = np.random.randn(n_states) * 0.01
Wc_a = 0.0
bc   = 0.0

# 타깃 네트워크 파라미터 (Actor / Critic)
Wa_tgt = Wa.copy()
ba_tgt = ba
Wc_s_tgt = Wc_s.copy()
Wc_a_tgt = Wc_a
bc_tgt   = bc

def actor_forward(state_idx, use_target=False):
    # 상태 인덱스를 받아 Actor 네트워크를 통해 연속 행동 μ(s)를 계산
    s = state_to_one_hot(state_idx)  # one-hot 상태 벡터
    if use_target:
        # 타깃 Actor 사용
        u = np.dot(s, Wa_tgt) + ba_tgt
    else:
        # 메인 Actor 사용
        u = np.dot(s, Wa) + ba
    # 출력에 tanh를 적용해 [-1, 1] 범위로 제한
    a = np.tanh(u)
    return a

def critic_forward(state_idx, action, use_target=False):
    # 상태 인덱스와 연속 행동 a를 받아 Q(s,a)를 계산
    s = state_to_one_hot(state_idx)
    if use_target:
        q = np.dot(s, Wc_s_tgt) + Wc_a_tgt * action + bc_tgt
    else:
        q = np.dot(s, Wc_s) + Wc_a * action + bc
    return q


# ==============================
# 6. 리플레이 버퍼 구현 (Demonstration 포함)
# ==============================
# 버퍼에는 (s, a, r, s_next, done, is_demo) 를 저장
replay_s      = []
replay_a      = []
replay_r      = []
replay_s_next = []
replay_done   = []
replay_is_demo = []   # 데모 여부 플래그 (True/False)

def add_transition(s, a, r, s_next, done, is_demo):
    # 새 transition을 리플레이 버퍼에 추가
    if len(replay_s) >= buffer_capacity:
        # 버퍼가 꽉 찬 경우 FIFO 방식으로 가장 오래된 transition 삭제
        replay_s.pop(0)
        replay_a.pop(0)
        replay_r.pop(0)
        replay_s_next.pop(0)
        replay_done.pop(0)
        replay_is_demo.pop(0)

    replay_s.append(s)
    replay_a.append(a)
    replay_r.append(r)
    replay_s_next.append(s_next)
    replay_done.append(done)
    replay_is_demo.append(is_demo)

def sample_minibatch(batch_size):
    # 리플레이 버퍼에서 무작위로 미니배치 샘플링
    size = len(replay_s)
    indices = np.random.choice(size, size=batch_size, replace=False)

    batch = {
        "s":      np.array([replay_s[i] for i in indices], dtype=np.int64),
        "a":      np.array([replay_a[i] for i in indices], dtype=np.float32),
        "r":      np.array([replay_r[i] for i in indices], dtype=np.float32),
        "s_next": np.array([replay_s_next[i] for i in indices], dtype=np.int64),
        "done":   np.array([replay_done[i] for i in indices], dtype=np.bool_),
        "is_demo":np.array([replay_is_demo[i] for i in indices], dtype=np.bool_),
    }
    return batch


# ==============================
# 7. DDPGfD용 전문가 Demonstration 프리필
# ==============================
print("=== 전문가 Demonstration 수집 및 리플레이 버퍼 프리필 시작 ===")

for ep in range(n_demo_episodes):
    state = reset()
    for step_idx in range(max_steps):
        # 전문가 정책으로 연속 행동 생성
        a_cont = expert_policy(state)  # 여기서는 항상 +1.0

        # 환경에는 이산 행동으로 전달 (a>0 → 오른쪽=1)
        env_action = 1 if a_cont > 0.0 else 0
        next_state, reward, done = step(state, env_action)

        # 데모 transition을 버퍼에 추가 (is_demo=True)
        add_transition(state, a_cont, reward, next_state, done, is_demo=True)

        state = next_state
        if done:
            break

print(f"전문가 Demonstration으로 프리필된 transition 수: {len(replay_s)}")
print("=== Demonstration 프리필 종료 ===\n")


# ==============================
# 8. DDPGfD 학습 루프
# ==============================
noise_std = noise_std_init           # 탐험용 노이즈 표준편차
returns_history = []                 # 에피소드별 Return 기록

print("=== 1차원 선형 월드에서의 DDPGfD(DDPG from Demonstrations, NumPy) 학습 시작 ===")

for episode in range(1, n_episodes + 1):
    state = reset()
    episode_return = 0.0

    for step_idx in range(max_steps):
        # 1) 현재 상태에서 Actor의 결정론적 행동 μ(s) 계산
        mu = actor_forward(state, use_target=False)

        # 2) 탐험을 위해 가우시안 노이즈 추가
        noise = np.random.randn() * noise_std
        a_cont = mu + noise

        # 3) 연속 행동을 [-1, 1]로 클리핑
        a_cont = np.clip(a_cont, -1.0, 1.0)

        # 4) 환경에 전달할 이산 행동으로 변환
        env_action = 1 if a_cont > 0.0 else 0

        # 5) 환경에서 한 스텝 진행
        next_state, reward, done = step(state, env_action)

        # 6) transition을 리플레이 버퍼에 추가 (is_demo=False)
        add_transition(state, a_cont, reward, next_state, done, is_demo=False)

        # 7) 리플레이 버퍼 기반으로 파라미터 업데이트
        if len(replay_s) >= batch_size:
            # 미니배치 샘플링
            batch = sample_minibatch(batch_size)

            s_batch      = batch["s"]
            a_batch      = batch["a"]
            r_batch      = batch["r"]
            s_next_batch = batch["s_next"]
            done_batch   = batch["done"]

            # --------------------------
            # 7-1) Critic 업데이트
            # --------------------------
            # 타깃 Actor로 next_action 계산
            a_next_batch = np.array([actor_forward(s_next_batch[i], use_target=True)
                                     for i in range(batch_size)], dtype=np.float32)

            # 타깃 Critic으로 target Q 계산
            q_next_batch = np.array([critic_forward(s_next_batch[i], a_next_batch[i], use_target=True)
                                     for i in range(batch_size)], dtype=np.float32)

            # TD target: y = r + γ (1-done) * Q_tgt(s', μ_tgt(s'))
            td_target = r_batch + gamma * (1.0 - done_batch.astype(np.float32)) * q_next_batch

            # 현재 Critic Q값
            q_batch = np.array([critic_forward(s_batch[i], a_batch[i], use_target=False)
                                for i in range(batch_size)], dtype=np.float32)

            # Critic 손실: MSE = mean( (q - y)^2 )
            td_error = q_batch - td_target
            critic_loss = np.mean(td_error ** 2)

            # Critic 파라미터에 대한 gradient 계산
            # dL/dQ = 2 * (Q - y) / N
            dL_dQ = 2.0 * td_error / batch_size  # shape: (batch_size,)

            # 상태 one-hot 벡터 묶기
            S_onehot = np.stack([state_to_one_hot(int(s_batch[i])) for i in range(batch_size)], axis=0)

            # dL/dWc_s = sum_i dL/dQ_i * s_i
            grad_Wc_s = S_onehot.T @ dL_dQ      # shape: (n_states,)

            # dL/dWc_a = sum_i dL/dQ_i * a_i
            grad_Wc_a = np.sum(dL_dQ * a_batch)

            # dL/dbc = sum_i dL/dQ_i
            grad_bc = np.sum(dL_dQ)

            # 파라미터 업데이트 (Gradient Descent)
            Wc_s -= critic_lr * grad_Wc_s
            Wc_a -= critic_lr * grad_Wc_a
            bc   -= critic_lr * grad_bc

            # --------------------------
            # 7-2) Actor 업데이트
            # --------------------------
            # Actor는 Q(s, μ(s))를 최대화하도록 업데이트
            # 여기서는 정책 gradient를 analytic하게 계산
            mu_batch = np.array([actor_forward(int(s_batch[i]), use_target=False)
                                 for i in range(batch_size)], dtype=np.float32)

            # Actor loss = - mean Q(s, μ(s))
            # Q(s, μ(s)) = s^T Wc_s + Wc_a * μ(s) + bc
            Q_mu_batch = np.sum(S_onehot * Wc_s, axis=1) + Wc_a * mu_batch + bc
            actor_loss = -np.mean(Q_mu_batch)

            # dL_actor/dμ = - Wc_a / N (스칼라)
            dL_dmu = -Wc_a / batch_size

            # μ(s) = tanh(u), u = s^T Wa + ba
            # dμ/du = 1 - tanh(u)^2 = 1 - μ^2
            dmu_du = (1.0 - mu_batch ** 2)  # shape: (batch_size,)

            # dL/du = dL/dμ * dμ/du
            dL_du = dL_dmu * dmu_du         # shape: (batch_size,)

            # dL/dWa = sum_i dL/du_i * s_i
            grad_Wa = S_onehot.T @ dL_du    # shape: (n_states,)

            # dL/dba = sum_i dL/du_i
            grad_ba = np.sum(dL_du)

            # Actor 파라미터 업데이트
            Wa -= actor_lr * grad_Wa
            ba -= actor_lr * grad_ba

            # --------------------------
            # 7-3) 타깃 네트워크 소프트 업데이트
            # --------------------------
            Wa_tgt   = (1.0 - tau) * Wa_tgt   + tau * Wa
            ba_tgt   = (1.0 - tau) * ba_tgt   + tau * ba
            Wc_s_tgt = (1.0 - tau) * Wc_s_tgt + tau * Wc_s
            Wc_a_tgt = (1.0 - tau) * Wc_a_tgt + tau * Wc_a
            bc_tgt   = (1.0 - tau) * bc_tgt   + tau * bc

        # 8) Return 누적 및 상태 갱신
        episode_return += reward
        state = next_state

        if done:
            break

    # 노이즈 표준편차 감소 (탐험 → 이용 전환)
    noise_std = max(noise_std_min, noise_std * noise_decay)

    # 에피소드 Return 기록
    returns_history.append(episode_return)

    # 50 에피소드마다 최근 50개 평균 Return 및 노이즈 출력
    if episode % 50 == 0:
        recent_returns = returns_history[-50:]
        avg_return = np.mean(recent_returns)
        print(f"[Episode {episode:4d}] 최근 50 에피소드 평균 Return = {avg_return:.3f}, noise_std = {noise_std:.3f}")

print("\n=== DDPGfD 학습 종료 ===\n")


# ==============================
# 9. 학습된 Actor 정책으로 상태별 행동 확인
# ==============================
print("▶ 학습된 Actor의 결정론적 정책 μ(s) (상태별 연속 행동 값)")

for s in range(n_states):
    a_det = actor_forward(s, use_target=False)
    print(f"상태 {s}: 행동 a = {a_det:.4f}")

print("\n▶ Greedy 기준 이산 정책(연속 a의 부호 기준 ←/→)")
action_symbol = {0: "←", 1: "→"}

policy_str = ""
for s in range(n_states):
    if s == n_states - 1:
        # 목표 상태는 G 로 표기
        policy_str += " G "
    else:
        a_det = actor_forward(s, use_target=False)
        env_action = 1 if a_det > 0.0 else 0
        policy_str += f" {action_symbol[env_action]} "

print("상태 0  1  2  3  4")
print("     " + policy_str)


# ==============================
# 10. 학습된 결정론적 정책으로 1회 에피소드 실행
# ==============================
print("\n▶ 학습된 결정론적 정책(DDPGfD)으로 1회 에피소드 실행 예시")

state = reset()
trajectory = [state]

for step_idx in range(max_steps):
    # Actor의 결정론적 행동 계산
    a_det = actor_forward(state, use_target=False)
    env_action = 1 if a_det > 0.0 else 0

    next_state, reward, done = step(state, env_action)
    trajectory.append(next_state)
    state = next_state

    if done:
        break

print("방문한 상태들:", trajectory)
print("스텝 수:", len(trajectory) - 1)
print("마지막 상태가 목표(4)면 학습 성공!")


=== 전문가 Demonstration 수집 및 리플레이 버퍼 프리필 시작 ===
전문가 Demonstration으로 프리필된 transition 수: 120
=== Demonstration 프리필 종료 ===

=== 1차원 선형 월드에서의 DDPGfD(DDPG from Demonstrations, NumPy) 학습 시작 ===
[Episode   50] 최근 50 에피소드 평균 Return = 0.907, noise_std = 0.233
[Episode  100] 최근 50 에피소드 평균 Return = 0.970, noise_std = 0.182
[Episode  150] 최근 50 에피소드 평균 Return = 0.970, noise_std = 0.141
[Episode  200] 최근 50 에피소드 평균 Return = 0.970, noise_std = 0.110
[Episode  250] 최근 50 에피소드 평균 Return = 0.970, noise_std = 0.086
[Episode  300] 최근 50 에피소드 평균 Return = 0.970, noise_std = 0.067
[Episode  350] 최근 50 에피소드 평균 Return = 0.970, noise_std = 0.052
[Episode  400] 최근 50 에피소드 평균 Return = 0.970, noise_std = 0.050
[Episode  450] 최근 50 에피소드 평균 Return = 0.970, noise_std = 0.050
[Episode  500] 최근 50 에피소드 평균 Return = 0.970, noise_std = 0.050

=== DDPGfD 학습 종료 ===

▶ 학습된 Actor의 결정론적 정책 μ(s) (상태별 연속 행동 값)
상태 0: 행동 a = 0.8633
상태 1: 행동 a = 0.8625
상태 2: 행동 a = 0.8588
상태 3: 행동 a = 0.8511
상태 4: 행동 a = 0.7715

▶ Greedy 기준 이산 정책(연속

In [30]:
###################################################################
##
###################################################################